<a href="https://colab.research.google.com/github/CGMMAPPING/scriptoqtemplate/blob/master/Script_Assure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notes


Overview:
- Uses version 3 of the [Display & Video 360 API](https://developers.google.com/display-video/api/reference/rest/v3)

Updates & Future work:
- Current code request ignore multi page responses, taking the first page only. This needs to be updated taking the nextPageToken if present in the response and re-requesting for that page, then again if needed.

# Filter - Yet to be put in place

In [ ]:
# Filter_Partner = [1407943]
# Filter_Advertiser = [3337341]
# Filter_Campaign = [54817423]
# Filter_InsertionOrder = [1018056021]
# Filter_LineItem = [21670600270]

# Codebase set up

## Import required packages

In [3]:
import pandas as pd
import numpy as np
import os
import ast
import math
import time
from requests.exceptions import SSLError
from pandas import json_normalize
from google.colab import files
from googleapiclient.errors import HttpError
from itertools import zip_longest

## Imports required for API

In [4]:
import json
from google.oauth2 import service_account
service_account_info = json.load(open('KEYFILE.json'))
credentials = service_account.Credentials.from_service_account_info(service_account_info)
project_id = 'media-audit-platform'
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient import discovery

## Build the DV360 discovery URL

In [5]:
discovery_url = f'https://displayvideo.googleapis.com/$discovery/rest?version=v3'
service = discovery.build(
    'displayvideo',
    'v3',
    discoveryServiceUrl=discovery_url,
    credentials=credentials)

# List all Partners, Advertisers, Campaigns, Insertion Orders, Line Items and AdGroups

## List the Partners the Service Account / User has access to

- Create the Partner list dataframe
  - dataFrame_Partners_list
- List Advertisers
  - partnerIdList

Using: [partners.list](https://developers.google.com/display-video/api/reference/rest/v3/partners/list)

In [6]:
# Initialize the DataFrame with the specified structure
dataFrame_Partners_List = pd.DataFrame({
    'name': pd.Series(dtype='str'),
    'partnerId': pd.Series(dtype='int'),
    'updateTime': pd.Series(dtype='datetime64[ns]'),
    'displayName': pd.Series(dtype='str'),
    'entityStatus': pd.Series(dtype='str'),
    'generalConfig.timeZone': pd.Series(dtype='str'),
    'generalConfig.currencyCode': pd.Series(dtype='str'),
    'adServerConfig.measurementConfig.dv360ToCmCostReportingEnabled': pd.Series(dtype='bool'),
    'adServerConfig.measurementConfig.dv360ToCmDataSharingEnabled': pd.Series(dtype='bool'),
    'dataAccessConfig.sdfConfig.version': pd.Series(dtype='str'),
    'dataAccessConfig.sdfConfig.adminEmail': pd.Series(dtype='str'),
    'exchangeConfig.enabledExchanges': pd.Series(dtype='str'),
    'billingConfig.billingProfileId': pd.Series(dtype='str')
})

# Prepare a list to collect all partner data
list_Partner = []

# Request data from the API
request = service.partners().list()
response = request.execute()

# Check if the 'partners' key is in the response
if 'partners' in response:
    response_PartnerList = pd.json_normalize(response['partners'])

    # Ensure the DataFrame has the same columns as your structure
    for column in dataFrame_Partners_List.columns:
        if column not in response_PartnerList.columns:
            response_PartnerList[column] = None  # Fill missing columns with None

    # Append the data to the list
    list_Partner.extend(response_PartnerList.to_dict('records'))

# Convert the list to a DataFrame and append to the existing DataFrame
new_dataFrame = pd.DataFrame(list_Partner)
dataFrame_Partners_List = pd.concat([dataFrame_Partners_List, new_dataFrame], ignore_index=True)

# Extract the partner IDs for further use
partnerIdList = dataFrame_Partners_List['partnerId'].tolist()

# Apply the 'get' function for each column to ensure all data is properly aligned
dataFrame_Partners_List['name'] = dataFrame_Partners_List.get('name')
dataFrame_Partners_List['partnerId'] = dataFrame_Partners_List.get('partnerId')
dataFrame_Partners_List['updateTime'] = dataFrame_Partners_List.get('updateTime')
dataFrame_Partners_List['displayName'] = dataFrame_Partners_List.get('displayName')
dataFrame_Partners_List['entityStatus'] = dataFrame_Partners_List.get('entityStatus')
dataFrame_Partners_List['generalConfig.timeZone'] = dataFrame_Partners_List.get('generalConfig.timeZone')
dataFrame_Partners_List['generalConfig.currencyCode'] = dataFrame_Partners_List.get('generalConfig.currencyCode')
dataFrame_Partners_List['adServerConfig.measurementConfig.dv360ToCmCostReportingEnabled'] = dataFrame_Partners_List.get('adServerConfig.measurementConfig.dv360ToCmCostReportingEnabled')
dataFrame_Partners_List['adServerConfig.measurementConfig.dv360ToCmDataSharingEnabled'] = dataFrame_Partners_List.get('adServerConfig.measurementConfig.dv360ToCmDataSharingEnabled')
dataFrame_Partners_List['dataAccessConfig.sdfConfig.version'] = dataFrame_Partners_List.get('dataAccessConfig.sdfConfig.version')
dataFrame_Partners_List['dataAccessConfig.sdfConfig.adminEmail'] = dataFrame_Partners_List.get('dataAccessConfig.sdfConfig.adminEmail')
dataFrame_Partners_List['exchangeConfig.enabledExchanges'] = dataFrame_Partners_List.get('exchangeConfig.enabledExchanges')
dataFrame_Partners_List['billingConfig.billingProfileId'] = dataFrame_Partners_List.get('billingConfig.billingProfileId')

columns_order = [
    'name',
    'partnerId',
    'updateTime',
    'displayName',
    'entityStatus',
    'generalConfig.timeZone',
    'generalConfig.currencyCode',
    'adServerConfig.measurementConfig.dv360ToCmCostReportingEnabled',
    'adServerConfig.measurementConfig.dv360ToCmDataSharingEnabled',
    'dataAccessConfig.sdfConfig.version',
    'dataAccessConfig.sdfConfig.adminEmail',
    'exchangeConfig.enabledExchanges',
    'billingConfig.billingProfileId'
]

# Reorder the DataFrame's columns
dataFrame_Partners_List = dataFrame_Partners_List[columns_order]

## List the Advertisers under each Partner

- Create the Advertiser list dataframe
  - dataFrame_Advertisers_list
- List Advertisers
  - advertiserIdList

Using: [advertisers.list](https://developers.google.com/display-video/api/reference/rest/v3/advertisers/list)

In [7]:
# Initialize the DataFrame with the specified structure
dataFrame_Advertisers_List = pd.DataFrame({
    'name': pd.Series(dtype='str'),
    'advertiserId': pd.Series(dtype='int'),
    'partnerId': pd.Series(dtype='int'),
    'displayName': pd.Series(dtype='str'),
    'entityStatus': pd.Series(dtype='str'),
    'updateTime': pd.Series(dtype='datetime64[ns]'),
    'generalConfig.domainUrl': pd.Series(dtype='str'),
    'generalConfig.timeZone': pd.Series(dtype='str'),
    'generalConfig.currencyCode': pd.Series(dtype='str'),
    'adServerConfig.thirdPartyOnlyConfig.pixelOrderIdReportingEnabled': pd.Series(dtype='bool'),
    'adServerConfig.cmHybridConfig.cmSyncableSiteIds': pd.Series(dtype='object'),
    'adServerConfig.cmHybridConfig.dv360ToCmDataSharingEnabled': pd.Series(dtype='bool'),
    'adServerConfig.cmHybridConfig.dv360ToCmCostReportingEnabled': pd.Series(dtype='bool'),
    'adServerConfig.cmHybridConfig.cmFloodlightLinkingAuthorized': pd.Series(dtype='bool'),
    'adServerConfig.cmHybridConfig.cmAdvertiserIds': pd.Series(dtype='object'),
    'creativeConfig.iasClientId': pd.Series(dtype='str'),
    'creativeConfig.obaComplianceDisabled': pd.Series(dtype='bool'),
    'creativeConfig.dynamicCreativeEnabled': pd.Series(dtype='bool'),
    'creativeConfig.videoCreativeDataSharingAuthorized': pd.Series(dtype='bool'),
    'dataAccessConfig': pd.Series(dtype='str'),
    'integrationDetails.integrationCode': pd.Series(dtype='str'),
    'integrationDetails.details': pd.Series(dtype='str'),
    'servingConfig.exemptTvFromViewabilityTargeting': pd.Series(dtype='bool'),
    'prismaEnabled': pd.Series(dtype='bool'),
    'billingConfig.billingProfileId': pd.Series(dtype='str')
})

list_Advertiser = []

# Loop through each partner ID and fetch the associated advertisers
for partner in dataFrame_Partners_List["partnerId"]:
    request = service.advertisers().list(partnerId=partner)
    response = request.execute()

    # Check if 'advertisers' key exists in the response
    if 'advertisers' in response:
        response_AdvertiserList = pd.json_normalize(response['advertisers'])

        # Ensure the DataFrame has the same columns as your structure
        for column in dataFrame_Advertisers_List.columns:
            if column not in response_AdvertiserList.columns:
                response_AdvertiserList[column] = None  # Fill missing columns with None

        # Append the data to the list
        list_Advertiser.extend(response_AdvertiserList.to_dict('records'))

# Convert the list to a DataFrame
new_dataFrame = pd.DataFrame(list_Advertiser)
dataFrame_Advertisers_List = pd.concat([dataFrame_Advertisers_List, new_dataFrame], ignore_index=True)

# Extract the advertiser IDs for further use
advertiserIdList = dataFrame_Advertisers_List['advertiserId'].tolist()

# Apply the 'get' function for each column to ensure all data is properly aligned
dataFrame_Advertisers_List['name'] = dataFrame_Advertisers_List.get('name')
dataFrame_Advertisers_List['advertiserId'] = dataFrame_Advertisers_List.get('advertiserId')
dataFrame_Advertisers_List['partnerId'] = dataFrame_Advertisers_List.get('partnerId')
dataFrame_Advertisers_List['displayName'] = dataFrame_Advertisers_List.get('displayName')
dataFrame_Advertisers_List['entityStatus'] = dataFrame_Advertisers_List.get('entityStatus')
dataFrame_Advertisers_List['updateTime'] = dataFrame_Advertisers_List.get('updateTime')
dataFrame_Advertisers_List['generalConfig.domainUrl'] = dataFrame_Advertisers_List.get('generalConfig.domainUrl')
dataFrame_Advertisers_List['generalConfig.timeZone'] = dataFrame_Advertisers_List.get('generalConfig.timeZone')
dataFrame_Advertisers_List['generalConfig.currencyCode'] = dataFrame_Advertisers_List.get('generalConfig.currencyCode')
dataFrame_Advertisers_List['adServerConfig.thirdPartyOnlyConfig.pixelOrderIdReportingEnabled'] = dataFrame_Advertisers_List.get('adServerConfig.thirdPartyOnlyConfig.pixelOrderIdReportingEnabled')
dataFrame_Advertisers_List['adServerConfig.cmHybridConfig.cmSyncableSiteIds'] = dataFrame_Advertisers_List.get('adServerConfig.cmHybridConfig.cmSyncableSiteIds')
dataFrame_Advertisers_List['adServerConfig.cmHybridConfig.dv360ToCmDataSharingEnabled'] = dataFrame_Advertisers_List.get('adServerConfig.cmHybridConfig.dv360ToCmDataSharingEnabled')
dataFrame_Advertisers_List['adServerConfig.cmHybridConfig.dv360ToCmCostReportingEnabled'] = dataFrame_Advertisers_List.get('adServerConfig.cmHybridConfig.dv360ToCmCostReportingEnabled')
dataFrame_Advertisers_List['adServerConfig.cmHybridConfig.cmFloodlightLinkingAuthorized'] = dataFrame_Advertisers_List.get('adServerConfig.cmHybridConfig.cmFloodlightLinkingAuthorized')
dataFrame_Advertisers_List['adServerConfig.cmHybridConfig.cmAdvertiserIds'] = dataFrame_Advertisers_List.get('adServerConfig.cmHybridConfig.cmAdvertiserIds')
dataFrame_Advertisers_List['creativeConfig.iasClientId'] = dataFrame_Advertisers_List.get('creativeConfig.iasClientId')
dataFrame_Advertisers_List['creativeConfig.obaComplianceDisabled'] = dataFrame_Advertisers_List.get('creativeConfig.obaComplianceDisabled')
dataFrame_Advertisers_List['creativeConfig.dynamicCreativeEnabled'] = dataFrame_Advertisers_List.get('creativeConfig.dynamicCreativeEnabled')
dataFrame_Advertisers_List['creativeConfig.videoCreativeDataSharingAuthorized'] = dataFrame_Advertisers_List.get('creativeConfig.videoCreativeDataSharingAuthorized')
dataFrame_Advertisers_List['dataAccessConfig'] = dataFrame_Advertisers_List.get('dataAccessConfig')
dataFrame_Advertisers_List['integrationDetails.integrationCode'] = dataFrame_Advertisers_List.get('integrationDetails.integrationCode')
dataFrame_Advertisers_List['integrationDetails.details'] = dataFrame_Advertisers_List.get('integrationDetails.details')
dataFrame_Advertisers_List['servingConfig.exemptTvFromViewabilityTargeting'] = dataFrame_Advertisers_List.get('servingConfig.exemptTvFromViewabilityTargeting')
dataFrame_Advertisers_List['prismaEnabled'] = dataFrame_Advertisers_List.get('prismaEnabled')
dataFrame_Advertisers_List['billingConfig.billingProfileId'] = dataFrame_Advertisers_List.get('billingConfig.billingProfileId')

columns_order = [
    'name',
    'advertiserId',
    'partnerId',
    'displayName',
    'entityStatus',
    'updateTime',
    'generalConfig.domainUrl',
    'generalConfig.timeZone',
    'generalConfig.currencyCode',
    'adServerConfig.thirdPartyOnlyConfig.pixelOrderIdReportingEnabled',
    'adServerConfig.cmHybridConfig.cmSyncableSiteIds',
    'adServerConfig.cmHybridConfig.dv360ToCmDataSharingEnabled',
    'adServerConfig.cmHybridConfig.dv360ToCmCostReportingEnabled',
    'adServerConfig.cmHybridConfig.cmFloodlightLinkingAuthorized',
    'adServerConfig.cmHybridConfig.cmAdvertiserIds',
    'creativeConfig.iasClientId',
    'creativeConfig.obaComplianceDisabled',
    'creativeConfig.dynamicCreativeEnabled',
    'creativeConfig.videoCreativeDataSharingAuthorized',
    'dataAccessConfig',
    'integrationDetails.integrationCode',
    'integrationDetails.details',
    'servingConfig.exemptTvFromViewabilityTargeting',
    'prismaEnabled',
    'billingConfig.billingProfileId'
]

# Reorder the DataFrame's columns
dataFrame_Advertisers_List = dataFrame_Advertisers_List[columns_order]

## List Campaigns under each Advertiser

- Create the Campaign list dataframe
  - dataFrame_Campaigns_List
- List Campaigns
  - campaignIdList

Using: [advertisers.campaigns.list](https://developers.google.com/display-video/api/reference/rest/v3/advertisers.campaigns/list)

In [8]:
# Initialize an empty DataFrame to store all campaigns across all advertisers
dataFrame_Campaigns_List = pd.DataFrame({
    'name': pd.Series(dtype='str'),
    'advertiserId': pd.Series(dtype='int'),
    'campaignId': pd.Series(dtype='int'),
    'displayName': pd.Series(dtype='str'),
    'entityStatus': pd.Series(dtype='str'),
    'updateTime': pd.Series(dtype='datetime64[ns]'),
    'campaignGoal.campaignGoalType': pd.Series(dtype='str'),
    'campaignGoal.performanceGoal.performanceGoalType': pd.Series(dtype='str'),
    'campaignGoal.performanceGoal.performanceGoalAmountMicros': pd.Series(dtype='int'),
    'campaignGoal.performanceGoal.performanceGoalPercentageMicros': pd.Series(dtype='int'),
    'campaignGoal.performanceGoal.performanceGoalString': pd.Series(dtype='str'),
    'campaignFlight': pd.Series(dtype='str'),
    'frequencyCap.unlimited': pd.Series(dtype='bool'),
    'frequencyCap.timeUnit': pd.Series(dtype='str'),
    'frequencyCap.timeUnitCount': pd.Series(dtype='int'),
    'frequencyCap.maxImpressions': pd.Series(dtype='int'),
    'campaignBudgets': pd.Series(dtype='str')
})

# Loop over each advertiser ID
for AID in advertiserIdList:
    try:
        # Request to list all campaigns under the current advertiser
        request = service.advertisers().campaigns().list(
            advertiserId=AID,
        )
        response = request.execute()
        campaignListResponse = pd.json_normalize(response.get('campaigns', []))

        # Check if there are campaigns to process
        if not campaignListResponse.empty:
            campaignListResponse['advertiserId'] = AID  # Add the advertiserId to the response

            # Append to the master DataFrame
            dataFrame_Campaigns_List = pd.concat([dataFrame_Campaigns_List, campaignListResponse], ignore_index=True)

    except Exception as e:
        print(f"An error occurred for advertiser ID {AID}: {e}")

# Generate the list of campaign IDs from the combined DataFrame
campaignIdList = dataFrame_Campaigns_List['campaignId'].tolist()

# After the loop, extract the necessary columns from the combined DataFrame
dataFrame_Campaigns_List['name'] = dataFrame_Campaigns_List.get('name')
dataFrame_Campaigns_List['advertiserId'] = dataFrame_Campaigns_List.get('advertiserId')
dataFrame_Campaigns_List['campaignId'] = dataFrame_Campaigns_List.get('campaignId')
dataFrame_Campaigns_List['displayName'] = dataFrame_Campaigns_List.get('displayName')
dataFrame_Campaigns_List['entityStatus'] = dataFrame_Campaigns_List.get('entityStatus')
dataFrame_Campaigns_List['updateTime'] = dataFrame_Campaigns_List.get('updateTime')
dataFrame_Campaigns_List['campaignGoal.campaignGoalType'] = dataFrame_Campaigns_List.get('campaignGoal.campaignGoalType')
dataFrame_Campaigns_List['campaignGoal.performanceGoal.performanceGoalType'] = dataFrame_Campaigns_List.get('campaignGoal.performanceGoal.performanceGoalType')
dataFrame_Campaigns_List['campaignGoal.performanceGoal.performanceGoalAmountMicros'] = dataFrame_Campaigns_List.get('campaignGoal.performanceGoal.performanceGoalAmountMicros')
dataFrame_Campaigns_List['campaignGoal.performanceGoal.performanceGoalPercentageMicros'] = dataFrame_Campaigns_List.get('campaignGoal.performanceGoal.performanceGoalPercentageMicros')
dataFrame_Campaigns_List['campaignGoal.performanceGoal.performanceGoalString'] = dataFrame_Campaigns_List.get('campaignGoal.performanceGoal.performanceGoalString')
dataFrame_Campaigns_List['campaignFlight'] = dataFrame_Campaigns_List.get('campaignFlight')
dataFrame_Campaigns_List['frequencyCap.unlimited'] = dataFrame_Campaigns_List.get('frequencyCap.unlimited')
dataFrame_Campaigns_List['frequencyCap.timeUnit'] = dataFrame_Campaigns_List.get('frequencyCap.timeUnit')
dataFrame_Campaigns_List['frequencyCap.timeUnitCount'] = dataFrame_Campaigns_List.get('frequencyCap.timeUnitCount')
dataFrame_Campaigns_List['frequencyCap.maxImpressions'] = dataFrame_Campaigns_List.get('frequencyCap.maxImpressions')
dataFrame_Campaigns_List['campaignBudgets'] = dataFrame_Campaigns_List.get('campaignBudgets')

columns_order = [
    'name',
    'advertiserId',
    'campaignId',
    'displayName',
    'entityStatus',
    'updateTime',
    'campaignGoal.campaignGoalType',
    'campaignGoal.performanceGoal.performanceGoalType',
    'campaignGoal.performanceGoal.performanceGoalAmountMicros',
    'campaignGoal.performanceGoal.performanceGoalPercentageMicros',
    'campaignGoal.performanceGoal.performanceGoalString',
    'campaignFlight',
    'frequencyCap.unlimited',
    'frequencyCap.timeUnit',
    'frequencyCap.timeUnitCount',
    'frequencyCap.maxImpressions',
    'campaignBudgets'
]

# Reorder the DataFrame's columns
dataFrame_Campaigns_List = dataFrame_Campaigns_List[columns_order]

## List Insertion Orders under each Advertiser

- Create the Insertion Order list dataframe
  - dataFrame_InsertionOrder_List
- List Insertion Orders
  - insertionOrderIdList

Using: [advertisers.insertionOrders.list](https://developers.google.com/display-video/api/reference/rest/v3/advertisers.insertionOrders/list)

In [9]:
dataFrame_InsertionOrders_List = pd.DataFrame({
    'name': pd.Series(dtype='str'),
    'advertiserId': pd.Series(dtype='int'),
    'campaignId': pd.Series(dtype='int'),
    'insertionOrderId': pd.Series(dtype='int'),
    'displayName': pd.Series(dtype='str'),
    'insertionOrderType': pd.Series(dtype='str'),
    'entityStatus': pd.Series(dtype='str'),
    'updateTime': pd.Series(dtype='datetime64[ns]'),
    'partnerCosts.costType': pd.Series(dtype='str'),
    'partnerCosts.feeType': pd.Series(dtype='str'),
    'partnerCosts.invoiceType': pd.Series(dtype='str'),
    'partnerCosts.feeAmount': pd.Series(dtype='int'),
    'partnerCosts.feePercentageMillis': pd.Series(dtype='int'),
    'pacing.pacingPeriod': pd.Series(dtype='str'),
    'pacing.pacingType': pd.Series(dtype='str'),
    'pacing.dailyMaxMicros': pd.Series(dtype='int'),
    'pacing.dailyMaxImpressions': pd.Series(dtype='int'),
    'frequencyCap.unlimited': pd.Series(dtype='bool'),
    'frequencyCap.timeUnit': pd.Series(dtype='str'),
    'frequencyCap.timeUnitCount': pd.Series(dtype='int'),
    'frequencyCap.maxImpressions': pd.Series(dtype='int'),
    'integrationDetails.integrationCode': pd.Series(dtype='str'),
    'integrationDetails.details': pd.Series(dtype='int'),
    'performanceGoal.performanceGoalType': pd.Series(dtype='str'),
    'performanceGoal.performanceGoalAmountMicros': pd.Series(dtype='int'),
    'performanceGoal.performanceGoalPercentageMicros': pd.Series(dtype='int'),
    'performanceGoal.performanceGoalString': pd.Series(dtype='str'),
    'insertionOrderBudget': pd.Series(dtype='str'),
    'bidStrategy.fixedBid.bidAmountMicros': pd.Series(dtype='int'),
    'bidStrategy.maximizeSpendAutoBid.performanceGoalType': pd.Series(dtype='str'),
    'bidStrategy.maximizeSpendAutoBid.maxAverageCpmBidAmountMicros': pd.Series(dtype='int'),
    'bidStrategy.maximizeSpendAutoBid.raiseBidForDeals': pd.Series(dtype='bool'),
    'bidStrategy.maximizeSpendAutoBid.customBiddingAlgorithmId': pd.Series(dtype='str'),
    'bidStrategy.performanceGoalAutoBid.performanceGoalType': pd.Series(dtype='str'),
    'bidStrategy.performanceGoalAutoBid.performanceGoalAmountMicros': pd.Series(dtype='int'),
    'bidStrategy.performanceGoalAutoBid.maxAverageCpmBidAmountMicros': pd.Series(dtype='int'),
    'bidStrategy.performanceGoalAutoBid.customBiddingAlgorithmId': pd.Series(dtype='str'),
    'reservationType': pd.Series(dtype='str'),
    'billableOutcome': pd.Series(dtype='str')
})

# Loop over each advertiser ID
for AID in advertiserIdList:
    try:
        # Request to list all insertion orders under the current advertiser
        request = service.advertisers().insertionOrders().list(
            advertiserId=AID,
        )
        response = request.execute()
        insertionOrderListResponse = pd.json_normalize(response.get('insertionOrders', []))

        # Check if there are insertion orders to process
        if not insertionOrderListResponse.empty:
            insertionOrderListResponse['advertiserId'] = AID  # Add the advertiserId to the response

            # Append to the master DataFrame
            dataFrame_InsertionOrders_List = pd.concat([dataFrame_InsertionOrders_List, insertionOrderListResponse], ignore_index=True)

    except Exception as e:
        print(f"An error occurred for advertiser ID {AID}: {e}")

# Generate the list of insertionOrder IDs from the combined DataFrame
insertionOrderIdList = dataFrame_InsertionOrders_List['insertionOrderId'].tolist()

# After the loop, extract the necessary columns from the combined DataFrame
dataFrame_InsertionOrders_List['name'] = dataFrame_InsertionOrders_List.get('name')
dataFrame_InsertionOrders_List['advertiserId'] = dataFrame_InsertionOrders_List.get('advertiserId')
dataFrame_InsertionOrders_List['campaignId'] = dataFrame_InsertionOrders_List.get('campaignId')
dataFrame_InsertionOrders_List['insertionOrderId'] = dataFrame_InsertionOrders_List.get('insertionOrderId')
dataFrame_InsertionOrders_List['displayName'] = dataFrame_InsertionOrders_List.get('displayName')
dataFrame_InsertionOrders_List['insertionOrderType'] = dataFrame_InsertionOrders_List.get('insertionOrderType')
dataFrame_InsertionOrders_List['entityStatus'] = dataFrame_InsertionOrders_List.get('entityStatus')
dataFrame_InsertionOrders_List['updateTime'] = dataFrame_InsertionOrders_List.get('updateTime')
dataFrame_InsertionOrders_List['partnerCosts.costType'] = dataFrame_InsertionOrders_List.get('partnerCosts.costType')
dataFrame_InsertionOrders_List['partnerCosts.feeType'] = dataFrame_InsertionOrders_List.get('partnerCosts.feeType')
dataFrame_InsertionOrders_List['partnerCosts.invoiceType'] = dataFrame_InsertionOrders_List.get('partnerCosts.invoiceType')
dataFrame_InsertionOrders_List['partnerCosts.feeAmount'] = dataFrame_InsertionOrders_List.get('partnerCosts.feeAmount')
dataFrame_InsertionOrders_List['partnerCosts.feePercentageMillis'] = dataFrame_InsertionOrders_List.get('partnerCosts.feePercentageMillis')
dataFrame_InsertionOrders_List['pacing.pacingPeriod'] = dataFrame_InsertionOrders_List.get('pacing.pacingPeriod')
dataFrame_InsertionOrders_List['pacing.pacingType'] = dataFrame_InsertionOrders_List.get('pacing.pacingType')
dataFrame_InsertionOrders_List['pacing.dailyMaxMicros'] = dataFrame_InsertionOrders_List.get('pacing.dailyMaxMicros')
dataFrame_InsertionOrders_List['pacing.dailyMaxImpressions'] = dataFrame_InsertionOrders_List.get('pacing.dailyMaxImpressions')
dataFrame_InsertionOrders_List['frequencyCap.unlimited'] = dataFrame_InsertionOrders_List.get('frequencyCap.unlimited')
dataFrame_InsertionOrders_List['frequencyCap.timeUnit'] = dataFrame_InsertionOrders_List.get('frequencyCap.timeUnit')
dataFrame_InsertionOrders_List['frequencyCap.timeUnitCount'] = dataFrame_InsertionOrders_List.get('frequencyCap.timeUnitCount')
dataFrame_InsertionOrders_List['frequencyCap.maxImpressions'] = dataFrame_InsertionOrders_List.get('frequencyCap.maxImpressions')
dataFrame_InsertionOrders_List['integrationDetails.integrationCode'] = dataFrame_InsertionOrders_List.get('integrationDetails.integrationCode')
dataFrame_InsertionOrders_List['integrationDetails.details'] = dataFrame_InsertionOrders_List.get('integrationDetails.details')
dataFrame_InsertionOrders_List['performanceGoal.performanceGoalType'] = dataFrame_InsertionOrders_List.get('performanceGoal.performanceGoalType')
dataFrame_InsertionOrders_List['performanceGoal.performanceGoalAmountMicros'] = dataFrame_InsertionOrders_List.get('performanceGoal.performanceGoalAmountMicros')
dataFrame_InsertionOrders_List['performanceGoal.performanceGoalPercentageMicros'] = dataFrame_InsertionOrders_List.get('performanceGoal.performanceGoalPercentageMicros')
dataFrame_InsertionOrders_List['performanceGoal.performanceGoalString'] = dataFrame_InsertionOrders_List.get('performanceGoal.performanceGoalString')
dataFrame_InsertionOrders_List['insertionOrderBudget'] = dataFrame_InsertionOrders_List.get('insertionOrderBudget')
dataFrame_InsertionOrders_List['bidStrategy.fixedBid.bidAmountMicros'] = dataFrame_InsertionOrders_List.get('bidStrategy.fixedBid.bidAmountMicros')
dataFrame_InsertionOrders_List['bidStrategy.maximizeSpendAutoBid.performanceGoalType'] = dataFrame_InsertionOrders_List.get('bidStrategy.maximizeSpendAutoBid.performanceGoalType')
dataFrame_InsertionOrders_List['bidStrategy.maximizeSpendAutoBid.maxAverageCpmBidAmountMicros'] = dataFrame_InsertionOrders_List.get('bidStrategy.maximizeSpendAutoBid.maxAverageCpmBidAmountMicros')
dataFrame_InsertionOrders_List['bidStrategy.maximizeSpendAutoBid.raiseBidForDeals'] = dataFrame_InsertionOrders_List.get('bidStrategy.maximizeSpendAutoBid.raiseBidForDeals')
dataFrame_InsertionOrders_List['bidStrategy.maximizeSpendAutoBid.customBiddingAlgorithmId'] = dataFrame_InsertionOrders_List.get('bidStrategy.maximizeSpendAutoBid.customBiddingAlgorithmId')
dataFrame_InsertionOrders_List['bidStrategy.performanceGoalAutoBid.performanceGoalType'] = dataFrame_InsertionOrders_List.get('bidStrategy.performanceGoalAutoBid.performanceGoalType')
dataFrame_InsertionOrders_List['bidStrategy.performanceGoalAutoBid.performanceGoalAmountMicros'] = dataFrame_InsertionOrders_List.get('bidStrategy.performanceGoalAutoBid.performanceGoalAmountMicros')
dataFrame_InsertionOrders_List['bidStrategy.performanceGoalAutoBid.maxAverageCpmBidAmountMicros'] = dataFrame_InsertionOrders_List.get('bidStrategy.performanceGoalAutoBid.maxAverageCpmBidAmountMicros')
dataFrame_InsertionOrders_List['bidStrategy.performanceGoalAutoBid.customBiddingAlgorithmId'] = dataFrame_InsertionOrders_List.get('bidStrategy.performanceGoalAutoBid.customBiddingAlgorithmId')
dataFrame_InsertionOrders_List['reservationType'] = dataFrame_InsertionOrders_List.get('reservationType')
dataFrame_InsertionOrders_List['billableOutcome'] = dataFrame_InsertionOrders_List.get('billableOutcome')

columns_order = [
    'name',
    'advertiserId',
    'campaignId',
    'insertionOrderId',
    'displayName',
    'insertionOrderType',
    'entityStatus',
    'updateTime',
    'partnerCosts.costType',
    'partnerCosts.feeType',
    'partnerCosts.invoiceType',
    'partnerCosts.feeAmount',
    'partnerCosts.feePercentageMillis',
    'pacing.pacingPeriod',
    'pacing.pacingType',
    'pacing.dailyMaxMicros',
    'pacing.dailyMaxImpressions',
    'frequencyCap.unlimited',
    'frequencyCap.timeUnit',
    'frequencyCap.timeUnitCount',
    'frequencyCap.maxImpressions',
    'integrationDetails.integrationCode',
    'integrationDetails.details',
    'performanceGoal.performanceGoalType',
    'performanceGoal.performanceGoalAmountMicros',
    'performanceGoal.performanceGoalPercentageMicros',
    'performanceGoal.performanceGoalString',
    'insertionOrderBudget',
    'bidStrategy.fixedBid.bidAmountMicros',
    'bidStrategy.maximizeSpendAutoBid.performanceGoalType',
    'bidStrategy.maximizeSpendAutoBid.maxAverageCpmBidAmountMicros',
    'bidStrategy.maximizeSpendAutoBid.raiseBidForDeals',
    'bidStrategy.maximizeSpendAutoBid.customBiddingAlgorithmId',
    'bidStrategy.performanceGoalAutoBid.performanceGoalType',
    'bidStrategy.performanceGoalAutoBid.performanceGoalAmountMicros',
    'bidStrategy.performanceGoalAutoBid.maxAverageCpmBidAmountMicros',
    'bidStrategy.performanceGoalAutoBid.customBiddingAlgorithmId',
    'reservationType',
    'billableOutcome'
]

# Reorder the DataFrame's columns
dataFrame_InsertionOrders_List = dataFrame_InsertionOrders_List[columns_order]

## List Line Items under each Advertiser

- Create the Line Item list dataframe
  - dataFrame_LineItems_List
- List Line Items
  - lineItemIdList

Using: [advertisers.insertionOrders.list](https://developers.google.com/display-video/api/reference/rest/v3/advertisers.insertionOrders/list)

In [10]:
dataFrame_LineItems_List = pd.DataFrame({
    'name': pd.Series(dtype='str'),
    'advertiserId': pd.Series(dtype='int'),
    'campaignId': pd.Series(dtype='int'),
    'insertionOrderId': pd.Series(dtype='int'),
    'lineItemId': pd.Series(dtype='int'),
    'displayName': pd.Series(dtype='str'),
    'lineItemType': pd.Series(dtype='str'),
    'entityStatus': pd.Series(dtype='str'),
    'updateTime': pd.Series(dtype='datetime64[ns]'),
    'partnerCosts': pd.Series(dtype='object'),
    'flight.flightDateType': pd.Series(dtype='str'),
    'flight.dateRange.startDate.year': pd.Series(dtype='int'),
    'flight.dateRange.startDate.month': pd.Series(dtype='int'),
    'flight.dateRange.startDate.day': pd.Series(dtype='int'),
    'flight.dateRange.endDate.year': pd.Series(dtype='int'),
    'flight.dateRange.endDate.month': pd.Series(dtype='int'),
    'flight.dateRange.endDate.day': pd.Series(dtype='int'),
    'budget.budgetAllocationType': pd.Series(dtype='str'),
    'budget.budgetUnit': pd.Series(dtype='str'),
    'budget.maxAmount': pd.Series(dtype='int'),
    'pacing.pacingPeriod': pd.Series(dtype='str'),
    'pacing.pacingType': pd.Series(dtype='str'),
    'pacing.dailyMaxMicros': pd.Series(dtype='int'),
    'pacing.dailyMaxImpressions': pd.Series(dtype='int'),
    'frequencyCap.unlimited': pd.Series(dtype='bool'),
    'frequencyCap.timeUnit': pd.Series(dtype='str'),
    'frequencyCap.timeUnitCount': pd.Series(dtype='int'),
    'frequencyCap.maxImpressions': pd.Series(dtype='int'),
    'partnerRevenueModel.markupType': pd.Series(dtype='str'),
    'partnerRevenueModel.markupAmount': pd.Series(dtype='int'),
    'conversionCounting.postViewCountPercentageMillis': pd.Series(dtype='int'),
    'conversionCounting.floodlightActivityConfigs': pd.Series(dtype='object'),
    'creativeIds': pd.Series(dtype='str'),
    'bidStrategy.fixedBid.bidAmountMicros': pd.Series(dtype='int'),
    'bidStrategy.maximizeSpendAutoBid.performanceGoalType': pd.Series(dtype='str'),
    'bidStrategy.maximizeSpendAutoBid.maxAverageCpmBidAmountMicros': pd.Series(dtype='int'),
    'bidStrategy.maximizeSpendAutoBid.raiseBidForDeals': pd.Series(dtype='bool'),
    'bidStrategy.maximizeSpendAutoBid.customBiddingAlgorithmId': pd.Series(dtype='str'),
    'bidStrategy.performanceGoalAutoBid.performanceGoalType': pd.Series(dtype='str'),
    'bidStrategy.performanceGoalAutoBid.performanceGoalAmountMicros': pd.Series(dtype='int'),
    'bidStrategy.performanceGoalAutoBid.maxAverageCpmBidAmountMicros': pd.Series(dtype='int'),
    'bidStrategy.performanceGoalAutoBid.customBiddingAlgorithmId': pd.Series(dtype='str'),
    'bidStrategy.youtubeAndPartnersBid.type': pd.Series(dtype='str'),
    'bidStrategy.youtubeAndPartnersBid.value': pd.Series(dtype='int'),
    'bidStrategy.youtubeAndPartnersBid.adGroupEffectiveTargetCpaValue': pd.Series(dtype='int'),
    'bidStrategy.youtubeAndPartnersBid.adGroupEffectiveTargetCpaSource': pd.Series(dtype='str'),
    'integrationDetails.integrationCode': pd.Series(dtype='str'),
    'integrationDetails.details': pd.Series(dtype='int'),
    'targetingExpansion.enableOptimizedTargeting': pd.Series(dtype='bool'),
    'targetingExpansion.audienceExpansionSeedListExcluded': pd.Series(dtype='bool'),
    'targetingExpansion.audienceExpansionLevel': pd.Series(dtype='str'),
    'warningMessages': pd.Series(dtype='str'),
    'mobileApp.appId': pd.Series(dtype='str'),
    'mobileApp.platform': pd.Series(dtype='str'),
    'mobileApp.displayName': pd.Series(dtype='str'),
    'mobileApp.publisher': pd.Series(dtype='str'),
    'reservationType': pd.Series(dtype='str'),
    'excludeNewExchanges': pd.Series(dtype='bool'),
    'youtubeAndPartnersSettings.viewFrequencyCap.unlimited': pd.Series(dtype='bool'),
    'youtubeAndPartnersSettings.viewFrequencyCap.timeUnit': pd.Series(dtype='str'),
    'youtubeAndPartnersSettings.viewFrequencyCap.timeUnitCount': pd.Series(dtype='int'),
    'youtubeAndPartnersSettings.viewFrequencyCap.maxImpressions': pd.Series(dtype='int'),
    'youtubeAndPartnersSettings.viewFrequencyCap.maxViews': pd.Series(dtype='int'),
    'youtubeAndPartnersSettings.thirdPartyMeasurementConfigs.viewabilityVendorConfigs.vendor': pd.Series(dtype='str'),
    'youtubeAndPartnersSettings.thirdPartyMeasurementConfigs.viewabilityVendorConfigs.placementId': pd.Series(dtype='str'),
    'youtubeAndPartnersSettings.thirdPartyMeasurementConfigs.brandSafetyVendorConfigs.vendor': pd.Series(dtype='str'),
    'youtubeAndPartnersSettings.thirdPartyMeasurementConfigs.brandSafetyVendorConfigs.placementId': pd.Series(dtype='str'),
    'youtubeAndPartnersSettings.thirdPartyMeasurementConfigs.reachVendorConfigs.vendor': pd.Series(dtype='str'),
    'youtubeAndPartnersSettings.thirdPartyMeasurementConfigs.reachVendorConfigs.placementId': pd.Series(dtype='str'),
    'youtubeAndPartnersSettings.thirdPartyMeasurementConfigs.brandLiftVendorConfigs.vendor': pd.Series(dtype='str'),
    'youtubeAndPartnersSettings.thirdPartyMeasurementConfigs.brandLiftVendorConfigs.placementId': pd.Series(dtype='str'),
    'youtubeAndPartnersSettings.inventorySourceSettings.includeYoutube': pd.Series(dtype='bool'),
    'youtubeAndPartnersSettings.inventorySourceSettings.includeGoogleTv': pd.Series(dtype='bool'),
    'youtubeAndPartnersSettings.inventorySourceSettings.includeYoutubeVideoPartners': pd.Series(dtype='bool'),
    'youtubeAndPartnersSettings.contentCategory': pd.Series(dtype='str'),
    'youtubeAndPartnersSettings.effectiveContentCategory': pd.Series(dtype='str'),
    'youtubeAndPartnersSettings.targetFrequency.targetCount': pd.Series(dtype='str'),
    'youtubeAndPartnersSettings.targetFrequency.timeUnit': pd.Series(dtype='str'),
    'youtubeAndPartnersSettings.targetFrequency.timeUnitCount': pd.Series(dtype='int'),
    'youtubeAndPartnersSettings.linkedMerchantId': pd.Series(dtype='str'),
    'youtubeAndPartnersSettings.relatedVideoIds': pd.Series(dtype='object'),
    'youtubeAndPartnersSettings.leadFormId': pd.Series(dtype='str'),
    'youtubeAndPartnersSettings.videoAdSequenceSettings.minimumDuration': pd.Series(dtype='str'),
    'youtubeAndPartnersSettings.videoAdSequenceSettings.steps.stepId': pd.Series(dtype='int'),
    'youtubeAndPartnersSettings.videoAdSequenceSettings.steps.adGroupId': pd.Series(dtype='int'),
    'youtubeAndPartnersSettings.videoAdSequenceSettings.steps.previousStepId': pd.Series(dtype='int'),
    'youtubeAndPartnersSettings.videoAdSequenceSettings.steps.interactionType': pd.Series(dtype='str')
})

# Loop over each advertiser ID
for AID in advertiserIdList:
    try:
        # Request to list all insertion orders under the current advertiser
        request = service.advertisers().lineItems().list(
            advertiserId=AID,
        )
        response = request.execute()
        LineItemListResponse = pd.json_normalize(response.get('lineItems', []))

        # Check if there are insertion orders to process
        if not LineItemListResponse.empty:
            LineItemListResponse['advertiserId'] = AID  # Add the advertiserId to the response

            # Append to the master DataFrame
            dataFrame_LineItems_List = pd.concat([dataFrame_LineItems_List, LineItemListResponse], ignore_index=True)

    except Exception as e:
        print(f"An error occurred for advertiser ID {AID}: {e}")

# Extract the necessary columns from the combined DataFrame in the order they were created
dataFrame_LineItems_List['name'] = dataFrame_LineItems_List.get('name')
dataFrame_LineItems_List['advertiserId'] = dataFrame_LineItems_List.get('advertiserId')
dataFrame_LineItems_List['campaignId'] = dataFrame_LineItems_List.get('campaignId')
dataFrame_LineItems_List['insertionOrderId'] = dataFrame_LineItems_List.get('insertionOrderId')
dataFrame_LineItems_List['lineItemId'] = dataFrame_LineItems_List.get('lineItemId')
dataFrame_LineItems_List['displayName'] = dataFrame_LineItems_List.get('displayName')
dataFrame_LineItems_List['lineItemType'] = dataFrame_LineItems_List.get('lineItemType')
dataFrame_LineItems_List['entityStatus'] = dataFrame_LineItems_List.get('entityStatus')
dataFrame_LineItems_List['updateTime'] = dataFrame_LineItems_List.get('updateTime')

# Partner costs
dataFrame_LineItems_List['partnerCosts'] = dataFrame_LineItems_List.get('partnerCosts')

# Line item flight details
dataFrame_LineItems_List['flight.flightDateType'] = dataFrame_LineItems_List.get('flight.flightDateType')
dataFrame_LineItems_List['flight.dateRange.startDate.year'] = dataFrame_LineItems_List.get('flight.dateRange.startDate.year')
dataFrame_LineItems_List['flight.dateRange.startDate.month'] = dataFrame_LineItems_List.get('flight.dateRange.startDate.month')
dataFrame_LineItems_List['flight.dateRange.startDate.day'] = dataFrame_LineItems_List.get('flight.dateRange.startDate.day')
dataFrame_LineItems_List['flight.dateRange.endDate.year'] = dataFrame_LineItems_List.get('flight.dateRange.endDate.year')
dataFrame_LineItems_List['flight.dateRange.endDate.month'] = dataFrame_LineItems_List.get('flight.dateRange.endDate.month')
dataFrame_LineItems_List['flight.dateRange.endDate.day'] = dataFrame_LineItems_List.get('flight.dateRange.endDate.day')

# Line item budget details
dataFrame_LineItems_List['budget.budgetAllocationType'] = dataFrame_LineItems_List.get('budget.budgetAllocationType')
dataFrame_LineItems_List['budget.budgetUnit'] = dataFrame_LineItems_List.get('budget.budgetUnit')
dataFrame_LineItems_List['budget.maxAmount'] = dataFrame_LineItems_List.get('budget.maxAmount')

# Pacing details
dataFrame_LineItems_List['pacing.pacingPeriod'] = dataFrame_LineItems_List.get('pacing.pacingPeriod')
dataFrame_LineItems_List['pacing.pacingType'] = dataFrame_LineItems_List.get('pacing.pacingType')
dataFrame_LineItems_List['pacing.dailyMaxMicros'] = dataFrame_LineItems_List.get('pacing.dailyMaxMicros')
dataFrame_LineItems_List['pacing.dailyMaxImpressions'] = dataFrame_LineItems_List.get('pacing.dailyMaxImpressions')

# Frequency cap details
dataFrame_LineItems_List['frequencyCap.unlimited'] = dataFrame_LineItems_List.get('frequencyCap.unlimited')
dataFrame_LineItems_List['frequencyCap.timeUnit'] = dataFrame_LineItems_List.get('frequencyCap.timeUnit')
dataFrame_LineItems_List['frequencyCap.timeUnitCount'] = dataFrame_LineItems_List.get('frequencyCap.timeUnitCount')
dataFrame_LineItems_List['frequencyCap.maxImpressions'] = dataFrame_LineItems_List.get('frequencyCap.maxImpressions')

# Partner revenue model
dataFrame_LineItems_List['partnerRevenueModel.markupType'] = dataFrame_LineItems_List.get('partnerRevenueModel.markupType')
dataFrame_LineItems_List['partnerRevenueModel.markupAmount'] = dataFrame_LineItems_List.get('partnerRevenueModel.markupAmount')

# Conversion counting configuration
dataFrame_LineItems_List['conversionCounting.postViewCountPercentageMillis'] = dataFrame_LineItems_List.get('conversionCounting.postViewCountPercentageMillis')
dataFrame_LineItems_List['conversionCounting.floodlightActivityConfigs'] = dataFrame_LineItems_List.get('conversionCounting.floodlightActivityConfigs')

# Creative IDs
dataFrame_LineItems_List['creativeIds'] = dataFrame_LineItems_List.get('creativeIds')

# Bid strategy
dataFrame_LineItems_List['bidStrategy.fixedBid.bidAmountMicros'] = dataFrame_LineItems_List.get('bidStrategy.fixedBid.bidAmountMicros')
dataFrame_LineItems_List['bidStrategy.maximizeSpendAutoBid.performanceGoalType'] = dataFrame_LineItems_List.get('bidStrategy.maximizeSpendAutoBid.performanceGoalType')
dataFrame_LineItems_List['bidStrategy.maximizeSpendAutoBid.maxAverageCpmBidAmountMicros'] = dataFrame_LineItems_List.get('bidStrategy.maximizeSpendAutoBid.maxAverageCpmBidAmountMicros')
dataFrame_LineItems_List['bidStrategy.maximizeSpendAutoBid.raiseBidForDeals'] = dataFrame_LineItems_List.get('bidStrategy.maximizeSpendAutoBid.raiseBidForDeals')
dataFrame_LineItems_List['bidStrategy.maximizeSpendAutoBid.customBiddingAlgorithmId'] = dataFrame_LineItems_List.get('bidStrategy.maximizeSpendAutoBid.customBiddingAlgorithmId')
dataFrame_LineItems_List['bidStrategy.performanceGoalAutoBid.performanceGoalType'] = dataFrame_LineItems_List.get('bidStrategy.performanceGoalAutoBid.performanceGoalType')
dataFrame_LineItems_List['bidStrategy.performanceGoalAutoBid.performanceGoalAmountMicros'] = dataFrame_LineItems_List.get('bidStrategy.performanceGoalAutoBid.performanceGoalAmountMicros')
dataFrame_LineItems_List['bidStrategy.performanceGoalAutoBid.maxAverageCpmBidAmountMicros'] = dataFrame_LineItems_List.get('bidStrategy.performanceGoalAutoBid.maxAverageCpmBidAmountMicros')
dataFrame_LineItems_List['bidStrategy.performanceGoalAutoBid.customBiddingAlgorithmId'] = dataFrame_LineItems_List.get('bidStrategy.performanceGoalAutoBid.customBiddingAlgorithmId')
dataFrame_LineItems_List['bidStrategy.youtubeAndPartnersBid.type'] = dataFrame_LineItems_List.get('bidStrategy.youtubeAndPartnersBid.type')
dataFrame_LineItems_List['bidStrategy.youtubeAndPartnersBid.value'] = dataFrame_LineItems_List.get('bidStrategy.youtubeAndPartnersBid.value')
dataFrame_LineItems_List['bidStrategy.youtubeAndPartnersBid.adGroupEffectiveTargetCpaValue'] = dataFrame_LineItems_List.get('bidStrategy.youtubeAndPartnersBid.adGroupEffectiveTargetCpaValue')
dataFrame_LineItems_List['bidStrategy.youtubeAndPartnersBid.adGroupEffectiveTargetCpaSource'] = dataFrame_LineItems_List.get('bidStrategy.youtubeAndPartnersBid.adGroupEffectiveTargetCpaSource')

# Integration details
dataFrame_LineItems_List['integrationDetails.integrationCode'] = dataFrame_LineItems_List.get('integrationDetails.integrationCode')
dataFrame_LineItems_List['integrationDetails.details'] = dataFrame_LineItems_List.get('integrationDetails.details')

# Targeting expansion details
dataFrame_LineItems_List['targetingExpansion.enableOptimizedTargeting'] = dataFrame_LineItems_List.get('targetingExpansion.enableOptimizedTargeting')
dataFrame_LineItems_List['targetingExpansion.audienceExpansionSeedListExcluded'] = dataFrame_LineItems_List.get('targetingExpansion.audienceExpansionSeedListExcluded')
dataFrame_LineItems_List['targetingExpansion.audienceExpansionLevel'] = dataFrame_LineItems_List.get('targetingExpansion.audienceExpansionLevel')

# Warning messages
dataFrame_LineItems_List['warningMessages'] = dataFrame_LineItems_List.get('warningMessages')

# Mobile app details
dataFrame_LineItems_List['mobileApp.appId'] = dataFrame_LineItems_List.get('mobileApp.appId')
dataFrame_LineItems_List['mobileApp.platform'] = dataFrame_LineItems_List.get('mobileApp.platform')
dataFrame_LineItems_List['mobileApp.displayName'] = dataFrame_LineItems_List.get('mobileApp.displayName')
dataFrame_LineItems_List['mobileApp.publisher'] = dataFrame_LineItems_List.get('mobileApp.publisher')

# Reservation details
dataFrame_LineItems_List['reservationType'] = dataFrame_LineItems_List.get('reservationType')
dataFrame_LineItems_List['excludeNewExchanges'] = dataFrame_LineItems_List.get('excludeNewExchanges')

# YouTube and Partners settings
dataFrame_LineItems_List['youtubeAndPartnersSettings.viewFrequencyCap.unlimited'] = dataFrame_LineItems_List.get('youtubeAndPartnersSettings.viewFrequencyCap.unlimited')
dataFrame_LineItems_List['youtubeAndPartnersSettings.viewFrequencyCap.timeUnit'] = dataFrame_LineItems_List.get('youtubeAndPartnersSettings.viewFrequencyCap.timeUnit')
dataFrame_LineItems_List['youtubeAndPartnersSettings.viewFrequencyCap.timeUnitCount'] = dataFrame_LineItems_List.get('youtubeAndPartnersSettings.viewFrequencyCap.timeUnitCount')
dataFrame_LineItems_List['youtubeAndPartnersSettings.viewFrequencyCap.maxImpressions'] = dataFrame_LineItems_List.get('youtubeAndPartnersSettings.viewFrequencyCap.maxImpressions')
dataFrame_LineItems_List['youtubeAndPartnersSettings.viewFrequencyCap.maxViews'] = dataFrame_LineItems_List.get('youtubeAndPartnersSettings.viewFrequencyCap.maxViews')
dataFrame_LineItems_List['youtubeAndPartnersSettings.thirdPartyMeasurementConfigs.viewabilityVendorConfigs.vendor'] = dataFrame_LineItems_List.get('youtubeAndPartnersSettings.thirdPartyMeasurementConfigs.viewabilityVendorConfigs.vendor')
dataFrame_LineItems_List['youtubeAndPartnersSettings.thirdPartyMeasurementConfigs.viewabilityVendorConfigs.placementId'] = dataFrame_LineItems_List.get('youtubeAndPartnersSettings.thirdPartyMeasurementConfigs.viewabilityVendorConfigs.placementId')
dataFrame_LineItems_List['youtubeAndPartnersSettings.thirdPartyMeasurementConfigs.brandSafetyVendorConfigs.vendor'] = dataFrame_LineItems_List.get('youtubeAndPartnersSettings.thirdPartyMeasurementConfigs.brandSafetyVendorConfigs.vendor')
dataFrame_LineItems_List['youtubeAndPartnersSettings.thirdPartyMeasurementConfigs.brandSafetyVendorConfigs.placementId'] = dataFrame_LineItems_List.get('youtubeAndPartnersSettings.thirdPartyMeasurementConfigs.brandSafetyVendorConfigs.placementId')
dataFrame_LineItems_List['youtubeAndPartnersSettings.thirdPartyMeasurementConfigs.reachVendorConfigs.vendor'] = dataFrame_LineItems_List.get('youtubeAndPartnersSettings.thirdPartyMeasurementConfigs.reachVendorConfigs.vendor')
dataFrame_LineItems_List['youtubeAndPartnersSettings.thirdPartyMeasurementConfigs.reachVendorConfigs.placementId'] = dataFrame_LineItems_List.get('youtubeAndPartnersSettings.thirdPartyMeasurementConfigs.reachVendorConfigs.placementId')
dataFrame_LineItems_List['youtubeAndPartnersSettings.thirdPartyMeasurementConfigs.brandLiftVendorConfigs.vendor'] = dataFrame_LineItems_List.get('youtubeAndPartnersSettings.thirdPartyMeasurementConfigs.brandLiftVendorConfigs.vendor')
dataFrame_LineItems_List['youtubeAndPartnersSettings.thirdPartyMeasurementConfigs.brandLiftVendorConfigs.placementId'] = dataFrame_LineItems_List.get('youtubeAndPartnersSettings.thirdPartyMeasurementConfigs.brandLiftVendorConfigs.placementId')

# Inventory source settings
dataFrame_LineItems_List['youtubeAndPartnersSettings.inventorySourceSettings.includeYoutube'] = dataFrame_LineItems_List.get('youtubeAndPartnersSettings.inventorySourceSettings.includeYoutube')
dataFrame_LineItems_List['youtubeAndPartnersSettings.inventorySourceSettings.includeGoogleTv'] = dataFrame_LineItems_List.get('youtubeAndPartnersSettings.inventorySourceSettings.includeGoogleTv')
dataFrame_LineItems_List['youtubeAndPartnersSettings.inventorySourceSettings.includeYoutubeVideoPartners'] = dataFrame_LineItems_List.get('youtubeAndPartnersSettings.inventorySourceSettings.includeYoutubeVideoPartners')

# Content category and targeting frequency
dataFrame_LineItems_List['youtubeAndPartnersSettings.contentCategory'] = dataFrame_LineItems_List.get('youtubeAndPartnersSettings.contentCategory')
dataFrame_LineItems_List['youtubeAndPartnersSettings.effectiveContentCategory'] = dataFrame_LineItems_List.get('youtubeAndPartnersSettings.effectiveContentCategory')
dataFrame_LineItems_List['youtubeAndPartnersSettings.targetFrequency.targetCount'] = dataFrame_LineItems_List.get('youtubeAndPartnersSettings.targetFrequency.targetCount')
dataFrame_LineItems_List['youtubeAndPartnersSettings.targetFrequency.timeUnit'] = dataFrame_LineItems_List.get('youtubeAndPartnersSettings.targetFrequency.timeUnit')
dataFrame_LineItems_List['youtubeAndPartnersSettings.targetFrequency.timeUnitCount'] = dataFrame_LineItems_List.get('youtubeAndPartnersSettings.targetFrequency.timeUnitCount')
dataFrame_LineItems_List['youtubeAndPartnersSettings.linkedMerchantId'] = dataFrame_LineItems_List.get('youtubeAndPartnersSettings.linkedMerchantId')
dataFrame_LineItems_List['youtubeAndPartnersSettings.relatedVideoIds'] = dataFrame_LineItems_List.get('youtubeAndPartnersSettings.relatedVideoIds')
dataFrame_LineItems_List['youtubeAndPartnersSettings.leadFormId'] = dataFrame_LineItems_List.get('youtubeAndPartnersSettings.leadFormId')

# Video ad sequence settings
dataFrame_LineItems_List['youtubeAndPartnersSettings.videoAdSequenceSettings.minimumDuration'] = dataFrame_LineItems_List.get('youtubeAndPartnersSettings.videoAdSequenceSettings.minimumDuration')
dataFrame_LineItems_List['youtubeAndPartnersSettings.videoAdSequenceSettings.steps.stepId'] = dataFrame_LineItems_List.get('youtubeAndPartnersSettings.videoAdSequenceSettings.steps.stepId')
dataFrame_LineItems_List['youtubeAndPartnersSettings.videoAdSequenceSettings.steps.adGroupId'] = dataFrame_LineItems_List.get('youtubeAndPartnersSettings.videoAdSequenceSettings.steps.adGroupId')
dataFrame_LineItems_List['youtubeAndPartnersSettings.videoAdSequenceSettings.steps.previousStepId'] = dataFrame_LineItems_List.get('youtubeAndPartnersSettings.videoAdSequenceSettings.steps.previousStepId')
dataFrame_LineItems_List['youtubeAndPartnersSettings.videoAdSequenceSettings.steps.interactionType'] = dataFrame_LineItems_List.get('youtubeAndPartnersSettings.videoAdSequenceSettings.steps.interactionType')

# Ensure the DataFrame columns are ordered as expected before saving
columns_order = [
    'name', 'advertiserId', 'campaignId', 'insertionOrderId', 'lineItemId', 'displayName',
    'lineItemType', 'entityStatus', 'updateTime', 'partnerCosts',
    'flight.flightDateType', 'flight.dateRange.startDate.year',
    'flight.dateRange.startDate.month', 'flight.dateRange.startDate.day',
    'flight.dateRange.endDate.year', 'flight.dateRange.endDate.month',
    'flight.dateRange.endDate.day', 'budget.budgetAllocationType',
    'budget.budgetUnit', 'budget.maxAmount', 'pacing.pacingPeriod',
    'pacing.pacingType', 'pacing.dailyMaxMicros', 'pacing.dailyMaxImpressions',
    'frequencyCap.unlimited', 'frequencyCap.timeUnit', 'frequencyCap.timeUnitCount',
    'frequencyCap.maxImpressions', 'partnerRevenueModel.markupType', 'partnerRevenueModel.markupAmount',
    'conversionCounting.postViewCountPercentageMillis',
    'conversionCounting.floodlightActivityConfigs',
    'creativeIds', 'bidStrategy.fixedBid.bidAmountMicros',
    'bidStrategy.maximizeSpendAutoBid.performanceGoalType',
    'bidStrategy.maximizeSpendAutoBid.maxAverageCpmBidAmountMicros',
    'bidStrategy.maximizeSpendAutoBid.raiseBidForDeals',
    'bidStrategy.maximizeSpendAutoBid.customBiddingAlgorithmId',
    'bidStrategy.performanceGoalAutoBid.performanceGoalType',
    'bidStrategy.performanceGoalAutoBid.performanceGoalAmountMicros',
    'bidStrategy.performanceGoalAutoBid.maxAverageCpmBidAmountMicros',
    'bidStrategy.performanceGoalAutoBid.customBiddingAlgorithmId',
    'bidStrategy.youtubeAndPartnersBid.type', 'bidStrategy.youtubeAndPartnersBid.value',
    'bidStrategy.youtubeAndPartnersBid.adGroupEffectiveTargetCpaValue',
    'bidStrategy.youtubeAndPartnersBid.adGroupEffectiveTargetCpaSource',
    'integrationDetails.integrationCode', 'integrationDetails.details',
    'targetingExpansion.enableOptimizedTargeting',
    'targetingExpansion.audienceExpansionSeedListExcluded',
    'targetingExpansion.audienceExpansionLevel', 'warningMessages', 'mobileApp.appId',
    'mobileApp.platform', 'mobileApp.displayName', 'mobileApp.publisher',
    'reservationType', 'excludeNewExchanges',
    'youtubeAndPartnersSettings.viewFrequencyCap.unlimited',
    'youtubeAndPartnersSettings.viewFrequencyCap.timeUnit',
    'youtubeAndPartnersSettings.viewFrequencyCap.timeUnitCount',
    'youtubeAndPartnersSettings.viewFrequencyCap.maxImpressions',
    'youtubeAndPartnersSettings.viewFrequencyCap.maxViews',
    'youtubeAndPartnersSettings.thirdPartyMeasurementConfigs.viewabilityVendorConfigs.vendor',
    'youtubeAndPartnersSettings.thirdPartyMeasurementConfigs.viewabilityVendorConfigs.placementId',
    'youtubeAndPartnersSettings.thirdPartyMeasurementConfigs.brandSafetyVendorConfigs.vendor',
    'youtubeAndPartnersSettings.thirdPartyMeasurementConfigs.brandSafetyVendorConfigs.placementId',
    'youtubeAndPartnersSettings.thirdPartyMeasurementConfigs.reachVendorConfigs.vendor',
    'youtubeAndPartnersSettings.thirdPartyMeasurementConfigs.reachVendorConfigs.placementId',
    'youtubeAndPartnersSettings.thirdPartyMeasurementConfigs.brandLiftVendorConfigs.vendor',
    'youtubeAndPartnersSettings.thirdPartyMeasurementConfigs.brandLiftVendorConfigs.placementId',
    'youtubeAndPartnersSettings.inventorySourceSettings.includeYoutube',
    'youtubeAndPartnersSettings.inventorySourceSettings.includeGoogleTv',
    'youtubeAndPartnersSettings.inventorySourceSettings.includeYoutubeVideoPartners',
    'youtubeAndPartnersSettings.contentCategory',
    'youtubeAndPartnersSettings.effectiveContentCategory',
    'youtubeAndPartnersSettings.targetFrequency.targetCount',
    'youtubeAndPartnersSettings.targetFrequency.timeUnit',
    'youtubeAndPartnersSettings.targetFrequency.timeUnitCount',
    'youtubeAndPartnersSettings.linkedMerchantId', 'youtubeAndPartnersSettings.relatedVideoIds',
    'youtubeAndPartnersSettings.leadFormId', 'youtubeAndPartnersSettings.videoAdSequenceSettings.minimumDuration',
    'youtubeAndPartnersSettings.videoAdSequenceSettings.steps.stepId',
    'youtubeAndPartnersSettings.videoAdSequenceSettings.steps.adGroupId',
    'youtubeAndPartnersSettings.videoAdSequenceSettings.steps.previousStepId',
    'youtubeAndPartnersSettings.videoAdSequenceSettings.steps.interactionType'
]

# Reorder the DataFrame's columns
dataFrame_LineItems_List = dataFrame_LineItems_List[columns_order]

# Generate the list of LineItem IDs from the combined DataFrame
LineItemIdList = dataFrame_LineItems_List['lineItemId'].tolist()

## List AdGroups under each Advertiser

- Create the AdGroup list dataframe
  - dataFrame_AdGroups_List
- AdGroup Items
  - adGroupIdList

Using: [advertisers.adGroups.list](https://developers.google.com/display-video/api/reference/rest/v3/advertisers.adGroups/list)

In [11]:
dataFrame_AdGroups_List = pd.DataFrame({
    'name': pd.Series(dtype='str'),
    'advertiserId': pd.Series(dtype='int'),
    'adGroupId': pd.Series(dtype='int'),
    'lineItemId': pd.Series(dtype='int'),
    'displayName': pd.Series(dtype='str'),
    'adGroupFormat': pd.Series(dtype='str'),
    'bidStrategy.fixedBid.bidAmountMicros': pd.Series(dtype='int'),
    'bidStrategy.maximizeSpendAutoBid.performanceGoalType': pd.Series(dtype='str'),
    'bidStrategy.maximizeSpendAutoBid.maxAverageCpmBidAmountMicros': pd.Series(dtype='int'),
    'bidStrategy.maximizeSpendAutoBid.raiseBidForDeals': pd.Series(dtype='bool'),
    'bidStrategy.maximizeSpendAutoBid.customBiddingAlgorithmId': pd.Series(dtype='str'),
    'bidStrategy.performanceGoalAutoBid.performanceGoalType': pd.Series(dtype='str'),
    'bidStrategy.performanceGoalAutoBid.performanceGoalAmountMicros': pd.Series(dtype='int'),
    'bidStrategy.performanceGoalAutoBid.maxAverageCpmBidAmountMicros': pd.Series(dtype='int'),
    'bidStrategy.performanceGoalAutoBid.customBiddingAlgorithmId': pd.Series(dtype='str'),
    'bidStrategy.youtubeAndPartnersBid.type': pd.Series(dtype='str'),
    'bidStrategy.youtubeAndPartnersBid.value': pd.Series(dtype='int'),
    'bidStrategy.youtubeAndPartnersBid.adGroupEffectiveTargetCpaValue': pd.Series(dtype='int'),
    'bidStrategy.youtubeAndPartnersBid.adGroupEffectiveTargetCpaSource': pd.Series(dtype='str'),
    'entityStatus': pd.Series(dtype='str'),
    'targetingExpansion.enableOptimizedTargeting': pd.Series(dtype='bool'),
    'targetingExpansion.audienceExpansionSeedListExcluded': pd.Series(dtype='bool'),
    'targetingExpansion.audienceExpansionLevel': pd.Series(dtype='str'),
    'productFeedData.productMatchType': pd.Series(dtype='str'),
    'productFeedData.productMatchDimensions.productOdderId': pd.Series(dtype='str'),
    'productFeedData.productMatchDimensions.customLabel.key': pd.Series(dtype='str'),
    'productFeedData.productMatchDimensions.customLabel.value': pd.Series(dtype='str'),
    'productFeedData.isFeedDisabled': pd.Series(dtype='bool')
})

# Loop over each advertiser ID
for AID in advertiserIdList:
    try:
        # Request to list all insertion orders under the current advertiser
        request = service.advertisers().adGroups().list(
            advertiserId=AID,
        )
        response = request.execute()
        AdGroupListResponse = pd.json_normalize(response.get('adGroups', []))

        # Check if there are insertion orders to process
        if not AdGroupListResponse.empty:
            AdGroupListResponse['advertiserId'] = AID  # Add the advertiserId to the response

            # Append to the master DataFrame
            dataFrame_AdGroups_List = pd.concat([dataFrame_AdGroups_List, AdGroupListResponse], ignore_index=True)

    except Exception as e:
        print(f"An error occurred for advertiser ID {AID}: {e}")

# Extract the necessary columns from the combined DataFrame in the order they were created
dataFrame_AdGroups_List['name'] = dataFrame_AdGroups_List.get('name')
dataFrame_AdGroups_List['advertiserId'] = dataFrame_AdGroups_List.get('advertiserId')
dataFrame_AdGroups_List['lineItemId'] = dataFrame_AdGroups_List.get('lineItemId')
dataFrame_AdGroups_List['displayName'] = dataFrame_AdGroups_List.get('displayName')
dataFrame_AdGroups_List['adGroupFormat'] = dataFrame_AdGroups_List.get('adGroupFormat')

# Bid strategy
dataFrame_AdGroups_List['bidStrategy.fixedBid.bidAmountMicros'] = dataFrame_AdGroups_List.get('bidStrategy.fixedBid.bidAmountMicros')
dataFrame_AdGroups_List['bidStrategy.maximizeSpendAutoBid.performanceGoalType'] = dataFrame_AdGroups_List.get('bidStrategy.maximizeSpendAutoBid.performanceGoalType')
dataFrame_AdGroups_List['bidStrategy.maximizeSpendAutoBid.maxAverageCpmBidAmountMicros'] = dataFrame_AdGroups_List.get('bidStrategy.maximizeSpendAutoBid.maxAverageCpmBidAmountMicros')
dataFrame_AdGroups_List['bidStrategy.maximizeSpendAutoBid.raiseBidForDeals'] = dataFrame_AdGroups_List.get('bidStrategy.maximizeSpendAutoBid.raiseBidForDeals')
dataFrame_AdGroups_List['bidStrategy.maximizeSpendAutoBid.customBiddingAlgorithmId'] = dataFrame_AdGroups_List.get('bidStrategy.maximizeSpendAutoBid.customBiddingAlgorithmId')
dataFrame_AdGroups_List['bidStrategy.performanceGoalAutoBid.performanceGoalType'] = dataFrame_AdGroups_List.get('bidStrategy.performanceGoalAutoBid.performanceGoalType')
dataFrame_AdGroups_List['bidStrategy.performanceGoalAutoBid.performanceGoalAmountMicros'] = dataFrame_AdGroups_List.get('bidStrategy.performanceGoalAutoBid.performanceGoalAmountMicros')
dataFrame_AdGroups_List['bidStrategy.performanceGoalAutoBid.maxAverageCpmBidAmountMicros'] = dataFrame_AdGroups_List.get('bidStrategy.performanceGoalAutoBid.maxAverageCpmBidAmountMicros')
dataFrame_AdGroups_List['bidStrategy.performanceGoalAutoBid.customBiddingAlgorithmId'] = dataFrame_AdGroups_List.get('bidStrategy.performanceGoalAutoBid.customBiddingAlgorithmId')
dataFrame_AdGroups_List['bidStrategy.youtubeAndPartnersBid.type'] = dataFrame_AdGroups_List.get('bidStrategy.youtubeAndPartnersBid.type')
dataFrame_AdGroups_List['bidStrategy.youtubeAndPartnersBid.value'] = dataFrame_AdGroups_List.get('bidStrategy.youtubeAndPartnersBid.value')
dataFrame_AdGroups_List['bidStrategy.youtubeAndPartnersBid.adGroupEffectiveTargetCpaValue'] = dataFrame_AdGroups_List.get('bidStrategy.youtubeAndPartnersBid.adGroupEffectiveTargetCpaValue')
dataFrame_AdGroups_List['bidStrategy.youtubeAndPartnersBid.adGroupEffectiveTargetCpaSource'] = dataFrame_AdGroups_List.get('bidStrategy.youtubeAndPartnersBid.adGroupEffectiveTargetCpaSource')

dataFrame_AdGroups_List['entityStatus'] = dataFrame_AdGroups_List.get('entityStatus')

# Targeting expansion details
dataFrame_AdGroups_List['targetingExpansion.enableOptimizedTargeting'] = dataFrame_AdGroups_List.get('targetingExpansion.enableOptimizedTargeting')
dataFrame_AdGroups_List['targetingExpansion.audienceExpansionSeedListExcluded'] = dataFrame_AdGroups_List.get('targetingExpansion.audienceExpansionSeedListExcluded')
dataFrame_AdGroups_List['targetingExpansion.audienceExpansionLevel'] = dataFrame_AdGroups_List.get('targetingExpansion.audienceExpansionLevel')

# Partner costs
dataFrame_AdGroups_List['productFeedData.productMatchType'] = dataFrame_AdGroups_List.get('productFeedData.productMatchType')
dataFrame_AdGroups_List['productFeedData.productMatchDimensions.productOdderId'] = dataFrame_AdGroups_List.get('productFeedData.productMatchDimensions.productOdderId')
dataFrame_AdGroups_List['productFeedData.productMatchDimensions.customLabel.key'] = dataFrame_AdGroups_List.get('productFeedData.productMatchDimensions.customLabel.key')
dataFrame_AdGroups_List['productFeedData.productMatchDimensions.customLabel.value'] = dataFrame_AdGroups_List.get('productFeedData.productMatchDimensions.customLabel.value')
dataFrame_AdGroups_List['productFeedData.isFeedDisabled'] = dataFrame_AdGroups_List.get('productFeedData.isFeedDisabled')

# Ensure the DataFrame columns are ordered as expected before saving
columns_order = [
    'name',
    'advertiserId',
    'adGroupId',
    'lineItemId',
    'displayName',
    'adGroupFormat',
    'bidStrategy.maximizeSpendAutoBid.performanceGoalType',
    'bidStrategy.maximizeSpendAutoBid.maxAverageCpmBidAmountMicros',
    'bidStrategy.maximizeSpendAutoBid.raiseBidForDeals',
    'bidStrategy.maximizeSpendAutoBid.customBiddingAlgorithmId',
    'bidStrategy.performanceGoalAutoBid.performanceGoalType',
    'bidStrategy.performanceGoalAutoBid.performanceGoalAmountMicros',
    'bidStrategy.performanceGoalAutoBid.maxAverageCpmBidAmountMicros',
    'bidStrategy.performanceGoalAutoBid.customBiddingAlgorithmId',
    'bidStrategy.youtubeAndPartnersBid.type', 'bidStrategy.youtubeAndPartnersBid.value',
    'bidStrategy.youtubeAndPartnersBid.adGroupEffectiveTargetCpaValue',
    'bidStrategy.youtubeAndPartnersBid.adGroupEffectiveTargetCpaSource',
    'entityStatus',
    'targetingExpansion.enableOptimizedTargeting',
    'targetingExpansion.audienceExpansionSeedListExcluded',
    'targetingExpansion.audienceExpansionLevel',
    'productFeedData.productMatchType',
    'productFeedData.productMatchDimensions.productOdderId',
    'productFeedData.productMatchDimensions.customLabel.key',
    'productFeedData.productMatchDimensions.customLabel.value',
    'productFeedData.isFeedDisabled'
]

# Reorder the DataFrame's columns
dataFrame_AdGroups_List = dataFrame_AdGroups_List[columns_order]

# Generate the list of LineItem IDs from the combined DataFrame
AdGroupList = dataFrame_AdGroups_List['lineItemId'].tolist()

## Create the id_Name_Table
- Table containing all IDs and Names for
  - Partner
  - Advertiser
  - Campaign
  - Insertion Order
  - Line Item

In [12]:
# Join dataFrame_Partners_list and dataFrame_Advertisers_list on partnerId
partners_advertisers_join = pd.merge(
    dataFrame_Partners_List[['partnerId', 'displayName']],
    dataFrame_Advertisers_List[['partnerId', 'advertiserId', 'displayName']],
    on='partnerId',
    how='right'  # Right join to ensure all advertisers are included
)

# Rename columns after the first merge
partners_advertisers_join.rename(columns={
    'displayName_x': 'partner_displayName',
    'displayName_y': 'advertiser_displayName'
}, inplace=True)

# Join the resulting DataFrame with dataFrame_Campaigns_List on advertiserId
all_IdName_table = pd.merge(
    partners_advertisers_join,
    dataFrame_Campaigns_List[['advertiserId', 'campaignId', 'displayName']],
    on='advertiserId',
    how='left'  # Left join to ensure all campaigns are included
)

# Rename columns after the second merge
all_IdName_table.rename(columns={
    'displayName': 'campaign_displayName'
}, inplace=True)

# Join the resulting DataFrame with dataFrame_InsertionOrders_List on campaignId
all_IdName_table = pd.merge(
    all_IdName_table,
    dataFrame_InsertionOrders_List[['campaignId', 'insertionOrderId', 'displayName']],
    on='campaignId',
    how='left'  # Left join to ensure all insertion orders are included
)

# Rename columns after the third merge
all_IdName_table.rename(columns={
    'displayName': 'insertionOrder_displayName'
}, inplace=True)

# Join the resulting DataFrame with dataFrame_LineItems_List on insertionOrderId
all_IdName_table = pd.merge(
    all_IdName_table,
    dataFrame_LineItems_List[['insertionOrderId', 'lineItemId', 'displayName']],
    on='insertionOrderId',
    how='left'  # Left join to ensure all line items are included
)

# Rename columns after the fourth merge
all_IdName_table.rename(columns={
    'displayName': 'lineItem_displayName'
}, inplace=True)

# Join the resulting DataFrame with dataFrame_AdGroups_List on lineItemId
all_IdName_table = pd.merge(
    all_IdName_table,
    dataFrame_AdGroups_List[['lineItemId', 'adGroupId', 'displayName']],
    on='lineItemId',
    how='left'  # Left join to ensure all ad groups are included
)

# Rename columns after the fifth merge
all_IdName_table.rename(columns={
    'displayName': 'adGroup_displayName'
}, inplace=True)

# Inspect the DataFrame columns before selecting
print("Columns in all_IdName_table:", all_IdName_table.columns.tolist())

# Select and rename the columns as necessary
id_Name_table = all_IdName_table[['partnerId', 'partner_displayName', 'advertiserId', 'advertiser_displayName', 'campaignId', 'campaign_displayName', 'insertionOrderId', 'insertionOrder_displayName', 'lineItemId', 'lineItem_displayName', 'adGroupId', 'adGroup_displayName']]

# Optional: Rename columns for clarity
id_Name_table.columns = [
    'Partner ID',
    'Partner Name',
    'Advertiser ID',
    'Advertiser Name',
    'Campaign ID',
    'Campaign Name',
    'Insertion Order ID',
    'Insertion Order Name',
    'Line Item ID',
    'Line Item Name',
    'Ad Group ID',
    'Ad Group Name'
]

# Output the final table
print(id_Name_table)


Columns in all_IdName_table: ['partnerId', 'partner_displayName', 'advertiserId', 'advertiser_displayName', 'campaignId', 'campaign_displayName', 'insertionOrderId', 'insertionOrder_displayName', 'lineItemId', 'lineItem_displayName', 'adGroupId', 'adGroup_displayName']
    Partner ID                                       Partner Name  \
0      1407943                                        Mail Online   
1      1407943                                        Mail Online   
2      1407943                                        Mail Online   
3      1407943                                        Mail Online   
4      1407943                                        Mail Online   
..         ...                                                ...   
288  941894575  TP - The Beyond Collective - Incubeta UK - DV3...   
289  941894575  TP - The Beyond Collective - Incubeta UK - DV3...   
290  941894575  TP - The Beyond Collective - Incubeta UK - DV3...   
291  941894575  TP - The Beyond Collecti

## Data & Directories - Turn all list dataFrames into a CSV

In [94]:
# Define the directory to save the CSV files
data_directory = '1. All levels - Heirachy List'

# Create the directory if it doesn't exist
if not os.path.exists(data_directory):
    os.makedirs(data_directory)

# Dictionary of additional DataFrames to save
heirachyList_dataframes_dict = {
    'dataFrame_Partners_List': dataFrame_Partners_List,
    'dataFrame_Advertisers_List': dataFrame_Advertisers_List,
    'dataFrame_Campaigns_List': dataFrame_Campaigns_List,
    'dataFrame_InsertionOrders_List': dataFrame_InsertionOrders_List,
    'dataFrame_LineItems_List': dataFrame_LineItems_List,
    'dataFrame_AdGroups_List': dataFrame_AdGroups_List,
    'id_Name_table': id_Name_table
}

# Save each DataFrame as a CSV file in the specified directory
for df_name, df in heirachyList_dataframes_dict.items():
    file_path = os.path.join(data_directory, f'{df_name}.csv')
    df.to_csv(file_path, index=False)
    print(f"Saved {df_name} to {file_path}")

Saved dataFrame_Partners_List to 1. All levels - Heirachy List/dataFrame_Partners_List.csv
Saved dataFrame_Advertisers_List to 1. All levels - Heirachy List/dataFrame_Advertisers_List.csv
Saved dataFrame_Campaigns_List to 1. All levels - Heirachy List/dataFrame_Campaigns_List.csv
Saved dataFrame_InsertionOrders_List to 1. All levels - Heirachy List/dataFrame_InsertionOrders_List.csv
Saved dataFrame_LineItems_List to 1. All levels - Heirachy List/dataFrame_LineItems_List.csv
Saved dataFrame_AdGroups_List to 1. All levels - Heirachy List/dataFrame_AdGroups_List.csv
Saved id_Name_table to 1. All levels - Heirachy List/id_Name_table.csv


# List all inclusion or exclusion lists

## List the Partner Channels and find their sites
- Create the Partner Channel ID and Site ID dataframe
  - dataFrame_Partner_ChannelIdSiteId

Using: [partners.channels.list](https://developers.google.com/display-video/api/reference/rest/v3/partners.channels/list) then [partners.channels.sites.list](https://developers.google.com/display-video/api/reference/rest/v3/partners.channels.sites/list)

In [14]:
# Initialize a list to hold all ChannelIdSiteId data across all partners
Partner_ChannelIdSiteId = []

# Loop through each partner
for partner in dataFrame_Partners_List["partnerId"]:
    # API listing the Partner Channels
    request = service.partners().channels().list(partnerId=partner)
    response = request.execute()

    # Ensure 'channels' key exists in the response
    if 'channels' in response:
        response_PartnerChannels = pd.json_normalize(response['channels'])

        # Loop through each channel ID and fetch the associated sites
        for _, row in response_PartnerChannels.iterrows():
            channelId = row['channelId']
            displayName = row['displayName']

            # Request to get the list of sites for the channel
            request = service.partners().channels().sites().list(
                partnerId=partner,
                channelId=channelId
            )
            response = request.execute()

            # Get list of site IDs if 'sites' is in the response, else an empty list
            list_PartnerChannelSites = pd.json_normalize(response.get('sites', []))['urlOrAppId'].tolist()

            # Append data to the master list
            Partner_ChannelIdSiteId.append({
                'PartnerID': partner,
                'ChannelID': channelId,
                'DisplayName': displayName,
                'SiteIDs': list_PartnerChannelSites
            })

# Convert the master list to a single DataFrame
dataFrame_Partner_ChannelIdSiteId = pd.DataFrame(Partner_ChannelIdSiteId)

## List the Advertiser Channels and find their sites
- Create the Advertiser Channel ID and Site ID dataframe
  - dataFrame_Advertiser_ChannelIdSiteId

Using: [advertisers.channels.list](https://developers.google.com/display-video/api/reference/rest/v3/advertisers.channels/list) then [advertisers.channels.sites.list](https://developers.google.com/display-video/api/reference/rest/v3/advertisers.channels.sites/list)

In [15]:
# Initialize a list to hold all ChannelIdSiteId data across all advertisers
Advertiser_ChannelIdSiteId = []

# Loop through each advertiser
for advertiser in advertiserIdList:
    # API listing the Advertiser Channels
    request = service.advertisers().channels().list(advertiserId=advertiser)
    response = request.execute()

    # Ensure 'channels' key exists in the response
    if 'channels' in response:
        response_AdvertiserChannels = pd.json_normalize(response['channels'])

        # Loop through each channel ID and fetch the associated sites
        for _, row in response_AdvertiserChannels.iterrows():
            channelId = row['channelId']
            displayName = row['displayName']

            # Request to get the list of sites for the channel
            request = service.advertisers().channels().sites().list(
                advertiserId=advertiser,
                channelId=channelId
            )
            response = request.execute()

            # Get list of site IDs if 'sites' is in the response, else an empty list
            list_AdvertiserChannelSites = pd.json_normalize(response.get('sites', []))['urlOrAppId'].tolist()

            # Append data to the master list
            Advertiser_ChannelIdSiteId.append({
                'AdvertiserID': advertiser,
                'ChannelID': channelId,
                'DisplayName': displayName,
                'SiteIDs': list_AdvertiserChannelSites
            })

# Convert the master list to a single DataFrame
dataFrame_Advertiser_ChannelIdSiteId = pd.DataFrame(Advertiser_ChannelIdSiteId)

## List the Negative Keyword Lists for each advertiser and their keywords
- Create the Advertiser Channel ID and Site ID dataframe
  - dataFrame_Advertiser_negativeKeywordLists
  - dataFrame_Advertiser_negativeKeywordListsNegativeKeywords

Using: [advertisers.negativeKeywordLists.list](https://developers.google.com/display-video/api/reference/rest/v3/advertisers.negativeKeywordLists/list) then [advertisers.negativeKeywordLists.negativeKeywords.list](https://developers.google.com/display-video/api/reference/rest/v3/advertisers.negativeKeywordLists.negativeKeywords/list)

In [16]:
# Initialize lists to hold all NegativeKeywordLists and their associated NegativeKeywords across all advertisers
Advertiser_NegativeKeywordLists = []
Advertiser_NegativeKeywordListsNegativeKeywords = []

# Loop through each advertiser
for advertiser in dataFrame_Advertisers_List["advertiserId"]:
    # API listing the Advertiser negativeKeywordLists
    request = service.advertisers().negativeKeywordLists().list(advertiserId=advertiser)
    response = request.execute()

    # Ensure 'negativeKeywordLists' key exists in the response
    if 'negativeKeywordLists' in response:
        response_advertiser_NegativeKeywordLists = pd.json_normalize(response['negativeKeywordLists'])

        # Loop through each negativeKeywordListId and fetch the associated negative keywords
        for _, row in response_advertiser_NegativeKeywordLists.iterrows():
            negativeKeywordListId = row['negativeKeywordListId']
            displayName = row['displayName']

            # Append the negative keyword list details to the master list
            Advertiser_NegativeKeywordLists.append({
                'advertiserId': advertiser,
                'negativeKeywordListId': negativeKeywordListId,
                'DisplayName': displayName
            })

            # Request to get the negative keywords for the negative keyword list
            request = service.advertisers().negativeKeywordLists().negativeKeywords().list(
                advertiserId=advertiser,
                negativeKeywordListId=negativeKeywordListId
            )
            response = request.execute()

            # Get list of negative keywords if 'negativeKeywords' is in the response, else an empty list
            list_AdvertiserNegativeKeywords = pd.json_normalize(response.get('negativeKeywords', []))['keywordValue'].tolist()

            # Append data to the master list for negative keywords
            Advertiser_NegativeKeywordListsNegativeKeywords.append({
                'advertiserId': advertiser,
                'negativeKeywordListId': negativeKeywordListId,
                'DisplayName': displayName,
                'negativeKeywords': list_AdvertiserNegativeKeywords
            })

# Convert the accumulated lists to DataFrames
dataFrame_Advertiser_NegativeKeywordListsNegativeKeywords = pd.DataFrame(Advertiser_NegativeKeywordListsNegativeKeywords)

## List the Creative IDs for each advertiser and their creatives
- Create the Creative ID and CreativeDetail dataframe
  - dataFrame_CreativeId
  - dataFrame_CreativeId_CreativeDetail

Using: [advertisers.creatives.list](https://developers.google.com/display-video/api/reference/rest/v3/advertisers.creatives/list) then [advertisers.creatives.get](https://developers.google.com/display-video/api/reference/rest/v3/advertisers.creatives/get)

In [17]:
# V2 Testing - Adding more information to the creative dataframe and removed the GET call

# Initialize the DataFrame with the specified structure for all Creatives
dataFrame_CreativeDetails_List = pd.DataFrame({
    'name': pd.Series(dtype='str'),
    'advertiserId': pd.Series(dtype='int'),
    'creativeId': pd.Series(dtype='str'),
    'cmPlacementId': pd.Series(dtype='str'),
    'displayName': pd.Series(dtype='str'),
    'entityStatus': pd.Series(dtype='str'),
    'updateTime': pd.Series(dtype='str'),
    'createTime': pd.Series(dtype='str'),
    'creativeType': pd.Series(dtype='str'),
    'hostingSource': pd.Series(dtype='str'),
    'dynamic': pd.Series(dtype='bool'),

    # Dimensions
    'dimensions.widthPixels': pd.Series(dtype='int'),
    'dimensions.heightPixels': pd.Series(dtype='int'),

    # Additional dimensions
    'additionalDimensions.widthPixels': pd.Series(dtype='int'),
    'additionalDimensions.heightPixels': pd.Series(dtype='int'),

    # Media details
    'mediaDuration': pd.Series(dtype='float'),
    'creativeAttributes': pd.Series(dtype='str'),

    # Review status
    'reviewStatus.approvalStatus': pd.Series(dtype='str'),
    'reviewStatus.creativeAndLandingPageReviewStatus': pd.Series(dtype='str'),
    'reviewStatus.contentAndPolicyReviewStatus': pd.Series(dtype='str'),
    'reviewStatus.exchangeReviewStatuses.exchange': pd.Series(dtype='str'),
    'reviewStatus.exchangeReviewStatuses.status': pd.Series(dtype='str'),
    'reviewStatus.publisherReviewStatuses.publisherName': pd.Series(dtype='str'),
    'reviewStatus.publisherReviewStatuses.status': pd.Series(dtype='str'),

    # Asset information
    'assets.asset.mediaId': pd.Series(dtype='str'),
    'assets.asset.content': pd.Series(dtype='str'),
    'assets.role': pd.Series(dtype='str'),

    # Events
    'exitEvents.type': pd.Series(dtype='str'),
    'exitEvents.url': pd.Series(dtype='str'),
    'exitEvents.name': pd.Series(dtype='str'),
    'exitEvents.reportingName': pd.Series(dtype='str'),

    'timerEvents.name': pd.Series(dtype='str'),
    'timerEvents.reportingName': pd.Series(dtype='str'),
    'counterEvents.name': pd.Series(dtype='str'),
    'counterEvents.reportingName': pd.Series(dtype='str'),

    'appendedTag': pd.Series(dtype='str'),
    'integrationCode': pd.Series(dtype='str'),
    'notes': pd.Series(dtype='str'),
    'iasCampaignMonitoring': pd.Series(dtype='bool'),
    'companionCreativeIds': pd.Series(dtype='object'),

    # Skip and progress offset
    'skippable': pd.Series(dtype='bool'),
    'skipOffset.percentage': pd.Series(dtype='float'),
    'skipOffset.seconds': pd.Series(dtype='float'),
    'progressOffset.percentage': pd.Series(dtype='float'),
    'progressOffset.seconds': pd.Series(dtype='float'),

    # UniversalAdId details
    'universalAdId.id': pd.Series(dtype='str'),
    'universalAdId.registry': pd.Series(dtype='str'),

    # Third-party URLs and transcodes
    'thirdPartyUrls.type': pd.Series(dtype='str'),
    'thirdPartyUrls.url': pd.Series(dtype='str'),

    # Transcodes
    'transcodes.name': pd.Series(dtype='str'),
    'transcodes.mimeType': pd.Series(dtype='str'),
    'transcodes.transcoded': pd.Series(dtype='bool'),
    'transcodes.dimensions.widthPixels': pd.Series(dtype='int'),
    'transcodes.dimensions.heightPixels': pd.Series(dtype='int'),
    'transcodes.bitRateKbps': pd.Series(dtype='int'),
    'transcodes.frameRate': pd.Series(dtype='int'),
    'transcodes.audioBitRateKbps': pd.Series(dtype='int'),
    'transcodes.audioSampleRateHz': pd.Series(dtype='int'),
    'transcodes.fileSizeBytes': pd.Series(dtype='int'),

    # Tracking URLs
    'trackerUrls': pd.Series(dtype='object'),
    'jsTrackerUrl': pd.Series(dtype='str'),

    # CM Tracking details
    'cmTrackingAd.cmPlacementId': pd.Series(dtype='str'),
    'cmTrackingAd.cmCreativeId': pd.Series(dtype='str'),
    'cmTrackingAd.cmAdId': pd.Series(dtype='str'),

    # OBA Icon details
    'obaIcon.resourceUrl': pd.Series(dtype='str'),
    'obaIcon.resourceMimeType': pd.Series(dtype='str'),
    'obaIcon.position': pd.Series(dtype='str'),
    'obaIcon.dimensions.widthPixels': pd.Series(dtype='int'),
    'obaIcon.dimensions.heightPixels': pd.Series(dtype='int'),
    'obaIcon.program': pd.Series(dtype='str'),
    'obaIcon.landingPageUrl': pd.Series(dtype='str'),
    'obaIcon.clickTrackingUrl': pd.Series(dtype='str'),
    'obaIcon.viewTrackingUrl': pd.Series(dtype='str'),

    # Tags and tracking
    'thirdPartyTag': pd.Series(dtype='str'),
    'requireMraid': pd.Series(dtype='bool'),
    'requireHtml5': pd.Series(dtype='bool'),
    'requirePingForAttribution': pd.Series(dtype='bool'),
    'expandingDirection': pd.Series(dtype='str'),
    'expandOnHover': pd.Series(dtype='bool'),
    'vastTagUrl': pd.Series(dtype='str'),
    'vpaid': pd.Series(dtype='bool'),
    'html5Video': pd.Series(dtype='bool'),
    'lineItemIds': pd.Series(dtype='object'),
    'mp3Audio': pd.Series(dtype='bool'),
    'oggAudio': pd.Series(dtype='bool')
})

# Initialize a list to store accumulated data for all advertisers
all_CreativeDetails = []

# Loop through each advertiser
for advertiser in dataFrame_Advertisers_List["advertiserId"]:
    try:
        # API listing the Advertiser CreativeIds
        request = service.advertisers().creatives().list(advertiserId=advertiser)
        response = request.execute()

        # Ensure 'creatives' key exists in the response
        if 'creatives' in response:
            response_advertiser_CreativeId = pd.json_normalize(response['creatives'])

            # Loop through each creative and extract relevant information
            for _, row in response_advertiser_CreativeId.iterrows():
                creativeDetails = {
                    'name': row.get('name', ''),
                    'advertiserId': advertiser,
                    'creativeId': row.get('creativeId', ''),
                    'cmPlacementId': row.get('cmPlacementId', ''),
                    'displayName': row.get('displayName', ''),
                    'entityStatus': row.get('entityStatus', ''),
                    'updateTime': row.get('updateTime', ''),
                    'createTime': row.get('createTime', ''),
                    'creativeType': row.get('creativeType', ''),
                    'hostingSource': row.get('hostingSource', ''),
                    'dynamic': row.get('dynamic', False),

                    # Dimensions
                    'dimensions.widthPixels': row.get('dimensions.widthPixels', None),
                    'dimensions.heightPixels': row.get('dimensions.heightPixels', None),

                    # Additional dimensions
                    'additionalDimensions.widthPixels': row.get('additionalDimensions.widthPixels', None),
                    'additionalDimensions.heightPixels': row.get('additionalDimensions.heightPixels', None),

                    # Media details
                    'mediaDuration': row.get('mediaDuration', None),
                    'creativeAttributes': row.get('creativeAttributes', ''),

                    # Review status
                    'reviewStatus.approvalStatus': row.get('reviewStatus.approvalStatus', ''),
                    'reviewStatus.creativeAndLandingPageReviewStatus': row.get('reviewStatus.creativeAndLandingPageReviewStatus', ''),
                    'reviewStatus.exchangeReviewStatuses.exchange': row.get('reviewStatus.exchangeReviewStatuses.exchange', ''),
                    'reviewStatus.exchangeReviewStatuses.status': row.get('reviewStatus.exchangeReviewStatuses.status', ''),
                    'reviewStatus.publisherReviewStatuses.publisherName': row.get('reviewStatus.publisherReviewStatuses.publisherName', ''),
                    'reviewStatus.publisherReviewStatuses.status': row.get('reviewStatus.publisherReviewStatuses.status', ''),

                    # Assets information
                    'assets.asset.mediaId': row.get('assets[0].asset.mediaId', ''),
                    'assets.asset.content': row.get('assets[0].asset.content', ''),
                    'assets.role': row.get('assets[0].role', ''),

                    # Events
                    'exitEvents.type': row.get('exitEvents[0].type', ''),
                    'exitEvents.url': row.get('exitEvents[0].url', ''),
                    'exitEvents.name': row.get('exitEvents[0].name', ''),
                    'exitEvents.reportingName': row.get('exitEvents[0].reportingName', ''),
                    'timerEvents.name': row.get('timerEvents[0].name', ''),
                    'timerEvents.reportingName': row.get('timerEvents[0].reportingName', ''),
                    'counterEvents.name': row.get('counterEvents[0].name', ''),
                    'counterEvents.reportingName': row.get('counterEvents[0].reportingName', ''),

                    # Additional information
                    'appendedTag': row.get('appendedTag', ''),
                    'integrationCode': row.get('integrationCode', ''),
                    'notes': row.get('notes', ''),
                    'iasCampaignMonitoring': row.get('iasCampaignMonitoring', False),
                    'companionCreativeIds': row.get('companionCreativeIds', []),

                    # Skip and progress offset
                    'skippable': row.get('skippable', False),
                    'skipOffset.percentage': row.get('skipOffset.percentage', None),
                    'skipOffset.seconds': row.get('skipOffset.seconds', None),
                    'progressOffset.percentage': row.get('progressOffset.percentage', None),
                    'progressOffset.seconds': row.get('progressOffset.seconds', None),

                    # UniversalAdId details
                    'universalAdId.id': row.get('universalAdId.id', ''),
                    'universalAdId.registry': row.get('universalAdId.registry', ''),

                    # Third-party URLs
                    'thirdPartyUrls.type': row.get('thirdPartyUrls[0].type', ''),
                    'thirdPartyUrls.url': row.get('thirdPartyUrls[0].url', ''),

                    # Transcodes
                    'transcodes.name': row.get('transcodes[0].name', ''),
                    'transcodes.mimeType': row.get('transcodes[0].mimeType', ''),
                    'transcodes.transcoded': row.get('transcodes[0].transcoded', False),
                    'transcodes.dimensions.widthPixels': row.get('transcodes[0].dimensions.widthPixels', None),
                    'transcodes.dimensions.heightPixels': row.get('transcodes[0].dimensions.heightPixels', None),
                    'transcodes.bitRateKbps': row.get('transcodes[0].bitRateKbps', None),
                    'transcodes.frameRate': row.get('transcodes[0].frameRate', None),
                    'transcodes.audioBitRateKbps': row.get('transcodes[0].audioBitRateKbps', None),
                    'transcodes.audioSampleRateHz': row.get('transcodes[0].audioSampleRateHz', None),
                    'transcodes.fileSizeBytes': row.get('transcodes[0].fileSizeBytes', None),

                    # Tracker URLs
                    'trackerUrls': row.get('trackerUrls', []),
                    'jsTrackerUrl': row.get('jsTrackerUrl', ''),

                    # CM Tracking details
                    'cmTrackingAd.cmPlacementId': row.get('cmTrackingAd.cmPlacementId', ''),
                    'cmTrackingAd.cmCreativeId': row.get('cmTrackingAd.cmCreativeId', ''),
                    'cmTrackingAd.cmAdId': row.get('cmTrackingAd.cmAdId', ''),

                    # OBA Icon details
                    'obaIcon.resourceUrl': row.get('obaIcon.resourceUrl', ''),
                    'obaIcon.resourceMimeType': row.get('obaIcon.resourceMimeType', ''),
                    'obaIcon.position': row.get('obaIcon.position', ''),
                    'obaIcon.dimensions.widthPixels': row.get('obaIcon.dimensions.widthPixels', None),
                    'obaIcon.dimensions.heightPixels': row.get('obaIcon.dimensions.heightPixels', None),
                    'obaIcon.program': row.get('obaIcon.program', ''),
                    'obaIcon.landingPageUrl': row.get('obaIcon.landingPageUrl', ''),
                    'obaIcon.clickTrackingUrl': row.get('obaIcon.clickTrackingUrl', ''),
                    'obaIcon.viewTrackingUrl': row.get('obaIcon.viewTrackingUrl', ''),

                    # Other creative details
                    'thirdPartyTag': row.get('thirdPartyTag', ''),
                    'requireMraid': row.get('requireMraid', False),
                    'requireHtml5': row.get('requireHtml5', False),
                    'requirePingForAttribution': row.get('requirePingForAttribution', False),
                    'expandingDirection': row.get('expandingDirection', ''),
                    'expandOnHover': row.get('expandOnHover', False),
                    'vastTagUrl': row.get('vastTagUrl', ''),
                    'vpaid': row.get('vpaid', False),
                    'html5Video': row.get('html5Video', False),
                    'lineItemIds': row.get('lineItemIds', []),
                    'mp3Audio': row.get('mp3Audio', False),
                    'oggAudio': row.get('oggAudio', False)
                }

                # Append the creative details to the list
                all_CreativeDetails.append(creativeDetails)
    except Exception as e:
        print(f"An error occurred for advertiser ID {advertiser}: {e}")

# Convert the accumulated list to a DataFrame
dataFrame_Advertiser_CreativeId_CreativeDetail = pd.DataFrame(all_CreativeDetails)

In [18]:
# ## Older code - Not being used at present
# # Initialize a list to store accumulated data for all advertisers
# all_CreativeDetails = []

# # Loop through each advertiser
# for advertiser in dataFrame_Advertisers_List["advertiserId"]:
#     try:
#         # API listing the Advertiser CreativeIds
#         request = service.advertisers().creatives().list(advertiserId=advertiser)
#         response = request.execute()

#         # Ensure 'creatives' key exists in the response
#         if 'creatives' in response:
#             response_advertiser_CreativeId = pd.json_normalize(response['creatives'])

#             # Loop through each creative and fetch detailed information
#             for _, row in response_advertiser_CreativeId.iterrows():
#                 creativeId = row['creativeId']

#                 # Request to get the creative details
#                 request = service.advertisers().creatives().get(
#                     advertiserId=advertiser,
#                     creativeId=creativeId
#                 )
#                 response = request.execute()

#                 # Merge initial creative list data with detailed creative data
#                 creativeDetails = {
#                     'name': row.get('name', ''),
#                     'advertiserId': advertiser,
#                     'creativeId': creativeId,
#                     'displayName': row.get('displayName', ''),
#                     'entityStatus': row.get('entityStatus', ''),
#                     'updateTime': row.get('updateTime', ''),
#                     'createTime': row.get('createTime', ''),
#                     'creativeType': row.get('creativeType', ''),
#                     'hostingSource': row.get('hostingSource', ''),
#                     'assets': response.get('assets', []),
#                     'exitEvents': response.get('exitEvents', []),
#                     'appendedTag': row.get('appendedTag', ''),
#                     'lineItemIds': row.get('lineItemIds', []),
#                     'dimensions.widthPixels': response.get('dimensions', {}).get('widthPixels', None),
#                     'dimensions.heightPixels': response.get('dimensions', {}).get('heightPixels', None),
#                     'reviewStatus.approvalStatus': response.get('reviewStatus', {}).get('approvalStatus', ''),
#                     'reviewStatus.creativeAndLandingPageReviewStatus': response.get('reviewStatus', {}).get('creativeAndLandingPageReviewStatus', ''),
#                     'reviewStatus.exchangeReviewStatuses': response.get('reviewStatus', {}).get('exchangeReviewStatuses', []),
#                     'mediaDuration': response.get('mediaDuration', None),
#                     'creativeAttributes': response.get('creativeAttributes', []),
#                     'thirdPartyUrls': response.get('thirdPartyUrls', []),
#                     'transcodes': response.get('transcodes', []),
#                     'skipOffset.seconds': response.get('skipOffset', {}).get('seconds', None),
#                     'progressOffset.seconds': response.get('progressOffset', {}).get('seconds', None),
#                     'universalAdId.id': response.get('universalAdId', {}).get('id', ''),
#                     'universalAdId.registry': response.get('universalAdId', {}).get('registry', ''),
#                     'thirdPartyTag': response.get('thirdPartyTag', ''),
#                     'vastTagUrl': response.get('vastTagUrl', ''),
#                     'requireMraid': response.get('requireMraid', False),
#                     'requireHtml5': response.get('requireHtml5', False),
#                     'dynamic': response.get('dynamic', False),
#                     'additionalDimensions': response.get('additionalDimensions', []),
#                     'cmPlacementId': response.get('cmPlacementId', ''),
#                     'cmTrackingAd.cmPlacementId': response.get('cmTrackingAd', {}).get('cmPlacementId', ''),
#                     'cmTrackingAd.cmCreativeId': response.get('cmTrackingAd', {}).get('cmCreativeId', ''),
#                     'cmTrackingAd.cmAdId': response.get('cmTrackingAd', {}).get('cmAdId', ''),
#                     'companionCreativeIds': response.get('companionCreativeIds', [])
#                 }

#                 # Append the creative details to the list
#                 all_CreativeDetails.append(creativeDetails)

#     except Exception as e:
#         print(f"An error occurred for advertiser ID {advertiser}: {e}")

# # Convert the accumulated list to a DataFrame
# dataFrame_Advertiser_CreativeId_CreativeDetail = pd.DataFrame(all_CreativeDetails)

## List the AdGroup IDs for each advertiser and the AdGroup Ads
- Create the AdGroup ID and AdGroup Ads dataframe
  - dataFrame_Advertiser_AdGroupIds
  - dataFrame_Advertiser_AdGroupAds

Using: [advertisers.adgroupads.list](https://developers.google.com/display-video/api/reference/rest/v3/advertisers.adgroupads/list) then [advertisers.adgroupads.get](https://developers.google.com/display-video/api/reference/rest/v3/advertisers.adgroupads/get)

In [19]:
## Live working code

## V2 - Updated to remove both list then get. Uses just List as the response contains all the details needed
## V2.1 Updated to expand upon dataframe columns

# Initialize the DataFrame with the specified structure for all AdGroupAds types
dataFrame_AdGroupAds_List = pd.DataFrame({
    'name': pd.Series(dtype='str'),
    'advertiserId': pd.Series(dtype='int'),
    'adGroupAdId': pd.Series(dtype='str'),
    'adGroupId': pd.Series(dtype='str'),
    'displayName': pd.Series(dtype='str'),
    'entityStatus': pd.Series(dtype='str'),

    # Ad URLs
    'adUrls.type': pd.Series(dtype='str'),
    'adUrls.url': pd.Series(dtype='str'),

    # displayVideoSourceAd fields
    'displayVideoSourceAd.creativeId': pd.Series(dtype='str'),

    # mastheadAd fields
    'mastheadAd.video.id': pd.Series(dtype='str'),
    'mastheadAd.video.unavailableReason': pd.Series(dtype='str'),
    'mastheadAd.headline': pd.Series(dtype='str'),
    'mastheadAd.description': pd.Series(dtype='str'),
    'mastheadAd.videoAspectRatio': pd.Series(dtype='str'),
    'mastheadAd.autoplayVideoStartMillisecond': pd.Series(dtype='str'),
    'mastheadAd.autoplayVideoDuration': pd.Series(dtype='str'),
    'mastheadAd.callToActionButtonLabel': pd.Series(dtype='str'),
    'mastheadAd.callToActionFinalUrl': pd.Series(dtype='str'),
    'mastheadAd.callToActionTrackingUrl': pd.Series(dtype='str'),
    'mastheadAd.showChannelArt': pd.Series(dtype='bool'),
    'mastheadAd.companionYoutubeVideos.id': pd.Series(dtype='str'),
    'mastheadAd.companionYoutubeVideos.unavailableReason': pd.Series(dtype='str'),

    # inStreamAd fields
    'inStreamAd.commonInStreamAttribute.displayUrl': pd.Series(dtype='str'),
    'inStreamAd.commonInStreamAttribute.finalUrl': pd.Series(dtype='str'),
    'inStreamAd.commonInStreamAttribute.trackingUrl': pd.Series(dtype='str'),
    'inStreamAd.commonInStreamAttribute.actionButtonLabel': pd.Series(dtype='str'),
    'inStreamAd.commonInStreamAttribute.actionHeadline': pd.Series(dtype='str'),
    'inStreamAd.commonInStreamAttribute.video.id': pd.Series(dtype='str'),
    'inStreamAd.commonInStreamAttribute.video.unavailableReason': pd.Series(dtype='str'),
    'inStreamAd.commonInStreamAttribute.companionBanner.mimeType': pd.Series(dtype='str'),
    'inStreamAd.commonInStreamAttribute.companionBanner.fullSize.widthPixels': pd.Series(dtype='int'),
    'inStreamAd.commonInStreamAttribute.companionBanner.fullSize.heightPixels': pd.Series(dtype='int'),
    'inStreamAd.commonInStreamAttribute.companionBanner.fileSize': pd.Series(dtype='int'),
    'inStreamAd.customParameters.string': pd.Series(dtype='str'),

    # nonSkippableAd fields
    'nonSkippableAd.commonInStreamAttribute.displayUrl': pd.Series(dtype='str'),
    'nonSkippableAd.commonInStreamAttribute.finalUrl': pd.Series(dtype='str'),
    'nonSkippableAd.commonInStreamAttribute.trackingUrl': pd.Series(dtype='str'),
    'nonSkippableAd.commonInStreamAttribute.actionButtonLabel': pd.Series(dtype='str'),
    'nonSkippableAd.commonInStreamAttribute.actionHeadline': pd.Series(dtype='str'),
    'nonSkippableAd.commonInStreamAttribute.video.id': pd.Series(dtype='str'),
    'nonSkippableAd.commonInStreamAttribute.video.unavailableReason': pd.Series(dtype='str'),
    'nonSkippableAd.commonInStreamAttribute.companionBanner.mimeType': pd.Series(dtype='str'),
    'nonSkippableAd.commonInStreamAttribute.companionBanner.fullSize.widthPixels': pd.Series(dtype='int'),
    'nonSkippableAd.commonInStreamAttribute.companionBanner.fullSize.heightPixels': pd.Series(dtype='int'),
    'nonSkippableAd.commonInStreamAttribute.companionBanner.fileSize': pd.Series(dtype='int'),
    'nonSkippableAd.customParameters.string': pd.Series(dtype='str'),

    # bumperAd fields
    'bumperAd.commonInStreamAttribute.displayUrl': pd.Series(dtype='str'),
    'bumperAd.commonInStreamAttribute.finalUrl': pd.Series(dtype='str'),
    'bumperAd.commonInStreamAttribute.trackingUrl': pd.Series(dtype='str'),
    'bumperAd.commonInStreamAttribute.actionButtonLabel': pd.Series(dtype='str'),
    'bumperAd.commonInStreamAttribute.actionHeadline': pd.Series(dtype='str'),
    'bumperAd.commonInStreamAttribute.video.id': pd.Series(dtype='str'),
    'bumperAd.commonInStreamAttribute.video.unavailableReason': pd.Series(dtype='str'),
    'bumperAd.commonInStreamAttribute.companionBanner.mimeType': pd.Series(dtype='str'),
    'bumperAd.commonInStreamAttribute.companionBanner.fullSize.widthPixels': pd.Series(dtype='int'),
    'bumperAd.commonInStreamAttribute.companionBanner.fullSize.heightPixels': pd.Series(dtype='int'),
    'bumperAd.commonInStreamAttribute.companionBanner.fileSize': pd.Series(dtype='int'),

    # audioAd fields
    'audioAd.displayUrl': pd.Series(dtype='str'),
    'audioAd.finalUrl': pd.Series(dtype='str'),
    'audioAd.trackingUrl': pd.Series(dtype='str'),
    'audioAd.video.id': pd.Series(dtype='str'),
    'audioAd.video.unavailableReason': pd.Series(dtype='str'),

    # videoDiscoverAd fields
    'videoDiscoverAd.headline': pd.Series(dtype='str'),
    'videoDiscoverAd.description1': pd.Series(dtype='str'),
    'videoDiscoverAd.description2': pd.Series(dtype='str'),
    'videoDiscoverAd.video.id': pd.Series(dtype='str'),
    'videoDiscoverAd.video.unavailableReason': pd.Series(dtype='str'),
    'videoDiscoverAd.thumbnail': pd.Series(dtype='str'),

    # videoPerformanceAd fields
    'videoPerformanceAd.finalUrl': pd.Series(dtype='str'),
    'videoPerformanceAd.trackingUrl': pd.Series(dtype='str'),
    'videoPerformanceAd.actionButtonLabels': pd.Series(dtype='str'),
    'videoPerformanceAd.headlines': pd.Series(dtype='str'),
    'videoPerformanceAd.longHeadlines': pd.Series(dtype='str'),
    'videoPerformanceAd.descriptions': pd.Series(dtype='str'),
    'videoPerformanceAd.displayUrlBreadcrumb1': pd.Series(dtype='str'),
    'videoPerformanceAd.displayUrlBreadcrumb2': pd.Series(dtype='str'),
    'videoPerformanceAd.domain': pd.Series(dtype='str'),
    'videoPerformanceAd.videos.id': pd.Series(dtype='str'),
    'videoPerformanceAd.videos.unavailableReason': pd.Series(dtype='str'),
    'videoPerformanceAd.customParameters.string': pd.Series(dtype='str'),
    'videoPerformanceAd.companionBanner.mimeType': pd.Series(dtype='str'),
    'videoPerformanceAd.companionBanner.fullSize.widthPixels': pd.Series(dtype='int'),
    'videoPerformanceAd.companionBanner.fullSize.heightPixels': pd.Series(dtype='int'),
    'videoPerformanceAd.companionBanner.fileSize': pd.Series(dtype='int')
})


# Loop through each advertiser to fetch and populate the AdGroupAds data
Advertiser_AdGroupAds = []

for advertiser in dataFrame_Advertisers_List["advertiserId"]:
    try:
        # API request to list AdGroupAds
        request = service.advertisers().adGroupAds().list(advertiserId=advertiser)
        response = request.execute()

        # Ensure 'adGroupAds' key exists in the response
        if 'adGroupAds' in response:
            response_advertiser_AdGroupAds = pd.json_normalize(response['adGroupAds'])

            # Iterate over each adGroupAd in the response
            for _, row in response_advertiser_AdGroupAds.iterrows():
                adGroupAd = {
                    'name': row.get('name', ''),
                    'advertiserId': advertiser,
                    'adGroupAdId': row.get('adGroupAdId', ''),
                    'adGroupId': row.get('adGroupId', ''),
                    'displayName': row.get('displayName', ''),
                    'entityStatus': row.get('entityStatus', ''),
                    'adUrls.type': row.get('adUrls.type', ''),
                    'adUrls.url': row.get('adUrls.url', ''),

                    # displayVideoSourceAd
                    'displayVideoSourceAd.creativeId': row.get('displayVideoSourceAd.creativeId', ''),

                    # mastheadAd
                    'mastheadAd.video.id': row.get('mastheadAd.video.id', ''),
                    'mastheadAd.video.unavailableReason': row.get('mastheadAd.video.unavailableReason', ''),
                    'mastheadAd.headline': row.get('mastheadAd.headline', ''),
                    'mastheadAd.description': row.get('mastheadAd.description', ''),
                    'mastheadAd.videoAspectRatio': row.get('mastheadAd.videoAspectRatio', ''),
                    'mastheadAd.autoplayVideoStartMillisecond': row.get('mastheadAd.autoplayVideoStartMillisecond', ''),
                    'mastheadAd.autoplayVideoDuration': row.get('mastheadAd.autoplayVideoDuration', ''),
                    'mastheadAd.callToActionButtonLabel': row.get('mastheadAd.callToActionButtonLabel', ''),
                    'mastheadAd.callToActionFinalUrl': row.get('mastheadAd.callToActionFinalUrl', ''),
                    'mastheadAd.callToActionTrackingUrl': row.get('mastheadAd.callToActionTrackingUrl', ''),
                    'mastheadAd.showChannelArt': row.get('mastheadAd.showChannelArt', ''),
                    'mastheadAd.companionYoutubeVideos.id': row.get('mastheadAd.companionYoutubeVideos.id', ''),
                    'mastheadAd.companionYoutubeVideos.unavailableReason': row.get('mastheadAd.companionYoutubeVideos.unavailableReason', ''),

                    # inStreamAd
                    'inStreamAd.commonInStreamAttribute.displayUrl': row.get('inStreamAd.commonInStreamAttribute.displayUrl', ''),
                    'inStreamAd.commonInStreamAttribute.finalUrl': row.get('inStreamAd.commonInStreamAttribute.finalUrl', ''),
                    'inStreamAd.commonInStreamAttribute.trackingUrl': row.get('inStreamAd.commonInStreamAttribute.trackingUrl', ''),
                    'inStreamAd.commonInStreamAttribute.actionButtonLabel': row.get('inStreamAd.commonInStreamAttribute.actionButtonLabel', ''),
                    'inStreamAd.commonInStreamAttribute.actionHeadline': row.get('inStreamAd.commonInStreamAttribute.actionHeadline', ''),
                    'inStreamAd.commonInStreamAttribute.video.id': row.get('inStreamAd.commonInStreamAttribute.video.id', ''),
                    'inStreamAd.commonInStreamAttribute.video.unavailableReason': row.get('inStreamAd.commonInStreamAttribute.video.unavailableReason', ''),
                    'inStreamAd.commonInStreamAttribute.companionBanner.mimeType': row.get('inStreamAd.commonInStreamAttribute.companionBanner.mimeType', ''),
                    'inStreamAd.commonInStreamAttribute.companionBanner.fullSize.widthPixels': row.get('inStreamAd.commonInStreamAttribute.companionBanner.fullSize.widthPixels', ''),
                    'inStreamAd.commonInStreamAttribute.companionBanner.fullSize.heightPixels': row.get('inStreamAd.commonInStreamAttribute.companionBanner.fullSize.heightPixels', ''),
                    'inStreamAd.commonInStreamAttribute.companionBanner.fileSize': row.get('inStreamAd.commonInStreamAttribute.companionBanner.fileSize', ''),
                    'inStreamAd.customParameters.string': row.get('inStreamAd.customParameters.string', ''),

                    # nonSkippableAd
                    'nonSkippableAd.commonInStreamAttribute.displayUrl': row.get('nonSkippableAd.commonInStreamAttribute.displayUrl', ''),
                    'nonSkippableAd.commonInStreamAttribute.finalUrl': row.get('nonSkippableAd.commonInStreamAttribute.finalUrl', ''),
                    'nonSkippableAd.commonInStreamAttribute.trackingUrl': row.get('nonSkippableAd.commonInStreamAttribute.trackingUrl', ''),
                    'nonSkippableAd.commonInStreamAttribute.actionButtonLabel': row.get('nonSkippableAd.commonInStreamAttribute.actionButtonLabel', ''),
                    'nonSkippableAd.commonInStreamAttribute.actionHeadline': row.get('nonSkippableAd.commonInStreamAttribute.actionHeadline', ''),
                    'nonSkippableAd.commonInStreamAttribute.video.id': row.get('nonSkippableAd.commonInStreamAttribute.video.id', ''),
                    'nonSkippableAd.commonInStreamAttribute.video.unavailableReason': row.get('nonSkippableAd.commonInStreamAttribute.video.unavailableReason', ''),
                    'nonSkippableAd.commonInStreamAttribute.companionBanner.mimeType': row.get('nonSkippableAd.commonInStreamAttribute.companionBanner.mimeType', ''),
                    'nonSkippableAd.commonInStreamAttribute.companionBanner.fullSize.widthPixels': row.get('nonSkippableAd.commonInStreamAttribute.companionBanner.fullSize.widthPixels', ''),
                    'nonSkippableAd.commonInStreamAttribute.companionBanner.fullSize.heightPixels': row.get('nonSkippableAd.commonInStreamAttribute.companionBanner.fullSize.heightPixels', ''),
                    'nonSkippableAd.commonInStreamAttribute.companionBanner.fileSize': row.get('nonSkippableAd.commonInStreamAttribute.companionBanner.fileSize', ''),
                    'nonSkippableAd.customParameters.string': row.get('nonSkippableAd.customParameters.string', ''),

                    # bumperAd
                    'bumperAd.commonInStreamAttribute.displayUrl': row.get('bumperAd.commonInStreamAttribute.displayUrl', ''),
                    'bumperAd.commonInStreamAttribute.finalUrl': row.get('bumperAd.commonInStreamAttribute.finalUrl', ''),
                    'bumperAd.commonInStreamAttribute.trackingUrl': row.get('bumperAd.commonInStreamAttribute.trackingUrl', ''),
                    'bumperAd.commonInStreamAttribute.actionButtonLabel': row.get('bumperAd.commonInStreamAttribute.actionButtonLabel', ''),
                    'bumperAd.commonInStreamAttribute.actionHeadline': row.get('bumperAd.commonInStreamAttribute.actionHeadline', ''),
                    'bumperAd.commonInStreamAttribute.video.id': row.get('bumperAd.commonInStreamAttribute.video.id', ''),
                    'bumperAd.commonInStreamAttribute.video.unavailableReason': row.get('bumperAd.commonInStreamAttribute.video.unavailableReason', ''),
                    'bumperAd.commonInStreamAttribute.companionBanner.mimeType': row.get('bumperAd.commonInStreamAttribute.companionBanner.mimeType', ''),
                    'bumperAd.commonInStreamAttribute.companionBanner.fullSize.widthPixels': row.get('bumperAd.commonInStreamAttribute.companionBanner.fullSize.widthPixels', ''),
                    'bumperAd.commonInStreamAttribute.companionBanner.fullSize.heightPixels': row.get('bumperAd.commonInStreamAttribute.companionBanner.fullSize.heightPixels', ''),
                    'bumperAd.commonInStreamAttribute.companionBanner.fileSize': row.get('bumperAd.commonInStreamAttribute.companionBanner.fileSize', ''),

                    # audioAd
                    'audioAd.displayUrl': row.get('audioAd.displayUrl', ''),
                    'audioAd.finalUrl': row.get('audioAd.finalUrl', ''),
                    'audioAd.trackingUrl': row.get('audioAd.trackingUrl', ''),
                    'audioAd.video.id': row.get('audioAd.video.id', ''),
                    'audioAd.video.unavailableReason': row.get('audioAd.video.unavailableReason', ''),

                    # videoDiscoverAd
                    'videoDiscoverAd.headline': row.get('videoDiscoverAd.headline', ''),
                    'videoDiscoverAd.description1': row.get('videoDiscoverAd.description1', ''),
                    'videoDiscoverAd.description2': row.get('videoDiscoverAd.description2', ''),
                    'videoDiscoverAd.video.id': row.get('videoDiscoverAd.video.id', ''),
                    'videoDiscoverAd.video.unavailableReason': row.get('videoDiscoverAd.video.unavailableReason', ''),
                    'videoDiscoverAd.thumbnail': row.get('videoDiscoverAd.thumbnail', ''),

                    # videoPerformanceAd
                    'videoPerformanceAd.finalUrl': row.get('videoPerformanceAd.finalUrl', ''),
                    'videoPerformanceAd.trackingUrl': row.get('videoPerformanceAd.trackingUrl', ''),
                    'videoPerformanceAd.actionButtonLabels': row.get('videoPerformanceAd.actionButtonLabels', ''),
                    'videoPerformanceAd.headlines': row.get('videoPerformanceAd.headlines', ''),
                    'videoPerformanceAd.longHeadlines': row.get('videoPerformanceAd.longHeadlines', ''),
                    'videoPerformanceAd.descriptions': row.get('videoPerformanceAd.descriptions', ''),
                    'videoPerformanceAd.displayUrlBreadcrumb1': row.get('videoPerformanceAd.displayUrlBreadcrumb1', ''),
                    'videoPerformanceAd.displayUrlBreadcrumb2': row.get('videoPerformanceAd.displayUrlBreadcrumb2', ''),
                    'videoPerformanceAd.domain': row.get('videoPerformanceAd.domain', ''),
                    'videoPerformanceAd.videos.id': row.get('videoPerformanceAd.videos.id', ''),
                    'videoPerformanceAd.videos.unavailableReason': row.get('videoPerformanceAd.videos.unavailableReason', ''),
                    'videoPerformanceAd.customParameters.string': row.get('videoPerformanceAd.videos.customParameters.string', ''),
                    'videoPerformanceAd.companionBanner.mimeType': row.get('videoPerformanceAd.companionBanner.mimeType', ''),
                    'videoPerformanceAd.companionBanner.fullSize.widthPixels': row.get('videoPerformanceAd.companionBanner.fullSize.widthPixels', ''),
                    'videoPerformanceAd.companionBanner.fullSize.heightPixels': row.get('videoPerformanceAd.companionBanner.fullSize.heightPixels', ''),
                    'videoPerformanceAd.companionBanner.fileSize': row.get('videoPerformanceAd.companionBanner.fileSize', ''),
                }

                # Append the adGroupAd details to the list
                Advertiser_AdGroupAds.append(adGroupAd)

    except Exception as e:
        print(f"An error occurred for advertiser ID {advertiser}: {e}")

# Convert the accumulated list to a DataFrame
dataFrame_Advertiser_AdGroupAds = pd.DataFrame(Advertiser_AdGroupAds)

In [20]:
## Old Code

# # Initialize lists to store accumulated data for all advertisers
# Advertiser_AdGroupIds = []
# Advertiser_AdGroupAds = []

# # Loop through each advertiser
# for advertiser in dataFrame_Advertisers_List["advertiserId"]:
#     try:
#         # API request to list the Advertiser AdGroupAds
#         request = service.advertisers().adGroupAds().list(advertiserId=advertiser)
#         response = request.execute()

#         # Ensure 'adGroupAds' key exists in the response
#         if 'adGroupAds' in response:
#             response_advertiser_AdGroupAds = pd.json_normalize(response['adGroupAds'])

#             # Append the data to Advertiser_AdGroupIds
#             for _, row in response_advertiser_AdGroupAds.iterrows():
#                 row_dict = row.to_dict()
#                 row_dict['advertiserId'] = advertiser
#                 Advertiser_AdGroupIds.append(row_dict)

#                 adGroupAdId = row['adGroupAdId']

#                 # Request to get the AdGroupAd details
#                 request = service.advertisers().adGroupAds().get(
#                     advertiserId=advertiser,
#                     adGroupAdId=adGroupAdId
#                 )
#                 response = request.execute()

#                 # Extract relevant details from the response
#                 adGroupAds = {
#                     'name': response.get('name', ''),
#                     'advertiserId': advertiser,
#                     'adGroupAdId': adGroupAdId,
#                     'adGroupId': row.get('adGroupId', ''),
#                     'displayName': response.get('displayName', ''),
#                     'entityStatus': response.get('entityStatus', ''),
#                     'adUrl': response.get('adUrl', ''),
#                     'displayVideoSourceAd': response.get('displayVideoSourceAd', ''),
#                     'mastheadAd': response.get('mastheadAd', ''),
#                     'inStreamAd': response.get('inStreamAd', ''),
#                     'nonSkippableAd': response.get('nonSkippableAd', ''),
#                     'bumperAd': response.get('bumperAd', ''),
#                     'audioAd': response.get('audioAd', ''),
#                     'videoDiscoverAd': response.get('videoDiscoverAd', ''),
#                     'videoPerformanceAd': response.get('videoPerformanceAd', '')
#                 }

#                 # Append data to the list for AdGroupAd details
#                 Advertiser_AdGroupAds.append(adGroupAds)

#     except Exception as e:
#         print(f"An error occurred for advertiser ID {advertiser}: {e}")

# # Convert the accumulated lists to DataFrames
# dataFrame_Advertiser_AdGroupIds = pd.DataFrame(Advertiser_AdGroupIds)
# dataFrame_Advertiser_AdGroupAds = pd.DataFrame(Advertiser_AdGroupAds)

## Data & Directories - Turn all list dataFrames into a CSV
- Turn all inclusion or exclusion lists into CSV in their own directory
- Turn all Creative and AdGroup files into CSV in their own directory

In [96]:
# Define the directory to save the CSV files
data_directory = '2.1 Inclusions and Exclusions'

# Create the directory if it doesn't exist
if not os.path.exists(data_directory):
    os.makedirs(data_directory)

# Dictionary of DataFrames to save
includeExclude_dataframes_dict = {
    'dataFrame_Partner_ChannelIdSiteId': dataFrame_Partner_ChannelIdSiteId,
    'dataFrame_Advertiser_ChannelIdSiteId': dataFrame_Advertiser_ChannelIdSiteId,
    'dataFrame_Advertiser_NegativeKeywordListsNegativeKeywords': dataFrame_Advertiser_NegativeKeywordListsNegativeKeywords
}

# Save each DataFrame as a CSV file in the specified directory
for df_name, df in includeExclude_dataframes_dict.items():
    file_path = os.path.join(data_directory, f'{df_name}.csv')
    df.to_csv(file_path, index=False)
    print(f"Saved {df_name} to {file_path}")


Saved dataFrame_Partner_ChannelIdSiteId to 2.1 Inclusions and Exclusions/dataFrame_Partner_ChannelIdSiteId.csv
Saved dataFrame_Advertiser_ChannelIdSiteId to 2.1 Inclusions and Exclusions/dataFrame_Advertiser_ChannelIdSiteId.csv
Saved dataFrame_Advertiser_NegativeKeywordListsNegativeKeywords to 2.1 Inclusions and Exclusions/dataFrame_Advertiser_NegativeKeywordListsNegativeKeywords.csv


In [95]:
# Define the directory to save the CSV files
data_directory = '2.2 Creative and AdGroups'

# Create the directory if it doesn't exist
if not os.path.exists(data_directory):
    os.makedirs(data_directory)

# Dictionary of DataFrames to save
creativeAdGroups_dataframes_dict = {
    'dataFrame_Advertiser_CreativeId_CreativeDetail': dataFrame_Advertiser_CreativeId_CreativeDetail,
    'dataFrame_Advertiser_AdGroupAds': dataFrame_Advertiser_AdGroupAds
}

# Save each DataFrame as a CSV file in the specified directory
for df_name, df in creativeAdGroups_dataframes_dict.items():
    file_path = os.path.join(data_directory, f'{df_name}.csv')
    df.to_csv(file_path, index=False)
    print(f"Saved {df_name} to {file_path}")

Saved dataFrame_Advertiser_CreativeId_CreativeDetail to 2.2 Creative and AdGroups/dataFrame_Advertiser_CreativeId_CreativeDetail.csv
Saved dataFrame_Advertiser_AdGroupAds to 2.2 Creative and AdGroups/dataFrame_Advertiser_AdGroupAds.csv


# Targeting Types

## Targeting Types and their options

In [88]:
# Assuming you have already set up the service and list_Advertiser
advertiserId = list_Advertiser[0]['advertiserId']

def fetch_targeting_data(targeting_types, service):
    # Dictionary to store dataframes
    dataframes = {}

    for ttype in targeting_types:
        try:
            request = service.targetingTypes().targetingOptions().list(
                targetingType=ttype,
                advertiserId=advertiserId
            )
            response = request.execute()

            # Convert the response to a dataframe
            df_name = 'dataframe_' + ttype
            dataframes[df_name] = pd.json_normalize(response['targetingOptions'])

        except Exception as e:
            print(f"An error occurred for targeting type {ttype}: {e}")

    return dataframes

# List of targeting types
TARGETING_TYPES = [
    'TARGETING_TYPE_APP_CATEGORY',
    'TARGETING_TYPE_AGE_RANGE',
    'TARGETING_TYPE_GENDER',
    'TARGETING_TYPE_VIDEO_PLAYER_SIZE',
    'TARGETING_TYPE_USER_REWARDED_CONTENT',
    'TARGETING_TYPE_PARENTAL_STATUS',
    'TARGETING_TYPE_CONTENT_INSTREAM_POSITION',
    'TARGETING_TYPE_CONTENT_OUTSTREAM_POSITION',
    'TARGETING_TYPE_DEVICE_TYPE',
    'TARGETING_TYPE_BROWSER',
    'TARGETING_TYPE_HOUSEHOLD_INCOME',
    'TARGETING_TYPE_ON_SCREEN_POSITION',
    'TARGETING_TYPE_CARRIER_AND_ISP',
    'TARGETING_TYPE_OPERATING_SYSTEM',
    'TARGETING_TYPE_DEVICE_MAKE_MODEL',
    'TARGETING_TYPE_ENVIRONMENT',
    'TARGETING_TYPE_CATEGORY',
    'TARGETING_TYPE_VIEWABILITY',
    'TARGETING_TYPE_AUTHORIZED_SELLER_STATUS',
    'TARGETING_TYPE_LANGUAGE',
    'TARGETING_TYPE_GEO_REGION',
    'TARGETING_TYPE_DIGITAL_CONTENT_LABEL_EXCLUSION',
    'TARGETING_TYPE_SENSITIVE_CATEGORY_EXCLUSION',
    'TARGETING_TYPE_EXCHANGE',
    'TARGETING_TYPE_SUB_EXCHANGE',
    'TARGETING_TYPE_NATIVE_CONTENT_POSITION',
    'TARGETING_TYPE_OMID'
]

## Data & Directories - Turn all list dataFrames into a CSV

In [97]:
# Define the directory to save the CSV files
data_directory = '2.3 Targeting Type'

# Create the directory if it does not exist
if not os.path.exists(data_directory):
    os.makedirs(data_directory)

# Call the function to fetch dataframes for targeting types
targetingType_dataframes_dict = fetch_targeting_data(TARGETING_TYPES, service)

# Save each DataFrame as a CSV file in the specified directory
for df_name, df in targetingType_dataframes_dict.items():
    file_path = os.path.join(data_directory, f'{df_name}.csv')
    df.to_csv(file_path, index=False)
    print(f"Saved {df_name} to {file_path}")

Saved dataframe_TARGETING_TYPE_APP_CATEGORY to 2.3 Targeting Type/dataframe_TARGETING_TYPE_APP_CATEGORY.csv
Saved dataframe_TARGETING_TYPE_AGE_RANGE to 2.3 Targeting Type/dataframe_TARGETING_TYPE_AGE_RANGE.csv
Saved dataframe_TARGETING_TYPE_GENDER to 2.3 Targeting Type/dataframe_TARGETING_TYPE_GENDER.csv
Saved dataframe_TARGETING_TYPE_VIDEO_PLAYER_SIZE to 2.3 Targeting Type/dataframe_TARGETING_TYPE_VIDEO_PLAYER_SIZE.csv
Saved dataframe_TARGETING_TYPE_USER_REWARDED_CONTENT to 2.3 Targeting Type/dataframe_TARGETING_TYPE_USER_REWARDED_CONTENT.csv
Saved dataframe_TARGETING_TYPE_PARENTAL_STATUS to 2.3 Targeting Type/dataframe_TARGETING_TYPE_PARENTAL_STATUS.csv
Saved dataframe_TARGETING_TYPE_CONTENT_INSTREAM_POSITION to 2.3 Targeting Type/dataframe_TARGETING_TYPE_CONTENT_INSTREAM_POSITION.csv
Saved dataframe_TARGETING_TYPE_CONTENT_OUTSTREAM_POSITION to 2.3 Targeting Type/dataframe_TARGETING_TYPE_CONTENT_OUTSTREAM_POSITION.csv
Saved dataframe_TARGETING_TYPE_DEVICE_TYPE to 2.3 Targeting Type/d

## Targeting Types Mapping

- Create the targeting_details_mapping dataframe
- Shows targeting types, their options and an index to sort

Using: [advertisers.adGroups.targetingTypes.assignedTargetingOptions](https://developers.google.com/display-video/api/reference/rest/v3/advertisers.adGroups.targetingTypes.assignedTargetingOptions#AssignedTargetingOption)

**Targeting by Level**

<ul type="disc">
<li>Partner</li>
  <ul>
    <li>TARGETING_TYPE_CHANNEL</li>
  </ul>
<li>Advertiser</li>
  <ul>
    <li>TARGETING_TYPE_CHANNEL</li>
    <li>TARGETING_TYPE_DIGITAL_CONTENT_LABEL_EXCLUSION</li>
    <li>TARGETING_TYPE_OMID</li>
    <li>TARGETING_TYPE_SENSITIVE_CATEGORY_EXCLUSION</li>
    <li>TARGETING_TYPE_YOUTUBE_VIDEO</li>
    <li>TARGETING_TYPE_YOUTUBE_CHANNEL</li>
    <li>TARGETING_TYPE_KEYWORD</li>
  </ul>
<li>Insertion Order & Line Item</li>
  <ul>
    <li>TARGETING_TYPE_AGE_RANGE</li>
    <li>TARGETING_TYPE_APP</li>
    <li>TARGETING_TYPE_APP_CATEGORY</li>
    <li>TARGETING_TYPE_AUDIENCE_GROUP</li>
    <li>TARGETING_TYPE_AUDIO_CONTENT_TYPE</li>
    <li>TARGETING_TYPE_AUTHORIZED_SELLER_STATUS</li>
    <li>TARGETING_TYPE_BROWSER</li>
    <li>TARGETING_TYPE_BUSINESS_CHAIN</li>
    <li>TARGETING_TYPE_CARRIER_AND_ISP</li>
    <li>TARGETING_TYPE_CATEGORY</li>
    <li>TARGETING_TYPE_CHANNEL</li>
    <li>TARGETING_TYPE_CONTENT_DURATION</li>
    <li>TARGETING_TYPE_CONTENT_GENRE</li>
    <li>TARGETING_TYPE_CONTENT_INSTREAM_POSITION</li>
    <li>TARGETING_TYPE_CONTENT_OUTSTREAM_POSITION</li>
    <li>TARGETING_TYPE_CONTENT_STREAM_TYPE</li>
    <li>TARGETING_TYPE_DAY_AND_TIME</li>
    <li>TARGETING_TYPE_DEVICE_MAKE_MODEL</li>
    <li>TARGETING_TYPE_DEVICE_TYPE</li>
    <li>TARGETING_TYPE_DIGITAL_CONTENT_LABEL_EXCLUSION</li>
    <li>TARGETING_TYPE_ENVIRONMENT</li>
    <li>TARGETING_TYPE_GENDER</li>
    <li>TARGETING_TYPE_GEO_REGION</li>
    <li>TARGETING_TYPE_HOUSEHOLD_INCOME</li>
    <li>TARGETING_TYPE_INVENTORY_SOURCE</li>
    <li>TARGETING_TYPE_INVENTORY_SOURCE_GROUP</li>
    <li>TARGETING_TYPE_KEYWORD</li>
    <li>TARGETING_TYPE_LANGUAGE</li>
    <li>TARGETING_TYPE_NATIVE_CONTENT_POSITION</li>
    <li>TARGETING_TYPE_NEGATIVE_KEYWORD_LIST</li>
    <li>TARGETING_TYPE_OMID</li>
    <li>TARGETING_TYPE_ON_SCREEN_POSITION</li>
    <li>TARGETING_TYPE_OPERATING_SYSTEM</li>
    <li>TARGETING_TYPE_PARENTAL_STATUS</li>
    <li>TARGETING_TYPE_POI</li>
    <li>TARGETING_TYPE_PROXIMITY_LOCATION_LIST</li>
    <li>TARGETING_TYPE_REGIONAL_LOCATION_LIST</li>
    <li>TARGETING_TYPE_SENSITIVE_CATEGORY_EXCLUSION</li>
    <li>TARGETING_TYPE_SUB_EXCHANGE</li>
    <li>TARGETING_TYPE_THIRD_PARTY_VERIFIER</li>
    <li>TARGETING_TYPE_URL</li>
    <li>TARGETING_TYPE_USER_REWARDED_CONTENT</li>
    <li>TARGETING_TYPE_VIDEO_PLAYER_SIZE</li>
    <li>TARGETING_TYPE_VIEWABILITY</li>
  </ul>
<li>Unique to Line Item (only for LINE_ITEM_TYPE_YOUTUBE_AND_PARTNERS_VIDEO_SEQUENCE line items)</li>
  <ul>
    <li>TARGETING_TYPE_YOUTUBE_CHANNEL</li>
    <li>TARGETING_TYPE_YOUTUBE_VIDEO</li>
  </ul>

In [24]:
## TESTING A NEW ORDER FOR GEO

targeting_details_mapping = {
    'TARGETING_TYPE_AGE_RANGE': ('assignedTargetingOptionId', 'ageRangeDetails.ageRange', 0, 'ageRangeDetails'),
    'TARGETING_TYPE_APP': ('assignedTargetingOptionId', 'appDetails.appId', 'appDetails.displayName', 'appDetails.negative', 'appDetails.appPlatform', 0, 'appDetails'),
    'TARGETING_TYPE_APP_CATEGORY': ('assignedTargetingOptionId', 'appCategoryDetails.displayName', 'appCategoryDetails.targetingOptionId', 'appCategoryDetails.negative', 0, 'appCategoryDetails'),
    'TARGETING_TYPE_AUDIENCE_GROUP': (
        'assignedTargetingOptionId',
        'audienceGroupDetails.includedFirstAndThirdPartyAudienceGroups',
        'audienceGroupDetails.includedGoogleAudienceGroup',
        'audienceGroupDetails.includedCustomListGroup',
        'audienceGroupDetails.includedCombinedAudienceGroup',
        'audienceGroupDetails.excludedFirstAndThirdPartyAudienceGroup',
        'audienceGroupDetails.excludedGoogleAudienceGroup',
        0,
        'audienceGroupDetails'
    ),
    'TARGETING_TYPE_AUDIO_CONTENT_TYPE': ('assignedTargetingOptionId', 'audioContentTypeDetails.audioContentType', 0, 'audioContentTypeDetails'),
    'TARGETING_TYPE_AUTHORIZED_SELLER_STATUS': ('assignedTargetingOptionId', 'authorizedSellerStatusDetails.authorizedSellerStatus', 'authorizedSellerStatusDetails.targetingOptionId', 0, 'authorizedSellerStatusDetails'),
    'TARGETING_TYPE_BROWSER': ('assignedTargetingOptionId', 'browserDetails.displayName', 'browserDetails.targetingOptionId', 'browserDetails.negative', 0, 'browserDetails'),
    'TARGETING_TYPE_BUSINESS_CHAIN': ('assignedTargetingOptionId', 'businessChainDetails.businessChainId', 'businessChainDetails.displayName', 'businessChainDetails.proximityRadiusAmount', 'businessChainDetails.proximityRadiusUnit', 0, 'businessChainDetails'),
    'TARGETING_TYPE_CARRIER_AND_ISP': ('assignedTargetingOptionId', 'carrierAndIspDetails.displayName', 'carrierAndIspDetails.targetingOptionId', 'carrierAndIspDetails.negative', 0, 'carrierAndIspDetails'),
    'TARGETING_TYPE_CATEGORY': ('assignedTargetingOptionId', 'categoryDetails.displayName', 'categoryDetails.targetingOptionId', 'categoryDetails.negative', 0, 'categoryDetails'),
    'TARGETING_TYPE_CHANNEL': ('assignedTargetingOptionId', 'channelDetails.channelId', 'channelDetails.negative', 0, 'channelDetails'),
    'TARGETING_TYPE_CONTENT_DURATION': ('assignedTargetingOptionId', 'contentDurationDetails.contentDuration', 0, 'contentDurationDetails'),
    'TARGETING_TYPE_CONTENT_GENRE': ('assignedTargetingOptionId', 'contentGenreDetails.displayName', 'contentGenreDetails.targetingOptionId', 'contentGenreDetails.negative', 0, 'contentGenreDetails'),
    'TARGETING_TYPE_CONTENT_INSTREAM_POSITION': ('assignedTargetingOptionId', 'contentInstreamPositionDetails.contentInstreamPosition', 'contentInstreamPositionDetails.adType', 0, 'contentInstreamPositionDetails'),
    'TARGETING_TYPE_CONTENT_OUTSTREAM_POSITION': ('assignedTargetingOptionId', 'contentOutstreamPositionDetails.contentOutstreamPosition', 'contentOutstreamPositionDetails.adType', 0, 'contentOutstreamPositionDetails'),
    'TARGETING_TYPE_CONTENT_STREAM_TYPE': ('assignedTargetingOptionId', 'contentStreamTypeDetails.contentStreamType', 0, 'contentStreamTypeDetails'),
    'TARGETING_TYPE_DAY_AND_TIME': ('assignedTargetingOptionId', 'dayAndTimeDetails.dayOfWeek', 'dayAndTimeDetails.startHour', 'dayAndTimeDetails.endHour', 'dayAndTimeDetails.timeZoneResolution', 0, 'dayAndTimeDetails'),
    'TARGETING_TYPE_DEVICE_MAKE_MODEL': ('assignedTargetingOptionId', 'deviceMakeModelDetails.displayName', 'deviceMakeModelDetails.targetingOptionId', 'deviceMakeModelDetails.negative', 0, 'deviceMakeModelDetails'),
    'TARGETING_TYPE_DEVICE_TYPE': ('assignedTargetingOptionId', 'deviceTypeDetails.deviceType', 'deviceTypeDetails.youtubeAndPartnersBidMultiplier', 0, 'deviceTypeDetails'),
    'TARGETING_TYPE_DIGITAL_CONTENT_LABEL_EXCLUSION': ('assignedTargetingOptionId', 'digitalContentLabelExclusionDetails.excludedContentRatingTier', 0, 'digitalContentLabelExclusionDetails'),
    'TARGETING_TYPE_ENVIRONMENT': ('assignedTargetingOptionId', 'environmentDetails.environment', 0, 'environmentDetails'),
    'TARGETING_TYPE_EXCHANGE': ('assignedTargetingOptionId', 'exchangeDetails.exchange', 0, 'exchangeDetails'),
    'TARGETING_TYPE_GENDER': ('assignedTargetingOptionId', 'genderDetails.gender', 0, 'genderDetails'),
    'TARGETING_TYPE_GEO_REGION': ('geoRegionDetails.displayName', 'assignedTargetingOptionId', 'geoRegionDetails.geoRegionType', 'geoRegionDetails.negative', 0, 'geoRegionDetails'),
    'TARGETING_TYPE_HOUSEHOLD_INCOME': ('assignedTargetingOptionId', 'householdIncomeDetails.householdIncome', 0, 'householdIncomeDetails'),
    'TARGETING_TYPE_INVENTORY_SOURCE': ('assignedTargetingOptionId', 'inventorySourceDetails.inventorySourceId', 0, 'inventorySourceDetails'),
    'TARGETING_TYPE_INVENTORY_SOURCE_GROUP': ('assignedTargetingOptionId', 'inventorySourceGroupDetails.inventorySourceGroupId', 0, 'inventorySourceGroupDetails'),
    'TARGETING_TYPE_KEYWORD': ('assignedTargetingOptionId', 'keywordDetails.keyword', 'keywordDetails.negative', 0, 'keywordDetails'),
    'TARGETING_TYPE_LANGUAGE': ('assignedTargetingOptionId', 'languageDetails.displayName', 'languageDetails.targetingOptionId', 'languageDetails.negative', 0, 'languageDetails'),
    'TARGETING_TYPE_NATIVE_CONTENT_POSITION': ('assignedTargetingOptionId', 'nativeContentPositionDetails.position', 'nativeContentPositionDetails.targetingOptionId', 0, 'nativeContentPositionDetails'),
    'TARGETING_TYPE_NEGATIVE_KEYWORD_LIST': ('assignedTargetingOptionId', 'negativeKeywordListDetails.negativeKeywordListId', 0, 'negativeKeywordListDetails'),
    'TARGETING_TYPE_OMID': ('assignedTargetingOptionId', 'omidDetails.omid', 0, 'omidDetails'),
    'TARGETING_TYPE_ON_SCREEN_POSITION': ('assignedTargetingOptionId', 'onScreenPositionDetails.onScreenPosition', 'onScreenPositionDetails.targetingOptionId', 'onScreenPositionDetails.adType', 0, 'onScreenPositionDetails'),
    'TARGETING_TYPE_OPERATING_SYSTEM': ('assignedTargetingOptionId', 'operatingSystemDetails.displayName', 'operatingSystemDetails.targetingOptionId', 'operatingSystemDetails.negative', 0, 'operatingSystemDetails'),
    'TARGETING_TYPE_PARENTAL_STATUS': ('assignedTargetingOptionId', 'parentalStatusDetails.parentalStatus', 0, 'parentalStatusDetails'),
    'TARGETING_TYPE_POI': ('assignedTargetingOptionId', 'poiDetails.poiId', 'poiDetails.displayName', 0, 'poiDetails'),
    'TARGETING_TYPE_PROXIMITY_LOCATION_LIST': ('assignedTargetingOptionId', 'proximityLocationListDetails.proximityLocationListId', 'proximityLocationListDetails.proximityRadius', 'proximityLocationListDetails.proximityRadiusUnit', 0, 'proximityLocationListDetails'),
    'TARGETING_TYPE_REGIONAL_LOCATION_LIST': ('assignedTargetingOptionId', 'regionalLocationListDetails.regionalLocationListId', 'regionalLocationListDetails.negative', 0, 'regionalLocationListDetails'),
    'TARGETING_TYPE_SENSITIVE_CATEGORY_EXCLUSION': ('assignedTargetingOptionId', 'sensitiveCategoryExclusionDetails.excludedSensitiveCategory', 0, 'sensitiveCategoryExclusionDetails'),
    'TARGETING_TYPE_SUB_EXCHANGE': ('assignedTargetingOptionId', 'subExchangeDetails.targetingOptionId', 0, 'subExchangeDetails'),
    'TARGETING_TYPE_THIRD_PARTY_VERIFIER': ('assignedTargetingOptionId', 'thirdPartyVerifierDetails.adloox', 'thirdPartyVerifierDetails.doubleVerify', 'thirdPartyVerifierDetails.integralAdScience', 0, 'thirdPartyVerifierDetails'),
    'TARGETING_TYPE_URL': ('assignedTargetingOptionId', 'urlDetails.url', 'urlDetails.negative', 0, 'urlDetails'),
    'TARGETING_TYPE_USER_REWARDED_CONTENT': ('assignedTargetingOptionId', 'userRewardedContentDetails.targetingOptionId', 'userRewardedContentDetails.userRewardedContent', 0, 'userRewardedContentDetails'),
    'TARGETING_TYPE_VIDEO_PLAYER_SIZE': ('assignedTargetingOptionId', 'videoPlayerSizeDetails.videoPlayerSize', 0, 'videoPlayerSizeDetails'),
    'TARGETING_TYPE_VIEWABILITY': ('assignedTargetingOptionId', 'viewabilityDetails.viewability', 'viewabilityDetails.targetingOptionId', 0, 'viewabilityDetails'),
    'TARGETING_TYPE_YOUTUBE_CHANNEL': ('assignedTargetingOptionId', 'youtubeChannelDetails.channelId', 'youtubeChannelDetails.targetingOptionId', 0, 'youtubeChannelDetails'),
    'TARGETING_TYPE_YOUTUBE_VIDEO': ('assignedTargetingOptionId', 'youtubeVideoDetails.videoId', 'youtubeVideoDetails.targetingOptionId', 0, 'youtubeVideoDetails')
}


In [25]:
targeting_details_mapping = {
    'TARGETING_TYPE_AGE_RANGE': ('assignedTargetingOptionId', 'ageRangeDetails.ageRange', 0, 'ageRangeDetails'),
    'TARGETING_TYPE_APP': ('assignedTargetingOptionId', 'appDetails.appId', 'appDetails.displayName', 'appDetails.negative', 'appDetails.appPlatform', 0, 'appDetails'),
    'TARGETING_TYPE_APP_CATEGORY': ('assignedTargetingOptionId', 'appCategoryDetails.displayName', 'appCategoryDetails.targetingOptionId', 'appCategoryDetails.negative', 0, 'appCategoryDetails'),
    'TARGETING_TYPE_AUDIENCE_GROUP': (
        'assignedTargetingOptionId',
        'audienceGroupDetails.includedFirstAndThirdPartyAudienceGroups',
        'audienceGroupDetails.includedGoogleAudienceGroup',
        'audienceGroupDetails.includedCustomListGroup',
        'audienceGroupDetails.includedCombinedAudienceGroup',
        'audienceGroupDetails.excludedFirstAndThirdPartyAudienceGroup',
        'audienceGroupDetails.excludedGoogleAudienceGroup',
        0,  # Sort by first group ID
        'audienceGroupDetails'
    ),
    'TARGETING_TYPE_AUDIO_CONTENT_TYPE': ('assignedTargetingOptionId', 'audioContentTypeDetails.audioContentType', 0, 'audioContentTypeDetails'),
    'TARGETING_TYPE_AUTHORIZED_SELLER_STATUS': ('assignedTargetingOptionId', 'authorizedSellerStatusDetails.authorizedSellerStatus', 'authorizedSellerStatusDetails.targetingOptionId', 0, 'authorizedSellerStatusDetails'),
    'TARGETING_TYPE_BROWSER': ('assignedTargetingOptionId', 'browserDetails.displayName', 'browserDetails.targetingOptionId', 'browserDetails.negative', 0, 'browserDetails'),
    'TARGETING_TYPE_BUSINESS_CHAIN': ('assignedTargetingOptionId', 'businessChainDetails.businessChainId', 'businessChainDetails.displayName', 'businessChainDetails.proximityRadiusAmount', 'businessChainDetails.proximityRadiusUnit', 0, 'businessChainDetails'),
    'TARGETING_TYPE_CARRIER_AND_ISP': ('assignedTargetingOptionId', 'carrierAndIspDetails.displayName', 'carrierAndIspDetails.targetingOptionId', 'carrierAndIspDetails.negative', 0, 'carrierAndIspDetails'),
    'TARGETING_TYPE_CATEGORY': ('assignedTargetingOptionId', 'categoryDetails.displayName', 'categoryDetails.targetingOptionId', 'categoryDetails.negative', 0, 'categoryDetails'),
    'TARGETING_TYPE_CHANNEL': ('assignedTargetingOptionId', 'channelDetails.channelId', 'channelDetails.negative', 0, 'channelDetails'),
    'TARGETING_TYPE_CONTENT_DURATION': ('assignedTargetingOptionId', 'contentDurationDetails.contentDuration', 0, 'contentDurationDetails'),
    'TARGETING_TYPE_CONTENT_GENRE': ('assignedTargetingOptionId', 'contentGenreDetails.displayName', 'contentGenreDetails.targetingOptionId', 'contentGenreDetails.negative', 0, 'contentGenreDetails'),
    'TARGETING_TYPE_CONTENT_INSTREAM_POSITION': ('assignedTargetingOptionId', 'contentInstreamPositionDetails.contentInstreamPosition', 'contentInstreamPositionDetails.adType', 0, 'contentInstreamPositionDetails'),
    'TARGETING_TYPE_CONTENT_OUTSTREAM_POSITION': ('assignedTargetingOptionId', 'contentOutstreamPositionDetails.contentOutstreamPosition', 'contentOutstreamPositionDetails.adType', 0, 'contentOutstreamPositionDetails'),
    'TARGETING_TYPE_CONTENT_STREAM_TYPE': ('assignedTargetingOptionId', 'contentStreamTypeDetails.contentStreamType', 0, 'contentStreamTypeDetails'),
    'TARGETING_TYPE_DAY_AND_TIME': ('assignedTargetingOptionId', 'dayAndTimeDetails.dayOfWeek', 'dayAndTimeDetails.startHour', 'dayAndTimeDetails.endHour', 'dayAndTimeDetails.timeZoneResolution', 0, 'dayAndTimeDetails'),
    'TARGETING_TYPE_DEVICE_MAKE_MODEL': ('assignedTargetingOptionId', 'deviceMakeModelDetails.displayName', 'deviceMakeModelDetails.targetingOptionId', 'deviceMakeModelDetails.negative', 0, 'deviceMakeModelDetails'),
    'TARGETING_TYPE_DEVICE_TYPE': ('assignedTargetingOptionId', 'deviceTypeDetails.deviceType', 'deviceTypeDetails.youtubeAndPartnersBidMultiplier', 0, 'deviceTypeDetails'),
    'TARGETING_TYPE_DIGITAL_CONTENT_LABEL_EXCLUSION': ('assignedTargetingOptionId', 'digitalContentLabelExclusionDetails.excludedContentRatingTier', 0, 'digitalContentLabelExclusionDetails'),
    'TARGETING_TYPE_ENVIRONMENT': ('assignedTargetingOptionId', 'environmentDetails.environment', 0, 'environmentDetails'),
    'TARGETING_TYPE_EXCHANGE': ('assignedTargetingOptionId', 'exchangeDetails.exchange', 0, 'exchangeDetails'),
    'TARGETING_TYPE_GENDER': ('assignedTargetingOptionId', 'genderDetails.gender', 0, 'genderDetails'),
    'TARGETING_TYPE_GEO_REGION': ('assignedTargetingOptionId', 'geoRegionDetails.displayName', 'geoRegionDetails.targetingOptionId', 'geoRegionDetails.geoRegionType', 'geoRegionDetails.negative', 0, 'geoRegionDetails'),
    'TARGETING_TYPE_HOUSEHOLD_INCOME': ('assignedTargetingOptionId', 'householdIncomeDetails.householdIncome', 0, 'householdIncomeDetails'),
    'TARGETING_TYPE_INVENTORY_SOURCE': ('assignedTargetingOptionId', 'inventorySourceDetails.inventorySourceId', 0, 'inventorySourceDetails'),
    'TARGETING_TYPE_INVENTORY_SOURCE_GROUP': ('assignedTargetingOptionId', 'inventorySourceGroupDetails.inventorySourceGroupId', 0, 'inventorySourceGroupDetails'),
    'TARGETING_TYPE_KEYWORD': ('assignedTargetingOptionId', 'keywordDetails.keyword', 'keywordDetails.negative', 0, 'keywordDetails'),
    'TARGETING_TYPE_LANGUAGE': ('assignedTargetingOptionId', 'languageDetails.displayName', 'languageDetails.targetingOptionId', 'languageDetails.negative', 0, 'languageDetails'),
    'TARGETING_TYPE_NATIVE_CONTENT_POSITION': ('assignedTargetingOptionId', 'nativeContentPositionDetails.position', 'nativeContentPositionDetails.targetingOptionId', 0, 'nativeContentPositionDetails'),
    'TARGETING_TYPE_NEGATIVE_KEYWORD_LIST': ('assignedTargetingOptionId', 'negativeKeywordListDetails.negativeKeywordListId', 0, 'negativeKeywordListDetails'),
    'TARGETING_TYPE_OMID': ('assignedTargetingOptionId', 'omidDetails.omid', 0, 'omidDetails'),
    'TARGETING_TYPE_ON_SCREEN_POSITION': ('assignedTargetingOptionId', 'onScreenPositionDetails.onScreenPosition', 'onScreenPositionDetails.targetingOptionId', 'onScreenPositionDetails.adType', 0, 'onScreenPositionDetails'),
    'TARGETING_TYPE_OPERATING_SYSTEM': ('assignedTargetingOptionId', 'operatingSystemDetails.displayName', 'operatingSystemDetails.targetingOptionId', 'operatingSystemDetails.negative', 0, 'operatingSystemDetails'),
    'TARGETING_TYPE_PARENTAL_STATUS': ('assignedTargetingOptionId', 'parentalStatusDetails.parentalStatus', 0, 'parentalStatusDetails'),
    'TARGETING_TYPE_POI': ('assignedTargetingOptionId', 'poiDetails.poiId', 'poiDetails.displayName', 0, 'poiDetails'),
    'TARGETING_TYPE_PROXIMITY_LOCATION_LIST': ('assignedTargetingOptionId', 'proximityLocationListDetails.proximityLocationListId', 'proximityLocationListDetails.proximityRadius', 'proximityLocationListDetails.proximityRadiusUnit', 0, 'proximityLocationListDetails'),
    'TARGETING_TYPE_REGIONAL_LOCATION_LIST': ('assignedTargetingOptionId', 'regionalLocationListDetails.regionalLocationListId', 'regionalLocationListDetails.negative', 0, 'regionalLocationListDetails'),
    'TARGETING_TYPE_SENSITIVE_CATEGORY_EXCLUSION': ('assignedTargetingOptionId', 'sensitiveCategoryExclusionDetails.excludedSensitiveCategory', 0, 'sensitiveCategoryExclusionDetails'),
    'TARGETING_TYPE_SUB_EXCHANGE': ('assignedTargetingOptionId', 'subExchangeDetails.targetingOptionId', 0, 'subExchangeDetails'),
    'TARGETING_TYPE_THIRD_PARTY_VERIFIER': ('assignedTargetingOptionId', 'thirdPartyVerifierDetails.adloox', 'thirdPartyVerifierDetails.doubleVerify', 'thirdPartyVerifierDetails.integralAdScience', 0, 'thirdPartyVerifierDetails'),
    'TARGETING_TYPE_URL': ('assignedTargetingOptionId', 'urlDetails.url', 'urlDetails.negative', 0, 'urlDetails'),
    'TARGETING_TYPE_USER_REWARDED_CONTENT': ('assignedTargetingOptionId', 'userRewardedContentDetails.targetingOptionId', 'userRewardedContentDetails.userRewardedContent', 0, 'userRewardedContentDetails'),
    'TARGETING_TYPE_VIDEO_PLAYER_SIZE': ('assignedTargetingOptionId', 'videoPlayerSizeDetails.videoPlayerSize', 0, 'videoPlayerSizeDetails'),
    'TARGETING_TYPE_VIEWABILITY': ('assignedTargetingOptionId', 'viewabilityDetails.viewability', 'viewabilityDetails.targetingOptionId', 0, 'viewabilityDetails'),
    'TARGETING_TYPE_YOUTUBE_CHANNEL': ('assignedTargetingOptionId', 'youtubeChannelDetails.channelId', 'youtubeChannelDetails.targetingOptionId', 0, 'youtubeChannelDetails'),
    'TARGETING_TYPE_YOUTUBE_VIDEO': ('assignedTargetingOptionId', 'youtubeVideoDetails.videoId', 'youtubeVideoDetails.targetingOptionId', 0, 'youtubeVideoDetails')
}


# Assigned Targeting Options

## Functions for use
Functions to get the targeting information and if necessarily sort

### get_targeting_details

In [26]:
def get_targeting_details(LevelTargetingResponse, targetingTypeColumnValue, *values):
    try:
        # Filter based on targetingType
        targetingDetails = LevelTargetingResponse[LevelTargetingResponse['targetingType'] == targetingTypeColumnValue].copy()

        # Adjustments for columns ending with '.negative'
        for val in values:
            if '.negative' in val:
                if val in targetingDetails.columns:
                    targetingDetails[val] = targetingDetails[val].apply(lambda x: True if x == True else False)
                else:
                    targetingDetails[val] = [False] * len(targetingDetails)  # Set default value to False

        # Extract column values
        output = []
        for val in values:
            if val in targetingDetails.columns:
                output.append(targetingDetails[val].tolist())
            else:
                output.append([None] * len(targetingDetails))

        # Flatten the output if only one attribute is being returned
        if len(values) == 1:
            output = output[0]

    except Exception as e:
        print(f"An error occurred: {e}")
        output = [None for _ in values]

    return output

### sort_by_column

In [27]:
def sort_by_column(details, sort_column_index):
    if all(item is None for item in details) or not isinstance(details[0], (list, type(None))):
        return details

    if not all(len(sublist) > sort_column_index if sublist is not None else False for sublist in details):
        return details

    try:
        # Create an order list based on the specified sublist
        order = sorted(range(len(details[sort_column_index])), key=lambda k: details[sort_column_index][k])

        # Reorder the sublists based on the created order
        sorted_details = [sorted(item, key=lambda x: order[item.index(x)]) if isinstance(item, list) else item for item in details]

        return sorted_details
    except IndexError as e:
        print(f"IndexError in sort_by_column: {e} for details: {details}")
        return details

### process_targeting_details

In [28]:
# Generalized function to extract and process targeting details for any context (Partner, Advertiser, LineItem, etc.)
def process_targeting_details(filtered_response, targeting_type):
    targeting_fields = targeting_details_mapping[targeting_type]
    required_columns = ['assignedTargetingOption.' + field for field in targeting_fields[:-2]]

    # Adjust for potential missing .negative columns and other expected columns
    for col in required_columns:
        if '.negative' in col:  # Check if the column has ".negative" in its name
            if col not in filtered_response.columns:
                filtered_response = filtered_response.copy()
                filtered_response.loc[:, col] = False  # Add the .negative column with default False
            else:
                filtered_response = filtered_response.copy()  # Ensure we're working with a copy
                filtered_response.loc[:, col] = filtered_response[col].apply(lambda x: False if pd.isna(x) or x == '' else x)
        else:
            if col not in filtered_response.columns:
                filtered_response = filtered_response.copy()
                filtered_response.loc[:, col] = np.nan  # Set missing columns to NaN using .loc

    # Check if all required columns exist in the filtered response
    if all(col in filtered_response.columns for col in required_columns):
        # Select the relevant fields based on the mapping
        filtered_details = filtered_response[required_columns]

        # Sort the details according to the specified index in the mapping
        sorting_column_index = targeting_fields[-2]  # Sorting is based on a field specified in the mapping
        sorted_details = filtered_details.sort_values(by=[required_columns[sorting_column_index]])

        # Convert the sorted DataFrame to a list of lists (each inner list corresponds to a row)
        return sorted_details.values.tolist()
    else:
        missing_columns = [col for col in required_columns if col not in filtered_response.columns]
        print(f"Missing columns for {targeting_type}: {missing_columns}")
        return None


In [29]:
## Testing
def process_targeting_details(filtered_response, targeting_type, level='standard'):
    """
    Function to process and format targeting details based on the structure of the response.

    Arguments:
    - filtered_response: The response data after filtering.
    - targeting_type: The targeting type being processed.
    - level: String indicating the structure of the response ('standard' for Partner/Advertiser/Campaign/Insertion Order, 'lineItem' for LineItem).

    Returns:
    - A formatted list of lists with the processed targeting details.
    """
    targeting_fields = targeting_details_mapping[targeting_type]

    # Adjust columns for LineItem vs. standard responses
    if level == 'lineItem':
        required_columns = ['assignedTargetingOption.' + field for field in targeting_fields[:-2]]
    else:
        required_columns = ['assignedTargetingOptionId'] + [field.split('.')[-1] for field in targeting_fields[:-2]]

    # Adjust for potential missing .negative columns and other expected columns
    for col in required_columns:
        if '.negative' in col:  # This is the equivalent of a wildcard check
            if col not in filtered_response.columns:
                filtered_response = filtered_response.copy()
                filtered_response[col] = False  # Add the .negative column with default False
            else:
                filtered_response = filtered_response.copy()
                filtered_response[col] = filtered_response[col].apply(lambda x: False if pd.isna(x) or x == '' else x)
        else:
            if col not in filtered_response.columns:
                filtered_response = filtered_response.copy()
                filtered_response[col] = np.nan  # Set missing columns to NaN

    # Extract required fields for standard and lineItem responses
    if level == 'lineItem':
        filtered_details = filtered_response.apply(lambda row: [
            row['assignedTargetingOption.' + field] for field in targeting_fields[:-2]
        ], axis=1).tolist()
    else:
        filtered_details = filtered_response.apply(lambda row: [
            row[field] for field in required_columns
        ], axis=1).tolist()

    # Check if all required columns exist in the filtered response
    if all(col in filtered_response.columns for col in required_columns):
        # Sort the details according to the specified index in the mapping
        sorting_column_index = targeting_fields[-2]
        sorted_details = sorted(filtered_details, key=lambda x: x[sorting_column_index])

        # Return the sorted details
        return sorted_details
    else:
        missing_columns = [col for col in required_columns if col not in filtered_response.columns]
        print(f"Missing columns for {targeting_type}: {missing_columns}")
        return None


## Partner

### Live

In [30]:
# V3 Updated to new output format

# Initialize the DataFrame with dynamic columns based on the specified targeting types
dataFrame_Partner_AssignedTargetingOptions = pd.DataFrame({'partnerId': []})

# List of targeting types to iterate over
targeting_types_list = [
    'TARGETING_TYPE_CHANNEL'
]

# Dynamically add columns based on the selected targeting types in targeting_types_list
for targeting_type in targeting_types_list:
    if targeting_type in targeting_details_mapping:
        dataFrame_Partner_AssignedTargetingOptions[f'{targeting_type}'] = []

# Use the provided `targeting_details_mapping` to fetch columns and sort index
def process_targeting_details(partnerListResponse, targeting_type):
    if targeting_type not in targeting_details_mapping:
        return []

    # Extract columns and sorting index for the current targeting type
    targeting_fields = targeting_details_mapping[targeting_type]
    sorting_column_index = targeting_fields[-2] if isinstance(targeting_fields[-2], int) else 0

    # Get targeting details using the predefined function
    targeting_details = get_targeting_details(
        partnerListResponse,
        targeting_type,
        *targeting_fields[:-2]  # Unpack the fields to be passed to the function
    )

    # Flatten and sort targeting details
    flattened_details = list(zip(*targeting_details))
    sorted_details = sort_by_column(flattened_details, sorting_column_index)

    # Format as list of lists for each targeting option (if required by output format)
    formatted_details = [
        [entry[i] for i in range(len(entry))] for entry in sorted_details
    ]

    return formatted_details

# Loop through each partner ID and targeting type
for PID in partnerIdList:
    for targeting_type in targeting_types_list:
        try:
            # Print the current partner ID and targeting type
            print(f"Processing partner ID: {PID} for targeting type: {targeting_type}")

            # Make the API request for each combination of partner and targeting type
            request = service.partners().targetingTypes().assignedTargetingOptions().list(
                partnerId=PID,
                targetingType=targeting_type
            )
            response = request.execute()

            # Initialize a dictionary to hold the data for this partner and targeting type
            new_row = {'partnerId': PID}

            # Process the targeting details in the response
            if 'assignedTargetingOptions' in response:
                partnerListResponse = pd.json_normalize(response['assignedTargetingOptions'])

                if not partnerListResponse.empty:
                    # Process and format the targeting details
                    targeting_details = process_targeting_details(partnerListResponse, targeting_type)
                    new_row[f'{targeting_type}'] = targeting_details
                else:
                    new_row[f'{targeting_type}'] = None  # If no data, set to None

            # Append the completed row for this partner and targeting type to the DataFrame
            dataFrame_Partner_AssignedTargetingOptions = pd.concat(
                [dataFrame_Partner_AssignedTargetingOptions, pd.DataFrame([new_row])],
                ignore_index=True
            )

        except Exception as e:
            print(f"An error occurred for PID {PID} and targeting type {targeting_type}: {e}")

# Output the final DataFrame
print(dataFrame_Partner_AssignedTargetingOptions)


Processing partner ID: 1407943 for targeting type: TARGETING_TYPE_CHANNEL
Processing partner ID: 941894575 for targeting type: TARGETING_TYPE_CHANNEL
   partnerId  TARGETING_TYPE_CHANNEL
0    1407943                     NaN
1  941894575                     NaN


### Testing

In [31]:
# Most recent but overridden in Testing
# # Initialize the DataFrame with dynamic columns based on the specified targeting types
# dataFrame_Partner_AssignedTargetingOptions = pd.DataFrame({
#     'partnerId': []
# })

# # List of targeting types to iterate over
# targeting_types_list = [
#     'TARGETING_TYPE_CHANNEL'
# ]

# # Dynamically add columns based on the selected targeting types in targeting_types_list
# for targeting_type in targeting_types_list:
#     if targeting_type in targeting_details_mapping:
#         dataFrame_Partner_AssignedTargetingOptions[f'{targeting_type}'] = []

# # Function to extract and process targeting details
# def process_targeting_details(partnerListResponse, targeting_type):
#     targeting_fields = targeting_details_mapping[targeting_type]
#     targeting_details = get_targeting_details(
#         partnerListResponse,
#         targeting_type,
#         *targeting_fields[:-2]  # Unpack the fields to be passed to the function
#     )
#     sorting_column_index = targeting_fields[-2]
#     targeting_details = sort_by_column(targeting_details, sorting_column_index)
#     return targeting_details

# # Loop through each partner ID and targeting type
# for PID in partnerIdList:
#     for targeting_type in targeting_types_list:
#         try:
#             # Print the current partner ID and targeting type
#             print(f"Processing partner ID: {PID} for targeting type: {targeting_type}")

#             # Make the API request for each combination of partner and targeting type
#             request = service.partners().targetingTypes().assignedTargetingOptions().list(
#                 partnerId=PID,
#                 targetingType=targeting_type
#             )
#             response = request.execute()

#             # Initialize a dictionary to hold the data for this partner and targeting type
#             new_row = {'partnerId': PID}

#             # Process the targeting details in the response
#             if 'assignedTargetingOptions' in response:
#                 partnerListResponse = pd.json_normalize(response['assignedTargetingOptions'])

#                 if not partnerListResponse.empty:
#                     # Process the targeting details for the current targeting type
#                     targeting_details = process_targeting_details(partnerListResponse, targeting_type)
#                     new_row[f'{targeting_type}'] = targeting_details
#                 else:
#                     new_row[f'{targeting_type}'] = None  # If no data, set to None

#             # Append the completed row for this partner and targeting type to the DataFrame
#             dataFrame_Partner_AssignedTargetingOptions = pd.concat([dataFrame_Partner_AssignedTargetingOptions, pd.DataFrame([new_row])], ignore_index=True)

#         except Exception as e:
#             print(f"An error occurred for PID {PID} and targeting type {targeting_type}: {e}")

# # Output the final DataFrame
# print(dataFrame_Partner_AssignedTargetingOptions)


In [32]:
# # Initialize the DataFrame with dynamic columns based on the specified targeting types
# dataFrame_Partner_AssignedTargetingOptions = pd.DataFrame({
#     'partnerId': []
# })

# # List of targeting types to iterate over
# targeting_types_list = [
#     'TARGETING_TYPE_CHANNEL'
# ]

# # Dynamically add columns based on the selected targeting types in targeting_types_list
# for targeting_type in targeting_types_list:
#     if targeting_type in targeting_details_mapping:
#         dataFrame_Partner_AssignedTargetingOptions[f'{targeting_type}'] = []


# # Loop through each partner ID and targeting type
# for PID in partnerIdList:
#     for targeting_type in targeting_types_list:
#         try:
#             print(f"Processing partner ID: {PID} for targeting type: {targeting_type}")
#             request = service.partners().targetingTypes().assignedTargetingOptions().list(partnerId=PID, targetingType=targeting_type)
#             response = request.execute()

#             new_row = {'partnerId': PID}

#             if 'assignedTargetingOptions' in response:
#                 partnerListResponse = pd.json_normalize(response['assignedTargetingOptions'])
#                 if not partnerListResponse.empty:
#                     targeting_details = process_targeting_details(partnerListResponse, targeting_type)
#                     new_row[f'{targeting_type}'] = targeting_details
#                 else:
#                     new_row[f'{targeting_type}'] = None

#             dataFrame_Partner_AssignedTargetingOptions = pd.concat([dataFrame_Partner_AssignedTargetingOptions, pd.DataFrame([new_row])], ignore_index=True)

#         except Exception as e:
#             print(f"An error occurred for PID {PID} and targeting type {targeting_type}: {e}")

# # Output the final DataFrame
# print(dataFrame_Partner_AssignedTargetingOptions)


In [33]:
# # Initialize the DataFrame with dynamic columns based on the specified targeting types
# dataFrame_Partner_AssignedTargetingOptions = pd.DataFrame({
#     'partnerId': []
# })

# # List of targeting types to iterate over
# targeting_types_list = [
#     'TARGETING_TYPE_CHANNEL'
# ]

# # Dynamically add columns based on the selected targeting types in targeting_types_list
# for targeting_type in targeting_types_list:
#     if targeting_type in targeting_details_mapping:
#         dataFrame_Partner_AssignedTargetingOptions[f'{targeting_type}'] = []

# # Function to extract and process targeting details
# def process_targeting_details(partnerListResponse, targeting_type):
#     targeting_fields = targeting_details_mapping[targeting_type]
#     targeting_details = get_targeting_details(
#         partnerListResponse,
#         targeting_type,
#         *targeting_fields[:-2]  # Unpack the fields to be passed to the function
#     )
#     sorting_column_index = targeting_fields[-2]
#     targeting_details = sort_by_column(targeting_details, sorting_column_index)
#     return targeting_details

# # Loop through each partner ID and targeting type
# for PID in partnerIdList:
#     for targeting_type in targeting_types_list:
#         try:
#             # Print the current partner ID and targeting type
#             print(f"Processing partner ID: {PID} for targeting type: {targeting_type}")

#             # Make the API request for each combination of partner and targeting type
#             request = service.partners().targetingTypes().assignedTargetingOptions().list(
#                 partnerId=PID,
#                 targetingType=targeting_type
#             )
#             response = request.execute()

#             # Initialize a dictionary to hold the data for this partner and targeting type
#             new_row = {'partnerId': PID}

#             # Process the targeting details in the response
#             if 'assignedTargetingOptions' in response:
#                 partnerListResponse = pd.json_normalize(response['assignedTargetingOptions'])

#                 if not partnerListResponse.empty:
#                     # Process the targeting details for the current targeting type
#                     targeting_details = process_targeting_details(partnerListResponse, targeting_type)
#                     new_row[f'{targeting_type}'] = targeting_details
#                 else:
#                     new_row[f'{targeting_type}'] = None  # If no data, set to None

#             # Append the completed row for this partner and targeting type to the DataFrame
#             dataFrame_Partner_AssignedTargetingOptions = pd.concat([dataFrame_Partner_AssignedTargetingOptions, pd.DataFrame([new_row])], ignore_index=True)

#         except Exception as e:
#             print(f"An error occurred for PID {PID} and targeting type {targeting_type}: {e}")

# # Output the final DataFrame
# print(dataFrame_Partner_AssignedTargetingOptions)


## Advertiser

### Live

In [34]:
# V3 Updated with correct [[id, name, negative], [id, name, negative]] values
dataFrame_Advertiser_AssignedTargetingOptions = pd.DataFrame({'advertiserId': []})

# List of targeting types to use
targeting_types_list = [
    'TARGETING_TYPE_CHANNEL',
    'TARGETING_TYPE_DIGITAL_CONTENT_LABEL_EXCLUSION',
    'TARGETING_TYPE_OMID',
    'TARGETING_TYPE_SENSITIVE_CATEGORY_EXCLUSION',
    'TARGETING_TYPE_YOUTUBE_VIDEO',
    'TARGETING_TYPE_YOUTUBE_CHANNEL',
    'TARGETING_TYPE_KEYWORD'
]

# Dynamically add columns based on the selected targeting types in targeting_types_list
for targeting_type in targeting_types_list:
    dataFrame_Advertiser_AssignedTargetingOptions[f'{targeting_type}'] = []

# Use the provided `targeting_details_mapping` to fetch columns and sort index
def process_targeting_details(advertiserListResponse, targeting_type):
    if targeting_type not in targeting_details_mapping:
        return []

    # Extract columns and sorting index for the current targeting type
    targeting_fields = targeting_details_mapping[targeting_type]
    sorting_column_index = targeting_fields[-2] if isinstance(targeting_fields[-2], int) else 0

    # Get targeting details using the predefined function
    targeting_details = get_targeting_details(
        advertiserListResponse,
        targeting_type,
        *targeting_fields[:-2]  # Unpack the fields to be passed to the function
    )

    # Flatten and sort targeting details
    flattened_details = list(zip(*targeting_details))
    sorted_details = sort_by_column(flattened_details, sorting_column_index)

    # Format as list of lists for each targeting option (if required by output format)
    formatted_details = [
        [entry[i] for i in range(len(entry))] for entry in sorted_details
    ]

    return formatted_details

# Loop through each advertiser ID and retrieve their assigned targeting options
for AID in advertiserIdList:
    try:
        print(f"Processing advertiser Id: {AID}")

        # Single API request for all targeting types
        request = service.advertisers().listAssignedTargetingOptions(advertiserId=AID)
        response = request.execute()

        # Initialize a dictionary for this advertiser's data
        new_row = {'advertiserId': AID}

        # Process each targeting type in the response
        if 'assignedTargetingOptions' in response:
            advertiserListResponse = pd.json_normalize(response['assignedTargetingOptions'])

            for targeting_type in targeting_types_list:
                if targeting_type in advertiserListResponse['targetingType'].values:
                    # Process and format the targeting details
                    targeting_details = process_targeting_details(advertiserListResponse, targeting_type)
                    new_row[f'{targeting_type}'] = targeting_details
                else:
                    new_row[f'{targeting_type}'] = None  # Set to None if not present

        # Append the advertiser data to the DataFrame
        dataFrame_Advertiser_AssignedTargetingOptions = pd.concat(
            [dataFrame_Advertiser_AssignedTargetingOptions, pd.DataFrame([new_row])],
            ignore_index=True
        )

    except Exception as e:
        print(f"An error occurred for AID {AID}: {e}")

# # Output the final DataFrame
# print(dataFrame_Advertiser_AssignedTargetingOptions)

Processing advertiser Id: 2690451
Processing advertiser Id: 2888635
Processing advertiser Id: 2979132
Processing advertiser Id: 3123344
Processing advertiser Id: 3123345
Processing advertiser Id: 3337341
Processing advertiser Id: 5356338
Processing advertiser Id: 5713331
Processing advertiser Id: 5727633
Processing advertiser Id: 523034835
Processing advertiser Id: 576381488
Processing advertiser Id: 578717665
Processing advertiser Id: 579214935
Processing advertiser Id: 581732152
Processing advertiser Id: 626560303
Processing advertiser Id: 630315971
Processing advertiser Id: 643634603
Processing advertiser Id: 645763109
Processing advertiser Id: 682575986
Processing advertiser Id: 690314094
Processing advertiser Id: 708298499
Processing advertiser Id: 710945674
Processing advertiser Id: 721042762
Processing advertiser Id: 761687671
Processing advertiser Id: 831087538
Processing advertiser Id: 863903687
Processing advertiser Id: 874597431
Processing advertiser Id: 977951733
Processing

### Testing

In [35]:
# # V2 Testing a new format
# # Initialize the DataFrame with dynamic columns based on the specified targeting types
# dataFrame_Advertiser_AssignedTargetingOptions = pd.DataFrame({
#     'advertiserId': [],
# })

# # List of targeting types to use
# targeting_types_list = [
#     'TARGETING_TYPE_CHANNEL',
#     'TARGETING_TYPE_DIGITAL_CONTENT_LABEL_EXCLUSION',
#     'TARGETING_TYPE_OMID',
#     'TARGETING_TYPE_SENSITIVE_CATEGORY_EXCLUSION',
#     'TARGETING_TYPE_YOUTUBE_VIDEO',
#     'TARGETING_TYPE_YOUTUBE_CHANNEL',
#     'TARGETING_TYPE_KEYWORD'
# ]

# # Dynamically add columns based on the selected targeting types in targeting_types_list
# for targeting_type in targeting_types_list:
#     if targeting_type in targeting_details_mapping:
#         dataFrame_Advertiser_AssignedTargetingOptions[f'{targeting_type}'] = []

# # Function to extract and process targeting details
# def process_targeting_details(advertiserListResponse, targeting_type):
#     targeting_fields = targeting_details_mapping[targeting_type]
#     targeting_details = get_targeting_details(
#         advertiserListResponse,
#         targeting_type,
#         *targeting_fields[:-2]  # Unpack the fields to be passed to the function
#     )
#     sorting_column_index = targeting_fields[-2]
#     targeting_details = sort_by_column(targeting_details, sorting_column_index)
#     return targeting_details

# # Loop through each advertiser Id
# for AID in advertiserIdList:
#     try:
#         # Print the current advertiser Id
#         print(f"Processing advertiser Id: {AID}")

#         # Make a single API request to get all targeting types
#         request = service.advertisers().listAssignedTargetingOptions(
#             advertiserId=AID
#         )
#         response = request.execute()

#         # Initialize a dictionary to hold the data for this advertiser
#         new_row = {'advertiserId': AID}

#         # Process each targeting type in the response
#         if 'assignedTargetingOptions' in response:
#             advertiserListResponse = pd.json_normalize(response['assignedTargetingOptions'])

#             for targeting_type in targeting_types_list:
#                 if targeting_type in advertiserListResponse['targetingType'].values:
#                     # Process the targeting details for the current targeting type
#                     targeting_details = process_targeting_details(advertiserListResponse, targeting_type)
#                     new_row[f'{targeting_type}'] = targeting_details
#                 else:
#                     new_row[f'{targeting_type}'] = None  # If the targeting type is not in the response, set to None

#         # Append the completed row for this advertiser to the DataFrame
#         dataFrame_Advertiser_AssignedTargetingOptions = pd.concat([dataFrame_Advertiser_AssignedTargetingOptions, pd.DataFrame([new_row])], ignore_index=True)

#     except Exception as e:
#         print(f"An error occurred for AID {AID}: {e}")

# # Output the final DataFrame
# print(dataFrame_Advertiser_AssignedTargetingOptions)


In [36]:
# # Initialize the DataFrame with dynamic columns based on the specified targeting types
# dataFrame_Advertiser_AssignedTargetingOptions = pd.DataFrame({'advertiserId': []})

# # List of targeting types to use
# targeting_types_list = [
#     'TARGETING_TYPE_CHANNEL',
#     'TARGETING_TYPE_DIGITAL_CONTENT_LABEL_EXCLUSION',
#     'TARGETING_TYPE_OMID',
#     'TARGETING_TYPE_SENSITIVE_CATEGORY_EXCLUSION',
#     'TARGETING_TYPE_YOUTUBE_VIDEO',
#     'TARGETING_TYPE_YOUTUBE_CHANNEL',
#     'TARGETING_TYPE_KEYWORD'
# ]

# # Dynamically add columns based on the selected targeting types in targeting_types_list
# for targeting_type in targeting_types_list:
#     dataFrame_Advertiser_AssignedTargetingOptions[f'{targeting_type}'] = []

# # Use the provided `targeting_details_mapping` to fetch columns and sort index
# def process_targeting_details(advertiserListResponse, targeting_type):
#     if targeting_type not in targeting_details_mapping:
#         return []

#     # Extract columns and sorting index for the current targeting type
#     targeting_fields = targeting_details_mapping[targeting_type]
#     sorting_column_index = targeting_fields[-2] if isinstance(targeting_fields[-2], int) else 0

#     # Get targeting details using the predefined function
#     targeting_details = get_targeting_details(
#         advertiserListResponse,
#         targeting_type,
#         *targeting_fields[:-2]  # Unpack the fields to be passed to the function
#     )

#     # Flatten and sort targeting details
#     flattened_details = list(zip(*targeting_details))
#     sorted_details = sort_by_column(flattened_details, sorting_column_index)

#     # Format as list of lists for each targeting option (if required by output format)
#     formatted_details = [
#         [entry[i] for i in range(len(entry))] for entry in sorted_details
#     ]

#     return formatted_details

# # Loop through each advertiser ID and retrieve their assigned targeting options
# for AID in advertiserIdList:
#     try:
#         print(f"Processing advertiser Id: {AID}")

#         # Single API request for all targeting types
#         request = service.advertisers().listAssignedTargetingOptions(advertiserId=AID)
#         response = request.execute()

#         # Initialize a dictionary for this advertiser's data
#         new_row = {'advertiserId': AID}

#         # Process each targeting type in the response
#         if 'assignedTargetingOptions' in response:
#             advertiserListResponse = pd.json_normalize(response['assignedTargetingOptions'])

#             for targeting_type in targeting_types_list:
#                 if targeting_type in advertiserListResponse['targetingType'].values:
#                     # Process and format the targeting details
#                     targeting_details = process_targeting_details(advertiserListResponse, targeting_type)
#                     new_row[f'{targeting_type}'] = targeting_details
#                 else:
#                     new_row[f'{targeting_type}'] = None  # Set to None if not present

#         # Append the advertiser data to the DataFrame
#         dataFrame_Advertiser_AssignedTargetingOptions = pd.concat(
#             [dataFrame_Advertiser_AssignedTargetingOptions, pd.DataFrame([new_row])],
#             ignore_index=True
#         )

#     except Exception as e:
#         print(f"An error occurred for AID {AID}: {e}")

# # Output the final DataFrame
# print(dataFrame_Advertiser_AssignedTargetingOptions)


In [37]:
# ## Testing with new process_targeting_details function

# # Initialize the DataFrame with dynamic columns based on the specified targeting types
# dataFrame_Advertiser_AssignedTargetingOptions = pd.DataFrame({
#     'advertiserId': [],
# })

# # List of targeting types to use
# targeting_types_list = [
#     'TARGETING_TYPE_CHANNEL',
#     'TARGETING_TYPE_DIGITAL_CONTENT_LABEL_EXCLUSION',
#     'TARGETING_TYPE_OMID',
#     'TARGETING_TYPE_SENSITIVE_CATEGORY_EXCLUSION',
#     'TARGETING_TYPE_YOUTUBE_VIDEO',
#     'TARGETING_TYPE_YOUTUBE_CHANNEL',
#     'TARGETING_TYPE_KEYWORD'
# ]

# # Dynamically add columns based on the selected targeting types in targeting_types_list
# for targeting_type in targeting_types_list:
#     if targeting_type in targeting_details_mapping:
#         dataFrame_Advertiser_AssignedTargetingOptions[f'{targeting_type}'] = []


# # Loop through each advertiser Id
# for AID in advertiserIdList:
#     try:
#         print(f"Processing advertiser Id: {AID}")
#         request = service.advertisers().listAssignedTargetingOptions(advertiserId=AID)
#         response = request.execute()

#         new_row = {'advertiserId': AID}

#         if 'assignedTargetingOptions' in response:
#             advertiserListResponse = pd.json_normalize(response['assignedTargetingOptions'])
#             for targeting_type in targeting_types_list:
#                 if targeting_type in advertiserListResponse['targetingType'].values:
#                     # Process targeting details for 'standard' level responses
#                     targeting_details = process_targeting_details(advertiserListResponse, targeting_type, level='standard')
#                     new_row[f'{targeting_type}'] = targeting_details
#                 else:
#                     new_row[f'{targeting_type}'] = None

#         dataFrame_Advertiser_AssignedTargetingOptions = pd.concat([dataFrame_Advertiser_AssignedTargetingOptions, pd.DataFrame([new_row])], ignore_index=True)

#     except Exception as e:
#         print(f"An error occurred for AID {AID}: {e}")

# # Output the final DataFrame
# print(dataFrame_Advertiser_AssignedTargetingOptions)


In [38]:
# # Initialize the DataFrame with dynamic columns based on the specified targeting types
# dataFrame_Advertiser_AssignedTargetingOptions = pd.DataFrame({
#     'advertiserId': [],
# })

# # List of targeting types to use
# targeting_types_list = [
#     'TARGETING_TYPE_CHANNEL',
#     'TARGETING_TYPE_DIGITAL_CONTENT_LABEL_EXCLUSION',
#     'TARGETING_TYPE_OMID',
#     'TARGETING_TYPE_SENSITIVE_CATEGORY_EXCLUSION',
#     'TARGETING_TYPE_YOUTUBE_VIDEO',
#     'TARGETING_TYPE_YOUTUBE_CHANNEL',
#     'TARGETING_TYPE_KEYWORD'
# ]

# # Dynamically add columns based on the selected targeting types in targeting_types_list
# for targeting_type in targeting_types_list:
#     if targeting_type in targeting_details_mapping:
#         dataFrame_Advertiser_AssignedTargetingOptions[f'{targeting_type}'] = []

# # Loop through each advertiser Id
# for AID in advertiserIdList:
#     try:
#         print(f"Processing advertiser Id: {AID}")
#         request = service.advertisers().listAssignedTargetingOptions(advertiserId=AID)
#         response = request.execute()

#         new_row = {'advertiserId': AID}

#         if 'assignedTargetingOptions' in response:
#             advertiserListResponse = pd.json_normalize(response['assignedTargetingOptions'])
#             for targeting_type in targeting_types_list:
#                 if targeting_type in advertiserListResponse['targetingType'].values:
#                     targeting_details = process_targeting_details(advertiserListResponse, targeting_type)
#                     new_row[f'{targeting_type}'] = targeting_details
#                 else:
#                     new_row[f'{targeting_type}'] = None

#         dataFrame_Advertiser_AssignedTargetingOptions = pd.concat([dataFrame_Advertiser_AssignedTargetingOptions, pd.DataFrame([new_row])], ignore_index=True)

#     except Exception as e:
#         print(f"An error occurred for AID {AID}: {e}")


# # Output the final DataFrame
# print(dataFrame_Advertiser_AssignedTargetingOptions)


In [39]:
# ## Batch Testing API calls
# import concurrent.futures
# import pandas as pd
# from googleapiclient.errors import HttpError

# # Initialize an empty list to hold all rows before converting to a DataFrame
# all_rows = []

# # Function to process each advertiser and make API call
# def process_advertiser(AID, service):
#     try:
#         # Print the current advertiser Id
#         print(f"Processing advertiser Id: {AID}")

#         # Make the API request to get all targeting types for the advertiser
#         request = service.advertisers().listAssignedTargetingOptions(advertiserId=AID)
#         response = request.execute()

#         # Initialize a dictionary to hold the data for this advertiser
#         new_row = {'advertiserId': AID}

#         # Process each targeting type in the response
#         if 'assignedTargetingOptions' in response:
#             advertiserListResponse = pd.json_normalize(response['assignedTargetingOptions'])

#             for targeting_type in targeting_types_list:
#                 if targeting_type in advertiserListResponse['targetingType'].values:
#                     # Process the targeting details for the current targeting type
#                     targeting_details = process_targeting_details(advertiserListResponse, targeting_type)
#                     new_row[f'{targeting_type}'] = targeting_details
#                 else:
#                     new_row[f'{targeting_type}'] = None  # If the targeting type is not in the response, set to None

#         return new_row

#     except HttpError as e:
#         print(f"An error occurred for AID {AID}: {e}")
#         return None

# # Function to handle parallel processing of advertisers
# def process_advertisers_in_parallel(advertiserIdList, service):
#     with concurrent.futures.ThreadPoolExecutor() as executor:
#         futures = {executor.submit(process_advertiser, AID, service): AID for AID in advertiserIdList}
#         for future in concurrent.futures.as_completed(futures):
#             row = future.result()
#             if row:
#                 all_rows.append(row)

# # Call the function to process all advertisers in parallel
# process_advertisers_in_parallel(advertiserIdList, service)

# # After all advertisers have been processed, convert the collected rows into a DataFrame
# dataFrame_Advertiser_AssignedTargetingOptions = pd.DataFrame(all_rows)

# # Output the final DataFrame
# print(dataFrame_Advertiser_AssignedTargetingOptions)


## Campaign

### Live

In [40]:
# V3 With adapted output format

# Initialize the DataFrame with dynamic columns based on the specified targeting types
dataFrame_Campaign_AssignedTargetingOptions = pd.DataFrame({
    'advertiserId': [],
    'campaignId': []  # Include campaignId in the DataFrame
})

# List of targeting types to use
targeting_types_list = [
    'TARGETING_TYPE_AGE_RANGE',
    'TARGETING_TYPE_AUTHORIZED_SELLER_STATUS',
    'TARGETING_TYPE_CONTENT_INSTREAM_POSITION',
    'TARGETING_TYPE_CONTENT_OUTSTREAM_POSITION',
    'TARGETING_TYPE_DIGITAL_CONTENT_LABEL_EXCLUSION',
    'TARGETING_TYPE_ENVIRONMENT',
    'TARGETING_TYPE_EXCHANGE',
    'TARGETING_TYPE_GENDER',
    'TARGETING_TYPE_GEO_REGION',
    'TARGETING_TYPE_HOUSEHOLD_INCOME',
    'TARGETING_TYPE_INVENTORY_SOURCE',
    'TARGETING_TYPE_INVENTORY_SOURCE_GROUP',
    'TARGETING_TYPE_LANGUAGE',
    'TARGETING_TYPE_ON_SCREEN_POSITION',
    'TARGETING_TYPE_PARENTAL_STATUS',
    'TARGETING_TYPE_SENSITIVE_CATEGORY_EXCLUSION',
    'TARGETING_TYPE_SUB_EXCHANGE',
    'TARGETING_TYPE_THIRD_PARTY_VERIFIER',
    'TARGETING_TYPE_VIEWABILITY'
]

# Dynamically add columns based on the selected targeting types in targeting_types_list
for targeting_type in targeting_types_list:
    if targeting_type in targeting_details_mapping:
        dataFrame_Campaign_AssignedTargetingOptions[f'{targeting_type}'] = []

# Function to extract and process targeting details
def process_targeting_details(campaignListResponse, targeting_type):
    if targeting_type not in targeting_details_mapping:
        return []

    # Extract columns and sorting index for the current targeting type
    targeting_fields = targeting_details_mapping[targeting_type]
    sorting_column_index = targeting_fields[-2] if isinstance(targeting_fields[-2], int) else 0

    # Get targeting details using the predefined function
    targeting_details = get_targeting_details(
        campaignListResponse,
        targeting_type,
        *targeting_fields[:-2]  # Unpack the fields to be passed to the function
    )

    # Flatten and sort targeting details
    flattened_details = list(zip(*targeting_details))
    sorted_details = sort_by_column(flattened_details, sorting_column_index)

    # Format as list of lists for each targeting option (if required by output format)
    formatted_details = [
        [entry[i] for i in range(len(entry))] for entry in sorted_details
    ]

    return formatted_details

# Deduplicate campaignIds when grouping by advertiserId
grouped_campaigns = id_Name_table.groupby('Advertiser ID')['Campaign ID'].apply(lambda x: list(set(x.dropna()))).to_dict()

# Loop through each advertiserId and its corresponding campaignIds
for AID, campaignIdList in grouped_campaigns.items():
    try:
        # Print the current advertiser Id
        print(f"Processing advertiser Id: {AID}")

        # Loop through each campaign ID
        for CID in campaignIdList:
            # Check if CID is not NaN before processing
            if pd.isna(CID):
                print(f"Skipping NaN campaignId for advertiser Id: {AID}")
                continue

            try:
                # Print the current campaign Id
                print(f"Processing campaign Id: {CID} for advertiser Id: {AID}")

                # Make a single API request to get all targeting types for this campaign
                request = service.advertisers().campaigns().listAssignedTargetingOptions(
                    advertiserId=AID,
                    campaignId=CID
                )
                response = request.execute()

                # Initialize a dictionary to hold the data for this advertiser and campaign
                new_row = {'advertiserId': AID, 'campaignId': CID}

                # Process each targeting type in the response
                if 'assignedTargetingOptions' in response:
                    campaignListResponse = pd.json_normalize(response['assignedTargetingOptions'])

                    for targeting_type in targeting_types_list:
                        if targeting_type in campaignListResponse['targetingType'].values:
                            # Process the targeting details for the current targeting type
                            targeting_details = process_targeting_details(campaignListResponse, targeting_type)
                            new_row[f'{targeting_type}'] = targeting_details
                        else:
                            new_row[f'{targeting_type}'] = None  # If the targeting type is not in the response, set to None

                # Append the completed row for this advertiser and campaign to the DataFrame
                dataFrame_Campaign_AssignedTargetingOptions = pd.concat(
                    [dataFrame_Campaign_AssignedTargetingOptions, pd.DataFrame([new_row])],
                    ignore_index=True
                )

            except Exception as e:
                print(f"An error occurred for CID {CID} in advertiser ID {AID}: {e}")

    except Exception as e:
        print(f"An error occurred for AID {AID}: {e}")

# # Output the final DataFrame
# print(dataFrame_Campaign_AssignedTargetingOptions)

Processing advertiser Id: 1053287434
Processing campaign Id: 52645775 for advertiser Id: 1053287434
Processing campaign Id: 52950322 for advertiser Id: 1053287434
Processing campaign Id: 52907803 for advertiser Id: 1053287434
Processing campaign Id: 52904521 for advertiser Id: 1053287434
Processing advertiser Id: 1074131012
Processing campaign Id: 52725665 for advertiser Id: 1074131012
Processing campaign Id: 52725701 for advertiser Id: 1074131012
Processing campaign Id: 54437222 for advertiser Id: 1074131012
Processing campaign Id: 54416977 for advertiser Id: 1074131012
Processing advertiser Id: 1074902854
Processing campaign Id: 52882126 for advertiser Id: 1074902854
Processing campaign Id: 52810390 for advertiser Id: 1074902854
Processing campaign Id: 53878690 for advertiser Id: 1074902854
Processing campaign Id: 52926746 for advertiser Id: 1074902854
Processing advertiser Id: 1083545221
Processing campaign Id: 52913253 for advertiser Id: 1083545221
Processing advertiser Id: 1087573

### Testing

In [41]:
# ## Previous live, now in testing

# ## Initialize the DataFrame with dynamic columns based on the specified targeting types
# dataFrame_Campaign_AssignedTargetingOptions = pd.DataFrame({
#     'advertiserId': [],
#     'campaignId': []  # Include campaignId in the DataFrame
# })

# # List of targeting types to use
# targeting_types_list = [
#     'TARGETING_TYPE_AGE_RANGE',
#     'TARGETING_TYPE_AUTHORIZED_SELLER_STATUS',
#     'TARGETING_TYPE_CONTENT_INSTREAM_POSITION',
#     'TARGETING_TYPE_CONTENT_OUTSTREAM_POSITION',
#     'TARGETING_TYPE_DIGITAL_CONTENT_LABEL_EXCLUSION',
#     'TARGETING_TYPE_ENVIRONMENT',
#     'TARGETING_TYPE_EXCHANGE',
#     'TARGETING_TYPE_GENDER',
#     'TARGETING_TYPE_GEO_REGION',
#     'TARGETING_TYPE_HOUSEHOLD_INCOME',
#     'TARGETING_TYPE_INVENTORY_SOURCE',
#     'TARGETING_TYPE_INVENTORY_SOURCE_GROUP',
#     'TARGETING_TYPE_LANGUAGE',
#     'TARGETING_TYPE_ON_SCREEN_POSITION',
#     'TARGETING_TYPE_PARENTAL_STATUS',
#     'TARGETING_TYPE_SENSITIVE_CATEGORY_EXCLUSION',
#     'TARGETING_TYPE_SUB_EXCHANGE',
#     'TARGETING_TYPE_THIRD_PARTY_VERIFIER',
#     'TARGETING_TYPE_VIEWABILITY'
# ]

# # Dynamically add columns based on the selected targeting types in targeting_types_list
# for targeting_type in targeting_types_list:
#     if targeting_type in targeting_details_mapping:
#         dataFrame_Campaign_AssignedTargetingOptions[f'{targeting_type}'] = []

# # Function to extract and process targeting details
# def process_targeting_details(campaignListResponse, targeting_type):
#     targeting_fields = targeting_details_mapping[targeting_type]
#     targeting_details = get_targeting_details(
#         campaignListResponse,
#         targeting_type,
#         *targeting_fields[:-2]  # Unpack the fields to be passed to the function
#     )
#     sorting_column_index = targeting_fields[-2]
#     targeting_details = sort_by_column(targeting_details, sorting_column_index)
#     return targeting_details

# # Deduplicate campaignIds when grouping by advertiserId
# grouped_campaigns = id_Name_table.groupby('Advertiser ID')['Campaign ID'].apply(lambda x: list(set(x.dropna()))).to_dict()

# # Loop through each advertiserId and its corresponding campaignIds
# for AID, campaignIdList in grouped_campaigns.items():
#     try:
#         # Print the current advertiser Id
#         print(f"Processing advertiser Id: {AID}")

#         # Loop through each campaign ID
#         for CID in campaignIdList:
#             # Check if CID is not NaN before processing
#             if pd.isna(CID):
#                 print(f"Skipping NaN campaignId for advertiser Id: {AID}")
#                 continue

#             try:
#                 # Print the current campaign Id
#                 print(f"Processing campaign Id: {CID} for advertiser Id: {AID}")

#                 # Make a single API request to get all targeting types for this campaign
#                 request = service.advertisers().campaigns().listAssignedTargetingOptions(
#                     advertiserId=AID,
#                     campaignId=CID
#                 )
#                 response = request.execute()

#                 # Initialize a dictionary to hold the data for this advertiser and campaign
#                 new_row = {'advertiserId': AID, 'campaignId': CID}

#                 # Process each targeting type in the response
#                 if 'assignedTargetingOptions' in response:
#                     campaignListResponse = pd.json_normalize(response['assignedTargetingOptions'])

#                     for targeting_type in targeting_types_list:
#                         if targeting_type in campaignListResponse['targetingType'].values:
#                             # Process the targeting details for the current targeting type
#                             targeting_details = process_targeting_details(campaignListResponse, targeting_type)
#                             new_row[f'{targeting_type}'] = targeting_details
#                         else:
#                             new_row[f'{targeting_type}'] = None  # If the targeting type is not in the response, set to None

#                 # Append the completed row for this advertiser and campaign to the DataFrame
#                 dataFrame_Campaign_AssignedTargetingOptions = pd.concat([dataFrame_Campaign_AssignedTargetingOptions, pd.DataFrame([new_row])], ignore_index=True)

#             except Exception as e:
#                 print(f"An error occurred for CID {CID} in advertiser ID {AID}: {e}")

#     except Exception as e:
#         print(f"An error occurred for AID {AID}: {e}")

# # Output the final DataFrame
# print(dataFrame_Campaign_AssignedTargetingOptions)

In [42]:
# ## Initialize the DataFrame with dynamic columns based on the specified targeting types
# dataFrame_Campaign_AssignedTargetingOptions = pd.DataFrame({
#     'advertiserId': [],
#     'campaignId': []  # Include campaignId in the DataFrame
# })

# # List of targeting types to use
# targeting_types_list = [
#     'TARGETING_TYPE_AGE_RANGE',
#     'TARGETING_TYPE_AUTHORIZED_SELLER_STATUS',
#     'TARGETING_TYPE_CONTENT_INSTREAM_POSITION',
#     'TARGETING_TYPE_CONTENT_OUTSTREAM_POSITION',
#     'TARGETING_TYPE_DIGITAL_CONTENT_LABEL_EXCLUSION',
#     'TARGETING_TYPE_ENVIRONMENT',
#     'TARGETING_TYPE_EXCHANGE',
#     'TARGETING_TYPE_GENDER',
#     'TARGETING_TYPE_GEO_REGION',
#     'TARGETING_TYPE_HOUSEHOLD_INCOME',
#     'TARGETING_TYPE_INVENTORY_SOURCE',
#     'TARGETING_TYPE_INVENTORY_SOURCE_GROUP',
#     'TARGETING_TYPE_LANGUAGE',
#     'TARGETING_TYPE_ON_SCREEN_POSITION',
#     'TARGETING_TYPE_PARENTAL_STATUS',
#     'TARGETING_TYPE_SENSITIVE_CATEGORY_EXCLUSION',
#     'TARGETING_TYPE_SUB_EXCHANGE',
#     'TARGETING_TYPE_THIRD_PARTY_VERIFIER',
#     'TARGETING_TYPE_VIEWABILITY'
# ]

# # Dynamically add columns based on the selected targeting types in targeting_types_list
# for targeting_type in targeting_types_list:
#     if targeting_type in targeting_details_mapping:
#         dataFrame_Campaign_AssignedTargetingOptions[f'{targeting_type}'] = []

# # Deduplicate campaignIds when grouping by advertiserId
# grouped_campaigns = id_Name_table.groupby('Advertiser ID')['Campaign ID'].apply(lambda x: list(set(x.dropna()))).to_dict()

# # Loop through each advertiserId and its corresponding campaignIds
# for AID, campaignIdList in grouped_campaigns.items():
#     try:
#         print(f"Processing advertiser Id: {AID}")
#         for CID in campaignIdList:
#             if pd.isna(CID):
#                 print(f"Skipping NaN campaignId for advertiser Id: {AID}")
#                 continue

#             try:
#                 print(f"Processing campaign Id: {CID} for advertiser Id: {AID}")
#                 request = service.advertisers().campaigns().listAssignedTargetingOptions(advertiserId=AID, campaignId=CID)
#                 response = request.execute()

#                 new_row = {'advertiserId': AID, 'campaignId': CID}

#                 if 'assignedTargetingOptions' in response:
#                     campaignListResponse = pd.json_normalize(response['assignedTargetingOptions'])
#                     for targeting_type in targeting_types_list:
#                         if targeting_type in campaignListResponse['targetingType'].values:
#                             targeting_details = process_targeting_details(campaignListResponse, targeting_type)
#                             new_row[f'{targeting_type}'] = targeting_details
#                         else:
#                             new_row[f'{targeting_type}'] = None

#                 dataFrame_Campaign_AssignedTargetingOptions = pd.concat([dataFrame_Campaign_AssignedTargetingOptions, pd.DataFrame([new_row])], ignore_index=True)

#             except Exception as e:
#                 print(f"An error occurred for CID {CID} in advertiser ID {AID}: {e}")

#     except Exception as e:
#         print(f"An error occurred for AID {AID}: {e}")

# # Output the final DataFrame
# print(dataFrame_Campaign_AssignedTargetingOptions)

## Insertion Order

### Live

In [43]:
## V3 With updated formatting on output

# Initialize the DataFrame with dynamic columns based on the specified targeting types
dataFrame_InsertionOrder_AssignedTargetingOptions = pd.DataFrame({
    'advertiserId': [],
    'insertionOrderId': []  # Include insertionOrderId in the DataFrame
})

# List of targeting types to use
targeting_types_list = [
    'TARGETING_TYPE_AGE_RANGE',
    'TARGETING_TYPE_APP',
    'TARGETING_TYPE_APP_CATEGORY',
    'TARGETING_TYPE_AUDIENCE_GROUP',
    'TARGETING_TYPE_AUDIO_CONTENT_TYPE',
    'TARGETING_TYPE_AUTHORIZED_SELLER_STATUS',
    'TARGETING_TYPE_BROWSER',
    'TARGETING_TYPE_BUSINESS_CHAIN',
    'TARGETING_TYPE_CARRIER_AND_ISP',
    'TARGETING_TYPE_CATEGORY',
    'TARGETING_TYPE_CHANNEL',
    'TARGETING_TYPE_CONTENT_DURATION',
    'TARGETING_TYPE_CONTENT_GENRE',
    'TARGETING_TYPE_CONTENT_INSTREAM_POSITION',
    'TARGETING_TYPE_CONTENT_OUTSTREAM_POSITION',
    'TARGETING_TYPE_CONTENT_STREAM_TYPE',
    'TARGETING_TYPE_DAY_AND_TIME',
    'TARGETING_TYPE_DEVICE_MAKE_MODEL',
    'TARGETING_TYPE_DEVICE_TYPE',
    'TARGETING_TYPE_DIGITAL_CONTENT_LABEL_EXCLUSION',
    'TARGETING_TYPE_ENVIRONMENT',
    'TARGETING_TYPE_EXCHANGE',
    'TARGETING_TYPE_GENDER',
    'TARGETING_TYPE_GEO_REGION',
    'TARGETING_TYPE_HOUSEHOLD_INCOME',
    'TARGETING_TYPE_INVENTORY_SOURCE',
    'TARGETING_TYPE_INVENTORY_SOURCE_GROUP',
    'TARGETING_TYPE_KEYWORD',
    'TARGETING_TYPE_LANGUAGE',
    'TARGETING_TYPE_NATIVE_CONTENT_POSITION',
    'TARGETING_TYPE_NEGATIVE_KEYWORD_LIST',
    'TARGETING_TYPE_OMID',
    'TARGETING_TYPE_ON_SCREEN_POSITION',
    'TARGETING_TYPE_OPERATING_SYSTEM',
    'TARGETING_TYPE_PARENTAL_STATUS',
    'TARGETING_TYPE_POI',
    'TARGETING_TYPE_PROXIMITY_LOCATION_LIST',
    'TARGETING_TYPE_REGIONAL_LOCATION_LIST',
    'TARGETING_TYPE_SENSITIVE_CATEGORY_EXCLUSION',
    'TARGETING_TYPE_SUB_EXCHANGE',
    'TARGETING_TYPE_THIRD_PARTY_VERIFIER',
    'TARGETING_TYPE_URL',
    'TARGETING_TYPE_USER_REWARDED_CONTENT',
    'TARGETING_TYPE_VIDEO_PLAYER_SIZE',
    'TARGETING_TYPE_VIEWABILITY'
]

# Dynamically add columns based on the selected targeting types in targeting_types_list
for targeting_type in targeting_types_list:
    if targeting_type in targeting_details_mapping:
        dataFrame_InsertionOrder_AssignedTargetingOptions[f'{targeting_type}'] = []

# Function to extract and process targeting details
def process_targeting_details(insertionOrderListResponse, targeting_type):
    if targeting_type not in targeting_details_mapping:
        return []

    # Extract columns and sorting index for the current targeting type
    targeting_fields = targeting_details_mapping[targeting_type]
    sorting_column_index = targeting_fields[-2] if isinstance(targeting_fields[-2], int) else 0

    # Get targeting details using the predefined function
    targeting_details = get_targeting_details(
        insertionOrderListResponse,
        targeting_type,
        *targeting_fields[:-2]  # Unpack the fields to be passed to the function
    )

    # Flatten and sort targeting details
    flattened_details = list(zip(*targeting_details))
    sorted_details = sort_by_column(flattened_details, sorting_column_index)

    # Format as list of lists for each targeting option (if required by output format)
    formatted_details = [
        [entry[i] for i in range(len(entry))] for entry in sorted_details
    ]

    return formatted_details

# Deduplicate insertionOrderId when grouping by advertiserId
grouped_insertion_orders = id_Name_table.groupby('Advertiser ID')['Insertion Order ID'].apply(lambda x: list(set(x.dropna()))).to_dict()

# Loop through each advertiserId and its corresponding insertionOrderIds
for AID, insertionOrderIdList in grouped_insertion_orders.items():
    try:
        # Print the current advertiser Id
        print(f"Processing advertiser Id: {AID}")

        # Loop through each insertionOrder ID
        for IOID in insertionOrderIdList:
            # Check if IOID is not NaN before processing
            if pd.isna(IOID):
                print(f"Skipping NaN insertionOrderId for advertiser Id: {AID}")
                continue

            try:
                # Print the current insertionOrder Id
                print(f"Processing insertionOrder Id: {IOID} for advertiser Id: {AID}")

                # Make a single API request to get all targeting types for this insertionOrder
                request = service.advertisers().insertionOrders().listAssignedTargetingOptions(
                    advertiserId=AID,
                    insertionOrderId=IOID
                )
                response = request.execute()

                # Initialize a dictionary to hold the data for this advertiser and insertionOrder
                new_row = {'advertiserId': AID, 'insertionOrderId': IOID}

                # Process each targeting type in the response
                if 'assignedTargetingOptions' in response:
                    insertionOrderListResponse = pd.json_normalize(response['assignedTargetingOptions'])

                    for targeting_type in targeting_types_list:
                        if targeting_type in insertionOrderListResponse['targetingType'].values:
                            # Process the targeting details for the current targeting type
                            targeting_details = process_targeting_details(insertionOrderListResponse, targeting_type)
                            new_row[f'{targeting_type}'] = targeting_details
                        else:
                            new_row[f'{targeting_type}'] = None  # If the targeting type is not in the response, set to None

                # Append the completed row for this advertiser and insertionOrder to the DataFrame
                dataFrame_InsertionOrder_AssignedTargetingOptions = pd.concat(
                    [dataFrame_InsertionOrder_AssignedTargetingOptions, pd.DataFrame([new_row])],
                    ignore_index=True
                )

            except Exception as e:
                print(f"An error occurred for IOID {IOID} in advertiser ID {AID}: {e}")

    except Exception as e:
        print(f"An error occurred for AID {AID}: {e}")

# # Output the final DataFrame
# print(dataFrame_InsertionOrder_AssignedTargetingOptions)


Processing advertiser Id: 1053287434
Processing insertionOrder Id: 1009818371 for advertiser Id: 1053287434
Processing insertionOrder Id: 1008848202 for advertiser Id: 1053287434
Processing insertionOrder Id: 1010293480 for advertiser Id: 1053287434
Processing insertionOrder Id: 1010044009 for advertiser Id: 1053287434
Processing advertiser Id: 1074131012
Processing insertionOrder Id: 1009108751 for advertiser Id: 1074131012
Processing insertionOrder Id: 1016669138 for advertiser Id: 1074131012
Processing insertionOrder Id: 1009150950 for advertiser Id: 1074131012
Processing insertionOrder Id: 1016964025 for advertiser Id: 1074131012
Processing advertiser Id: 1074902854
Processing insertionOrder Id: 1010130940 for advertiser Id: 1074902854
Processing insertionOrder Id: 1009960984 for advertiser Id: 1074902854
Processing insertionOrder Id: 1009520877 for advertiser Id: 1074902854
Processing insertionOrder Id: 1014549101 for advertiser Id: 1074902854
Processing advertiser Id: 1083545221


### Testing

In [44]:
# ## Previous live, now in testing

# # Initialize the DataFrame with dynamic columns based on the specified targeting types
# dataFrame_InsertionOrder_AssignedTargetingOptions = pd.DataFrame({
#     'advertiserId': [],
#     'insertionOrderId': []  # Include insertionOrderId in the DataFrame
# })

# # List of targeting types to use
# targeting_types_list = [
#     'TARGETING_TYPE_AGE_RANGE',
#     'TARGETING_TYPE_APP',
#     'TARGETING_TYPE_APP_CATEGORY',
#     'TARGETING_TYPE_AUDIENCE_GROUP',
#     'TARGETING_TYPE_AUDIO_CONTENT_TYPE',
#     'TARGETING_TYPE_AUTHORIZED_SELLER_STATUS',
#     'TARGETING_TYPE_BROWSER',
#     'TARGETING_TYPE_BUSINESS_CHAIN',
#     'TARGETING_TYPE_CARRIER_AND_ISP',
#     'TARGETING_TYPE_CATEGORY',
#     'TARGETING_TYPE_CHANNEL',
#     'TARGETING_TYPE_CONTENT_DURATION',
#     'TARGETING_TYPE_CONTENT_GENRE',
#     'TARGETING_TYPE_CONTENT_INSTREAM_POSITION',
#     'TARGETING_TYPE_CONTENT_OUTSTREAM_POSITION',
#     'TARGETING_TYPE_CONTENT_STREAM_TYPE',
#     'TARGETING_TYPE_DAY_AND_TIME',
#     'TARGETING_TYPE_DEVICE_MAKE_MODEL',
#     'TARGETING_TYPE_DEVICE_TYPE',
#     'TARGETING_TYPE_DIGITAL_CONTENT_LABEL_EXCLUSION',
#     'TARGETING_TYPE_ENVIRONMENT',
#     'TARGETING_TYPE_EXCHANGE',
#     'TARGETING_TYPE_GENDER',
#     'TARGETING_TYPE_GEO_REGION',
#     'TARGETING_TYPE_HOUSEHOLD_INCOME',
#     'TARGETING_TYPE_INVENTORY_SOURCE',
#     'TARGETING_TYPE_INVENTORY_SOURCE_GROUP',
#     'TARGETING_TYPE_KEYWORD',
#     'TARGETING_TYPE_LANGUAGE',
#     'TARGETING_TYPE_NATIVE_CONTENT_POSITION',
#     'TARGETING_TYPE_NEGATIVE_KEYWORD_LIST',
#     'TARGETING_TYPE_OMID',
#     'TARGETING_TYPE_ON_SCREEN_POSITION',
#     'TARGETING_TYPE_OPERATING_SYSTEM',
#     'TARGETING_TYPE_PARENTAL_STATUS',
#     'TARGETING_TYPE_POI',
#     'TARGETING_TYPE_PROXIMITY_LOCATION_LIST',
#     'TARGETING_TYPE_REGIONAL_LOCATION_LIST',
#     'TARGETING_TYPE_SENSITIVE_CATEGORY_EXCLUSION',
#     'TARGETING_TYPE_SUB_EXCHANGE',
#     'TARGETING_TYPE_THIRD_PARTY_VERIFIER',
#     'TARGETING_TYPE_URL',
#     'TARGETING_TYPE_USER_REWARDED_CONTENT',
#     'TARGETING_TYPE_VIDEO_PLAYER_SIZE',
#     'TARGETING_TYPE_VIEWABILITY'
# ]

# # Dynamically add columns based on the selected targeting types in targeting_types_list
# for targeting_type in targeting_types_list:
#     if targeting_type in targeting_details_mapping:
#         dataFrame_InsertionOrder_AssignedTargetingOptions[f'{targeting_type}'] = []

# # Function to extract and process targeting details
# def process_targeting_details(insertionOrderListResponse, targeting_type):
#     targeting_fields = targeting_details_mapping[targeting_type]
#     targeting_details = get_targeting_details(
#         insertionOrderListResponse,
#         targeting_type,
#         *targeting_fields[:-2]  # Unpack the fields to be passed to the function
#     )
#     sorting_column_index = targeting_fields[-2]
#     targeting_details = sort_by_column(targeting_details, sorting_column_index)
#     return targeting_details

# # Deduplicate insertionOrderId when grouping by advertiserId
# grouped_insertion_orders = id_Name_table.groupby('Advertiser ID')['Insertion Order ID'].apply(lambda x: list(set(x.dropna()))).to_dict()

# # Loop through each advertiserId and its corresponding insertionOrderIds
# for AID, insertionOrderIdList in grouped_insertion_orders.items():
#     try:
#         # Print the current advertiser Id
#         print(f"Processing advertiser Id: {AID}")

#         # Loop through each insertionOrder ID
#         for IOID in insertionOrderIdList:
#             # Check if IOID is not NaN before processing
#             if pd.isna(IOID):
#                 print(f"Skipping NaN insertionOrderId for advertiser Id: {AID}")
#                 continue

#             try:
#                 # Print the current insertionOrder Id
#                 print(f"Processing insertionOrder Id: {IOID} for advertiser Id: {AID}")

#                 # Make a single API request to get all targeting types for this insertionOrder
#                 request = service.advertisers().insertionOrders().listAssignedTargetingOptions(
#                     advertiserId=AID,
#                     insertionOrderId=IOID
#                 )
#                 response = request.execute()

#                 # Initialize a dictionary to hold the data for this advertiser and insertionOrder
#                 new_row = {'advertiserId': AID, 'insertionOrderId': IOID}

#                 # Process each targeting type in the response
#                 if 'assignedTargetingOptions' in response:
#                     insertionOrderListResponse = pd.json_normalize(response['assignedTargetingOptions'])

#                     for targeting_type in targeting_types_list:
#                         if targeting_type in insertionOrderListResponse['targetingType'].values:
#                             # Process the targeting details for the current targeting type
#                             targeting_details = process_targeting_details(insertionOrderListResponse, targeting_type)
#                             new_row[f'{targeting_type}'] = targeting_details
#                         else:
#                             new_row[f'{targeting_type}'] = None  # If the targeting type is not in the response, set to None

#                 # Append the completed row for this advertiser and insertionOrder to the DataFrame
#                 dataFrame_InsertionOrder_AssignedTargetingOptions = pd.concat([dataFrame_InsertionOrder_AssignedTargetingOptions, pd.DataFrame([new_row])], ignore_index=True)

#             except Exception as e:
#                 print(f"An error occurred for IOID {IOID} in advertiser ID {AID}: {e}")

#     except Exception as e:
#         print(f"An error occurred for AID {AID}: {e}")

# # Output the final DataFrame
# print(dataFrame_InsertionOrder_AssignedTargetingOptions)

In [45]:
# def process_targeting_details(filtered_response, targeting_type):
#     targeting_fields = targeting_details_mapping[targeting_type]
#     required_columns = ['assignedTargetingOption.' + field for field in targeting_fields[:-2]]

#     # Adjust for potential missing .negative columns and other expected columns
#     for col in required_columns:
#         if '.negative' in col:  # Check if the column has ".negative" in its name
#             if col not in filtered_response.columns:
#                 filtered_response = filtered_response.copy()
#                 filtered_response.loc[:, col] = False  # Add the .negative column with default False
#             else:
#                 filtered_response = filtered_response.copy()  # Ensure we're working with a copy
#                 filtered_response.loc[:, col] = filtered_response[col].apply(lambda x: False if pd.isna(x) or x == '' else x)
#         else:
#             if col not in filtered_response.columns:
#                 filtered_response = filtered_response.copy()
#                 filtered_response.loc[:, col] = np.nan  # Set missing columns to NaN using .loc

#     # Check if all required columns exist in the filtered response
#     if all(col in filtered_response.columns for col in required_columns):
#         # Select the relevant fields based on the mapping
#         filtered_details = filtered_response[required_columns]

#         # Sort the details according to the specified index in the mapping
#         sorting_column_index = targeting_fields[-2]  # Sorting is based on a field specified in the mapping
#         sorted_details = filtered_details.sort_values(by=[required_columns[sorting_column_index]])

#         # Convert the sorted DataFrame to a list of lists (each inner list corresponds to a row)
#         # Ensure the list is structured correctly and flattened, not nested lists
#         return sorted_details.apply(lambda x: x.tolist(), axis=1).tolist()

#     else:
#         missing_columns = [col for col in required_columns if col not in filtered_response.columns]
#         print(f"Missing columns for {targeting_type}: {missing_columns}")
#         return None


# # Initialize the DataFrame with dynamic columns based on the specified targeting types
# dataFrame_InsertionOrder_AssignedTargetingOptions = pd.DataFrame({
#     'advertiserId': [],
#     'insertionOrderId': []  # Include insertionOrderId in the DataFrame
# })

# # List of targeting types to use
# targeting_types_list = [
#     'TARGETING_TYPE_AGE_RANGE',
#     'TARGETING_TYPE_APP',
#     'TARGETING_TYPE_APP_CATEGORY',
#     'TARGETING_TYPE_AUDIENCE_GROUP',
#     'TARGETING_TYPE_AUDIO_CONTENT_TYPE',
#     'TARGETING_TYPE_AUTHORIZED_SELLER_STATUS',
#     'TARGETING_TYPE_BROWSER',
#     'TARGETING_TYPE_BUSINESS_CHAIN',
#     'TARGETING_TYPE_CARRIER_AND_ISP',
#     'TARGETING_TYPE_CATEGORY',
#     'TARGETING_TYPE_CHANNEL',
#     'TARGETING_TYPE_CONTENT_DURATION',
#     'TARGETING_TYPE_CONTENT_GENRE',
#     'TARGETING_TYPE_CONTENT_INSTREAM_POSITION',
#     'TARGETING_TYPE_CONTENT_OUTSTREAM_POSITION',
#     'TARGETING_TYPE_CONTENT_STREAM_TYPE',
#     'TARGETING_TYPE_DAY_AND_TIME',
#     'TARGETING_TYPE_DEVICE_MAKE_MODEL',
#     'TARGETING_TYPE_DEVICE_TYPE',
#     'TARGETING_TYPE_DIGITAL_CONTENT_LABEL_EXCLUSION',
#     'TARGETING_TYPE_ENVIRONMENT',
#     'TARGETING_TYPE_EXCHANGE',
#     'TARGETING_TYPE_GENDER',
#     'TARGETING_TYPE_GEO_REGION',
#     'TARGETING_TYPE_HOUSEHOLD_INCOME',
#     'TARGETING_TYPE_INVENTORY_SOURCE',
#     'TARGETING_TYPE_INVENTORY_SOURCE_GROUP',
#     'TARGETING_TYPE_KEYWORD',
#     'TARGETING_TYPE_LANGUAGE',
#     'TARGETING_TYPE_NATIVE_CONTENT_POSITION',
#     'TARGETING_TYPE_NEGATIVE_KEYWORD_LIST',
#     'TARGETING_TYPE_OMID',
#     'TARGETING_TYPE_ON_SCREEN_POSITION',
#     'TARGETING_TYPE_OPERATING_SYSTEM',
#     'TARGETING_TYPE_PARENTAL_STATUS',
#     'TARGETING_TYPE_POI',
#     'TARGETING_TYPE_PROXIMITY_LOCATION_LIST',
#     'TARGETING_TYPE_REGIONAL_LOCATION_LIST',
#     'TARGETING_TYPE_SENSITIVE_CATEGORY_EXCLUSION',
#     'TARGETING_TYPE_SUB_EXCHANGE',
#     'TARGETING_TYPE_THIRD_PARTY_VERIFIER',
#     'TARGETING_TYPE_URL',
#     'TARGETING_TYPE_USER_REWARDED_CONTENT',
#     'TARGETING_TYPE_VIDEO_PLAYER_SIZE',
#     'TARGETING_TYPE_VIEWABILITY'
# ]

# # Dynamically add columns based on the selected targeting types in targeting_types_list
# for targeting_type in targeting_types_list:
#     if targeting_type in targeting_details_mapping:
#         dataFrame_InsertionOrder_AssignedTargetingOptions[f'{targeting_type}'] = []



# # Deduplicate insertionOrderId when grouping by advertiserId
# grouped_insertion_orders = id_Name_table.groupby('Advertiser ID')['Insertion Order ID'].apply(lambda x: list(set(x.dropna()))).to_dict()

# # Loop through each advertiserId and its corresponding insertionOrderIds
# for AID, insertionOrderIdList in grouped_insertion_orders.items():
#     try:
#         print(f"Processing advertiser Id: {AID}")
#         for IOID in insertionOrderIdList:
#             if pd.isna(IOID):
#                 print(f"Skipping NaN insertionOrderId for advertiser Id: {AID}")
#                 continue

#             try:
#                 print(f"Processing insertionOrder Id: {IOID} for advertiser Id: {AID}")
#                 request = service.advertisers().insertionOrders().listAssignedTargetingOptions(
#                     advertiserId=AID,
#                     insertionOrderId=IOID
#                 )
#                 response = request.execute()

#                 new_row = {'advertiserId': AID, 'insertionOrderId': IOID}

#                 if 'assignedTargetingOptions' in response:
#                     insertionOrderListResponse = pd.json_normalize(response['assignedTargetingOptions'])
#                     for targeting_type in targeting_types_list:
#                         if targeting_type in insertionOrderListResponse['targetingType'].values:
#                             # Process the targeting details for the current targeting type
#                             targeting_details = process_targeting_details(insertionOrderListResponse, targeting_type)
#                             new_row[f'{targeting_type}'] = targeting_details
#                         else:
#                             new_row[f'{targeting_type}'] = None  # If the targeting type is not in the response, set to None

#                 dataFrame_InsertionOrder_AssignedTargetingOptions = pd.concat([dataFrame_InsertionOrder_AssignedTargetingOptions, pd.DataFrame([new_row])], ignore_index=True)

#             except Exception as e:
#                 print(f"An error occurred for IOID {IOID} in advertiser ID {AID}: {e}")

#     except Exception as e:
#         print(f"An error occurred for AID {AID}: {e}")

# # Output the final DataFrame
# print(dataFrame_InsertionOrder_AssignedTargetingOptions)


# # Output the final DataFrame
# print(dataFrame_InsertionOrder_AssignedTargetingOptions)

In [46]:
# # Initialize the DataFrame with dynamic columns based on the specified targeting types
# dataFrame_InsertionOrder_AssignedTargetingOptions = pd.DataFrame({
#     'advertiserId': [],
#     'insertionOrderId': []  # Include insertionOrderId in the DataFrame
# })

# # List of targeting types to use
# targeting_types_list = [
#     'TARGETING_TYPE_AGE_RANGE',
#     'TARGETING_TYPE_APP',
#     'TARGETING_TYPE_APP_CATEGORY',
#     'TARGETING_TYPE_AUDIENCE_GROUP',
#     'TARGETING_TYPE_AUDIO_CONTENT_TYPE',
#     'TARGETING_TYPE_AUTHORIZED_SELLER_STATUS',
#     'TARGETING_TYPE_BROWSER',
#     'TARGETING_TYPE_BUSINESS_CHAIN',
#     'TARGETING_TYPE_CARRIER_AND_ISP',
#     'TARGETING_TYPE_CATEGORY',
#     'TARGETING_TYPE_CHANNEL',
#     'TARGETING_TYPE_CONTENT_DURATION',
#     'TARGETING_TYPE_CONTENT_GENRE',
#     'TARGETING_TYPE_CONTENT_INSTREAM_POSITION',
#     'TARGETING_TYPE_CONTENT_OUTSTREAM_POSITION',
#     'TARGETING_TYPE_CONTENT_STREAM_TYPE',
#     'TARGETING_TYPE_DAY_AND_TIME',
#     'TARGETING_TYPE_DEVICE_MAKE_MODEL',
#     'TARGETING_TYPE_DEVICE_TYPE',
#     'TARGETING_TYPE_DIGITAL_CONTENT_LABEL_EXCLUSION',
#     'TARGETING_TYPE_ENVIRONMENT',
#     'TARGETING_TYPE_EXCHANGE',
#     'TARGETING_TYPE_GENDER',
#     'TARGETING_TYPE_GEO_REGION',
#     'TARGETING_TYPE_HOUSEHOLD_INCOME',
#     'TARGETING_TYPE_INVENTORY_SOURCE',
#     'TARGETING_TYPE_INVENTORY_SOURCE_GROUP',
#     'TARGETING_TYPE_KEYWORD',
#     'TARGETING_TYPE_LANGUAGE',
#     'TARGETING_TYPE_NATIVE_CONTENT_POSITION',
#     'TARGETING_TYPE_NEGATIVE_KEYWORD_LIST',
#     'TARGETING_TYPE_OMID',
#     'TARGETING_TYPE_ON_SCREEN_POSITION',
#     'TARGETING_TYPE_OPERATING_SYSTEM',
#     'TARGETING_TYPE_PARENTAL_STATUS',
#     'TARGETING_TYPE_POI',
#     'TARGETING_TYPE_PROXIMITY_LOCATION_LIST',
#     'TARGETING_TYPE_REGIONAL_LOCATION_LIST',
#     'TARGETING_TYPE_SENSITIVE_CATEGORY_EXCLUSION',
#     'TARGETING_TYPE_SUB_EXCHANGE',
#     'TARGETING_TYPE_THIRD_PARTY_VERIFIER',
#     'TARGETING_TYPE_URL',
#     'TARGETING_TYPE_USER_REWARDED_CONTENT',
#     'TARGETING_TYPE_VIDEO_PLAYER_SIZE',
#     'TARGETING_TYPE_VIEWABILITY'
# ]

# # Dynamically add columns based on the selected targeting types in targeting_types_list
# for targeting_type in targeting_types_list:
#     if targeting_type in targeting_details_mapping:
#         dataFrame_InsertionOrder_AssignedTargetingOptions[f'{targeting_type}'] = []


# # Deduplicate insertionOrderId when grouping by advertiserId
# grouped_insertion_orders = id_Name_table.groupby('Advertiser ID')['Insertion Order ID'].apply(lambda x: list(set(x.dropna()))).to_dict()

# # Loop through each advertiserId and its corresponding insertionOrderIds
# for AID, insertionOrderIdList in grouped_insertion_orders.items():
#     try:
#         print(f"Processing advertiser Id: {AID}")
#         for IOID in insertionOrderIdList:
#             if pd.isna(IOID):
#                 print(f"Skipping NaN insertionOrderId for advertiser Id: {AID}")
#                 continue

#             try:
#                 print(f"Processing insertionOrder Id: {IOID} for advertiser Id: {AID}")
#                 request = service.advertisers().insertionOrders().listAssignedTargetingOptions(advertiserId=AID, insertionOrderId=IOID)
#                 response = request.execute()

#                 new_row = {'advertiserId': AID, 'insertionOrderId': IOID}

#                 if 'assignedTargetingOptions' in response:
#                     insertionOrderListResponse = pd.json_normalize(response['assignedTargetingOptions'])
#                     for targeting_type in targeting_types_list:
#                         if targeting_type in insertionOrderListResponse['targetingType'].values:
#                             targeting_details = process_targeting_details(insertionOrderListResponse, targeting_type)
#                             new_row[f'{targeting_type}'] = targeting_details
#                         else:
#                             new_row[f'{targeting_type}'] = None

#                 dataFrame_InsertionOrder_AssignedTargetingOptions = pd.concat([dataFrame_InsertionOrder_AssignedTargetingOptions, pd.DataFrame([new_row])], ignore_index=True)

#             except Exception as e:
#                 print(f"An error occurred for IOID {IOID} in advertiser ID {AID}: {e}")

#     except Exception as e:
#         print(f"An error occurred for AID {AID}: {e}")

# # Output the final DataFrame
# print(dataFrame_InsertionOrder_AssignedTargetingOptions)

## Line Item

### Live

In [191]:
## Updated with full False Values

# Initialize the DataFrame with dynamic columns based on the specified targeting types
dataFrame_LineItem_AssignedTargetingOptions = pd.DataFrame({
    'advertiserId': [],
    'lineItemId': []
})

# List of targeting types to use
targeting_types_list = [
    'TARGETING_TYPE_AGE_RANGE',
    'TARGETING_TYPE_APP',
    'TARGETING_TYPE_APP_CATEGORY',
    'TARGETING_TYPE_AUDIENCE_GROUP',
    'TARGETING_TYPE_AUDIO_CONTENT_TYPE',
    'TARGETING_TYPE_AUTHORIZED_SELLER_STATUS',
    'TARGETING_TYPE_BROWSER',
    'TARGETING_TYPE_BUSINESS_CHAIN',
    'TARGETING_TYPE_CARRIER_AND_ISP',
    'TARGETING_TYPE_CATEGORY',
    'TARGETING_TYPE_CHANNEL',
    'TARGETING_TYPE_CONTENT_DURATION',
    'TARGETING_TYPE_CONTENT_GENRE',
    'TARGETING_TYPE_CONTENT_INSTREAM_POSITION',
    'TARGETING_TYPE_CONTENT_OUTSTREAM_POSITION',
    'TARGETING_TYPE_CONTENT_STREAM_TYPE',
    'TARGETING_TYPE_DAY_AND_TIME',
    'TARGETING_TYPE_DEVICE_MAKE_MODEL',
    'TARGETING_TYPE_DEVICE_TYPE',
    'TARGETING_TYPE_DIGITAL_CONTENT_LABEL_EXCLUSION',
    'TARGETING_TYPE_ENVIRONMENT',
    'TARGETING_TYPE_EXCHANGE',
    'TARGETING_TYPE_GENDER',
    'TARGETING_TYPE_GEO_REGION',
    'TARGETING_TYPE_HOUSEHOLD_INCOME',
    'TARGETING_TYPE_INVENTORY_SOURCE',
    'TARGETING_TYPE_INVENTORY_SOURCE_GROUP',
    'TARGETING_TYPE_KEYWORD',
    'TARGETING_TYPE_LANGUAGE',
    'TARGETING_TYPE_NATIVE_CONTENT_POSITION',
    'TARGETING_TYPE_NEGATIVE_KEYWORD_LIST',
    'TARGETING_TYPE_OMID',
    'TARGETING_TYPE_ON_SCREEN_POSITION',
    'TARGETING_TYPE_OPERATING_SYSTEM',
    'TARGETING_TYPE_PARENTAL_STATUS',
    'TARGETING_TYPE_POI',
    'TARGETING_TYPE_PROXIMITY_LOCATION_LIST',
    'TARGETING_TYPE_REGIONAL_LOCATION_LIST',
    'TARGETING_TYPE_SENSITIVE_CATEGORY_EXCLUSION',
    'TARGETING_TYPE_SUB_EXCHANGE',
    'TARGETING_TYPE_THIRD_PARTY_VERIFIER',
    'TARGETING_TYPE_URL',
    'TARGETING_TYPE_USER_REWARDED_CONTENT',
    'TARGETING_TYPE_VIDEO_PLAYER_SIZE',
    'TARGETING_TYPE_VIEWABILITY',
    'TARGETING_TYPE_YOUTUBE_CHANNEL',
    'TARGETING_TYPE_YOUTUBE_VIDEO'
]

# Dynamically add columns based on the selected targeting types in targeting_types_list
for targeting_type in targeting_types_list:
    if targeting_type in targeting_details_mapping:
        dataFrame_LineItem_AssignedTargetingOptions[f'{targeting_type}'] = []

# Function to extract and process targeting details
def process_targeting_details(filtered_response, targeting_type):
    targeting_fields = targeting_details_mapping[targeting_type]
    required_columns = ['assignedTargetingOption.' + field for field in targeting_fields[:-2]]

    # Adjust for potential missing .negative columns and other expected columns
    for col in required_columns:
        if '.negative' in col:  # This is the equivalent of a wildcard check
            # Handle the .negative columns by:
            # 1. Adding the column if it doesn't exist, with a default value of False.
            # 2. If the column exists but has null, NaN, or empty values, replace those with False.
            if col not in filtered_response.columns:
                # Ensure we're working with a copy and use .loc for assignment
                filtered_response = filtered_response.copy()
                filtered_response.loc[:, col] = False  # Add the .negative column with default False
            else:
                # Replace NaN, None, and empty values with False
                filtered_response = filtered_response.copy()  # Work on a copy to avoid modifying views
                filtered_response.loc[:, col] = filtered_response[col].apply(lambda x: False if pd.isna(x) or x == '' else x)
        else:
            # For other missing columns, set them to None/NaN
            if col not in filtered_response.columns:
                filtered_response = filtered_response.copy()
                filtered_response.loc[:, col] = np.nan  # Set missing columns to NaN using .loc

    # Check if all required columns exist in the filtered response
    if all(col in filtered_response.columns for col in required_columns):
        # Select the relevant fields based on the mapping
        filtered_details = filtered_response[required_columns]

        # Sort the details according to the specified index in the mapping
        sorting_column_index = targeting_fields[-2]
        sorted_details = filtered_details.sort_values(by=[required_columns[sorting_column_index]])

        # Convert the sorted DataFrame to a list of lists (each inner list corresponds to a row)
        return sorted_details.values.tolist()
    else:
        missing_columns = [col for col in required_columns if col not in filtered_response.columns]
        print(f"Missing columns for {targeting_type}: {missing_columns}")
        return None


# Deduplicate lineItemId when grouping by advertiserId
grouped_line_items = id_Name_table.groupby('Advertiser ID')['Line Item ID'].apply(lambda x: list(set(x.dropna()))).to_dict()

# Loop through each advertiserId and its corresponding lineItemIds
for AID, lineItemIdList in grouped_line_items.items():
    try:
        # Print the current advertiser Id
        print(f"Processing advertiser Id: {AID}")

        # Loop through each lineItem ID
        for LIID in lineItemIdList:
            # Check if LIID is not NaN before processing
            if pd.isna(LIID):
                print(f"Skipping NaN lineItemId for advertiser Id: {AID}")
                continue

            try:
                # Print the current lineItem Id
                print(f"Processing lineItem Id: {LIID} for advertiser Id: {AID}")

                # Make a single API request to get all targeting types for this lineItem
                request = service.advertisers().lineItems().bulkListAssignedTargetingOptions(
                    advertiserId=AID,
                    lineItemIds=[LIID]  # Ensure lineItemId is passed as a list
                )
                response = request.execute()

                # Initialize a dictionary to hold the data for this advertiser and lineItem
                new_row = {'advertiserId': AID, 'lineItemId': LIID}

                # Process each targeting type in the response
                if 'lineItemAssignedTargetingOptions' in response:
                    lineItemListResponse = pd.json_normalize(response['lineItemAssignedTargetingOptions'])

                    if 'assignedTargetingOption.targetingType' in lineItemListResponse.columns:
                        for targeting_type in targeting_types_list:
                            if targeting_type in lineItemListResponse['assignedTargetingOption.targetingType'].values:
                                # Filter rows that match the targeting_type
                                filtered_response = lineItemListResponse[lineItemListResponse['assignedTargetingOption.targetingType'] == targeting_type]
                                # Process the targeting details for the current targeting type
                                targeting_details = process_targeting_details(filtered_response, targeting_type)
                                new_row[f'{targeting_type}'] = targeting_details
                            else:
                                new_row[f'{targeting_type}'] = None  # If the targeting type is not in the response, set to None
                    else:
                        print(f"targetingType not found in response for lineItem Id: {LIID}")

                # Append the completed row for this advertiser and lineItem to the DataFrame
                dataFrame_LineItem_AssignedTargetingOptions = pd.concat([dataFrame_LineItem_AssignedTargetingOptions, pd.DataFrame([new_row])], ignore_index=True)

            except Exception as e:
                print(f"An error occurred for LIID {LIID} in advertiser ID {AID}: {e}")

    except Exception as e:
        print(f"An error occurred for AID {AID}: {e}")

# # Output the final DataFrame
# print(dataFrame_LineItem_AssignedTargetingOptions)

Processing advertiser Id: 1053287434
Processing advertiser Id: 1074131012
Processing lineItem Id: 21262902387 for advertiser Id: 1074131012
Processing lineItem Id: 21258000160 for advertiser Id: 1074131012
Processing lineItem Id: 21247607274 for advertiser Id: 1074131012
Processing lineItem Id: 21262901451 for advertiser Id: 1074131012
Processing advertiser Id: 1074902854
Processing advertiser Id: 1083545221
Processing advertiser Id: 1087573164
Processing advertiser Id: 1092229838
Processing advertiser Id: 1118736880
Processing advertiser Id: 1158669972
Processing advertiser Id: 1180481268
Processing advertiser Id: 1180925680
Processing advertiser Id: 1190803835
Processing advertiser Id: 1346154400
Processing lineItem Id: 21297076960 for advertiser Id: 1346154400
Processing lineItem Id: 21293035862 for advertiser Id: 1346154400
Processing lineItem Id: 21296966839 for advertiser Id: 1346154400
Processing lineItem Id: 21293142155 for advertiser Id: 1346154400
Processing lineItem Id: 2128

### Testing

In [48]:
# ## Testing inclusion or negative or not with true or false

# # Initialize the DataFrame with dynamic columns based on the specified targeting types
# dataFrame_LineItem_AssignedTargetingOptions = pd.DataFrame({
#     'advertiserId': [],
#     'lineItemId': []
# })

# # List of targeting types to use
# targeting_types_list = [
#     'TARGETING_TYPE_AGE_RANGE',
#     'TARGETING_TYPE_APP',
#     'TARGETING_TYPE_APP_CATEGORY',
#     'TARGETING_TYPE_AUDIENCE_GROUP',
#     'TARGETING_TYPE_AUDIO_CONTENT_TYPE',
#     'TARGETING_TYPE_AUTHORIZED_SELLER_STATUS',
#     'TARGETING_TYPE_BROWSER',
#     'TARGETING_TYPE_BUSINESS_CHAIN',
#     'TARGETING_TYPE_CARRIER_AND_ISP',
#     'TARGETING_TYPE_CATEGORY',
#     'TARGETING_TYPE_CHANNEL',
#     'TARGETING_TYPE_CONTENT_DURATION',
#     'TARGETING_TYPE_CONTENT_GENRE',
#     'TARGETING_TYPE_CONTENT_INSTREAM_POSITION',
#     'TARGETING_TYPE_CONTENT_OUTSTREAM_POSITION',
#     'TARGETING_TYPE_CONTENT_STREAM_TYPE',
#     'TARGETING_TYPE_DAY_AND_TIME',
#     'TARGETING_TYPE_DEVICE_MAKE_MODEL',
#     'TARGETING_TYPE_DEVICE_TYPE',
#     'TARGETING_TYPE_DIGITAL_CONTENT_LABEL_EXCLUSION',
#     'TARGETING_TYPE_ENVIRONMENT',
#     'TARGETING_TYPE_EXCHANGE',
#     'TARGETING_TYPE_GENDER',
#     'TARGETING_TYPE_GEO_REGION',
#     'TARGETING_TYPE_HOUSEHOLD_INCOME',
#     'TARGETING_TYPE_INVENTORY_SOURCE',
#     'TARGETING_TYPE_INVENTORY_SOURCE_GROUP',
#     'TARGETING_TYPE_KEYWORD',
#     'TARGETING_TYPE_LANGUAGE',
#     'TARGETING_TYPE_NATIVE_CONTENT_POSITION',
#     'TARGETING_TYPE_NEGATIVE_KEYWORD_LIST',
#     'TARGETING_TYPE_OMID',
#     'TARGETING_TYPE_ON_SCREEN_POSITION',
#     'TARGETING_TYPE_OPERATING_SYSTEM',
#     'TARGETING_TYPE_PARENTAL_STATUS',
#     'TARGETING_TYPE_POI',
#     'TARGETING_TYPE_PROXIMITY_LOCATION_LIST',
#     'TARGETING_TYPE_REGIONAL_LOCATION_LIST',
#     'TARGETING_TYPE_SENSITIVE_CATEGORY_EXCLUSION',
#     'TARGETING_TYPE_SUB_EXCHANGE',
#     'TARGETING_TYPE_THIRD_PARTY_VERIFIER',
#     'TARGETING_TYPE_URL',
#     'TARGETING_TYPE_USER_REWARDED_CONTENT',
#     'TARGETING_TYPE_VIDEO_PLAYER_SIZE',
#     'TARGETING_TYPE_VIEWABILITY',
#     'TARGETING_TYPE_YOUTUBE_CHANNEL',
#     'TARGETING_TYPE_YOUTUBE_VIDEO'
# ]

# # Dynamically add columns based on the selected targeting types in targeting_types_list
# for targeting_type in targeting_types_list:
#     if targeting_type in targeting_details_mapping:
#         dataFrame_LineItem_AssignedTargetingOptions[f'{targeting_type}'] = []

# # Function to extract and process targeting details
# def process_targeting_details(filtered_response, targeting_type):
#     targeting_fields = targeting_details_mapping[targeting_type]
#     required_columns = ['assignedTargetingOption.' + field for field in targeting_fields[:-2]]

#     # Adjust for potential missing .negative columns and other expected columns
#     for col in required_columns:
#         if '.negative' in col:
#             # Handle the .negative columns specifically by setting to False if missing
#             if col not in filtered_response.columns:
#                 filtered_response = filtered_response.copy()  # Ensure we're working with a copy to avoid the warning
#                 filtered_response.loc[:, col] = False  # Set default value to False if .negative column is missing
#         else:
#             # For other missing columns, set them to None/NaN
#             if col not in filtered_response.columns:
#                 filtered_response = filtered_response.copy()
#                 filtered_response.loc[:, col] = np.nan  # Set missing columns to NaN

#     # Check if the required columns exist in the filtered response
#     if all(col in filtered_response.columns for col in required_columns):
#         # Select the relevant fields based on the mapping
#         filtered_details = filtered_response[required_columns]

#         # Sort the details according to the specified index in the mapping
#         sorting_column_index = targeting_fields[-2]
#         sorted_details = filtered_details.sort_values(by=[required_columns[sorting_column_index]])

#         # Convert the sorted DataFrame to a list of lists (each inner list corresponds to a row)
#         return sorted_details.values.tolist()
#     else:
#         missing_columns = [col for col in required_columns if col not in filtered_response.columns]
#         print(f"Missing columns for {targeting_type}: {missing_columns}")
#         return None

# # Deduplicate lineItemId when grouping by advertiserId
# grouped_line_items = id_Name_table.groupby('Advertiser ID')['Line Item ID'].apply(lambda x: list(set(x.dropna()))).to_dict()

# # Loop through each advertiserId and its corresponding lineItemIds
# for AID, lineItemIdList in grouped_line_items.items():
#     try:
#         # Print the current advertiser Id
#         print(f"Processing advertiser Id: {AID}")

#         # Loop through each lineItem ID
#         for LIID in lineItemIdList:
#             # Check if LIID is not NaN before processing
#             if pd.isna(LIID):
#                 print(f"Skipping NaN lineItemId for advertiser Id: {AID}")
#                 continue

#             try:
#                 # Print the current lineItem Id
#                 print(f"Processing lineItem Id: {LIID} for advertiser Id: {AID}")

#                 # Make a single API request to get all targeting types for this lineItem
#                 request = service.advertisers().lineItems().bulkListAssignedTargetingOptions(
#                     advertiserId=AID,
#                     lineItemIds=[LIID]  # Ensure lineItemId is passed as a list
#                 )
#                 response = request.execute()

#                 # Initialize a dictionary to hold the data for this advertiser and lineItem
#                 new_row = {'advertiserId': AID, 'lineItemId': LIID}

#                 # Process each targeting type in the response
#                 if 'lineItemAssignedTargetingOptions' in response:
#                     lineItemListResponse = pd.json_normalize(response['lineItemAssignedTargetingOptions'])

#                     if 'assignedTargetingOption.targetingType' in lineItemListResponse.columns:
#                         for targeting_type in targeting_types_list:
#                             if targeting_type in lineItemListResponse['assignedTargetingOption.targetingType'].values:
#                                 # Filter rows that match the targeting_type
#                                 filtered_response = lineItemListResponse[lineItemListResponse['assignedTargetingOption.targetingType'] == targeting_type]
#                                 # Process the targeting details for the current targeting type
#                                 targeting_details = process_targeting_details(filtered_response, targeting_type)
#                                 new_row[f'{targeting_type}'] = targeting_details
#                             else:
#                                 new_row[f'{targeting_type}'] = None  # If the targeting type is not in the response, set to None
#                     else:
#                         print(f"targetingType not found in response for lineItem Id: {LIID}")

#                 # Append the completed row for this advertiser and lineItem to the DataFrame
#                 dataFrame_LineItem_AssignedTargetingOptions = pd.concat([dataFrame_LineItem_AssignedTargetingOptions, pd.DataFrame([new_row])], ignore_index=True)

#             except Exception as e:
#                 print(f"An error occurred for LIID {LIID} in advertiser ID {AID}: {e}")

#     except Exception as e:
#         print(f"An error occurred for AID {AID}: {e}")

# # Output the final DataFrame
# print(dataFrame_LineItem_AssignedTargetingOptions)


In [49]:
# # Function to extract and process targeting details
# def process_targeting_details(filtered_response, targeting_type):
#     targeting_fields = targeting_details_mapping[targeting_type]
#     required_columns = ['assignedTargetingOption.' + field for field in targeting_fields[:-2]]

#     # Adjust for potential missing .negative columns and other expected columns
#     for col in required_columns:
#         if '.negative' in col:  # This is the equivalent of a wildcard check
#             # Handle the .negative columns specifically by setting to False if missing or NaN
#             if col not in filtered_response.columns:
#                 filtered_response = filtered_response.copy()  # Ensure we're working with a copy to avoid the warning
#                 filtered_response[col] = False  # Set default value to False if .negative column is missing
#             else:
#                 filtered_response[col] = filtered_response[col].fillna(False)  # Replace NaN with False
#         else:
#             # For other missing columns, set them to None/NaN
#             if col not in filtered_response.columns:
#                 filtered_response = filtered_response.copy()
#                 filtered_response[col] = np.nan  # Set missing columns to NaN

#     # Check if all required columns exist in the filtered response
#     if all(col in filtered_response.columns for col in required_columns):
#         # Select the relevant fields based on the mapping
#         filtered_details = filtered_response[required_columns]

#         # Sort the details according to the specified index in the mapping
#         sorting_column_index = targeting_fields[-2]
#         sorted_details = filtered_details.sort_values(by=[required_columns[sorting_column_index]])

#         # Convert the sorted DataFrame to a list of lists (each inner list corresponds to a row)
#         return sorted_details.values.tolist()
#     else:
#         missing_columns = [col for col in required_columns if col not in filtered_response.columns]
#         print(f"Missing columns for {targeting_type}: {missing_columns}")
#         return None

# # Example usage of the updated function:
# # This assumes your 'id_Name_table' and 'grouped_line_items' are already defined.
# for AID, lineItemIdList in grouped_line_items.items():
#     try:
#         for LIID in lineItemIdList:
#             if pd.isna(LIID):
#                 continue
#             try:
#                 # Make a single API request to get all targeting types for this lineItem
#                 request = service.advertisers().lineItems().bulkListAssignedTargetingOptions(
#                     advertiserId=AID,
#                     lineItemIds=[LIID]
#                 )
#                 response = request.execute()

#                 new_row = {'advertiserId': AID, 'lineItemId': LIID}

#                 if 'lineItemAssignedTargetingOptions' in response:
#                     lineItemListResponse = pd.json_normalize(response['lineItemAssignedTargetingOptions'])
#                     if 'assignedTargetingOption.targetingType' in lineItemListResponse.columns:
#                         for targeting_type in targeting_types_list:
#                             if targeting_type in lineItemListResponse['assignedTargetingOption.targetingType'].values:
#                                 filtered_response = lineItemListResponse[lineItemListResponse['assignedTargetingOption.targetingType'] == targeting_type]
#                                 targeting_details = process_targeting_details(filtered_response, targeting_type)
#                                 new_row[f'{targeting_type}'] = targeting_details
#                             else:
#                                 new_row[f'{targeting_type}'] = None
#                     else:
#                         print(f"targetingType not found in response for lineItem Id: {LIID}")

#                 dataFrame_LineItem_AssignedTargetingOptions = pd.concat([dataFrame_LineItem_AssignedTargetingOptions, pd.DataFrame([new_row])], ignore_index=True)

#             except Exception as e:
#                 print(f"An error occurred for LIID {LIID} in advertiser ID {AID}: {e}")

#     except Exception as e:
#         print(f"An error occurred for AID {AID}: {e}")

# # Output the final DataFrame
# print(dataFrame_LineItem_AssignedTargetingOptions)


## AdGroup

### Live

In [50]:
## Testing

# Initialize the DataFrame with dynamic columns based on the specified targeting types
dataFrame_AdGroup_AssignedTargetingOptions = pd.DataFrame({
    'advertiserId': [],
    'adGroupId': []
})

# List of targeting types to use
targeting_types_list = [

    'TARGETING_TYPE_AGE_RANGE',
    'TARGETING_TYPE_APP',
    'TARGETING_TYPE_APP_CATEGORY',
    'TARGETING_TYPE_AUDIENCE_GROUP',
    'TARGETING_TYPE_CATEGORY',
    'TARGETING_TYPE_GENDER',
    'TARGETING_TYPE_HOUSEHOLD_INCOME',
    'TARGETING_TYPE_KEYWORD',
    'TARGETING_TYPE_PARENTAL_STATUS',
    'TARGETING_TYPE_SESSION_POSITION',
    'TARGETING_TYPE_URL',
    'TARGETING_TYPE_YOUTUBE_CHANNEL',
    'TARGETING_TYPE_YOUTUBE_VIDEO'
]

# Dynamically add columns based on the selected targeting types in targeting_types_list
for targeting_type in targeting_types_list:
    if targeting_type in targeting_details_mapping:
        dataFrame_AdGroup_AssignedTargetingOptions[f'{targeting_type}'] = []

# Function to extract and process targeting details
def process_targeting_details(filtered_response, targeting_type):
    targeting_fields = targeting_details_mapping[targeting_type]
    required_columns = ['assignedTargetingOption.' + field for field in targeting_fields[:-2]]

    # Adjust for potential missing .negative columns and other expected columns
    for col in required_columns:
        if '.negative' in col:  # This is the equivalent of a wildcard check
            # Handle the .negative columns by:
            # 1. Adding the column if it doesn't exist, with a default value of False.
            # 2. If the column exists but has null, NaN, or empty values, replace those with False.
            if col not in filtered_response.columns:
                # Ensure we're working with a copy and use .loc for assignment
                filtered_response = filtered_response.copy()
                filtered_response.loc[:, col] = False  # Add the .negative column with default False
            else:
                # Replace NaN, None, and empty values with False
                filtered_response = filtered_response.copy()  # Work on a copy to avoid modifying views
                filtered_response.loc[:, col] = filtered_response[col].apply(lambda x: False if pd.isna(x) or x == '' else x)
        else:
            # For other missing columns, set them to None/NaN
            if col not in filtered_response.columns:
                filtered_response = filtered_response.copy()
                filtered_response.loc[:, col] = np.nan  # Set missing columns to NaN using .loc

    # Check if all required columns exist in the filtered response
    if all(col in filtered_response.columns for col in required_columns):
        # Select the relevant fields based on the mapping
        filtered_details = filtered_response[required_columns]

        # Sort the details according to the specified index in the mapping
        sorting_column_index = targeting_fields[-2]
        sorted_details = filtered_details.sort_values(by=[required_columns[sorting_column_index]])

        # Convert the sorted DataFrame to a list of lists (each inner list corresponds to a row)
        return sorted_details.values.tolist()
    else:
        missing_columns = [col for col in required_columns if col not in filtered_response.columns]
        print(f"Missing columns for {targeting_type}: {missing_columns}")
        return None


# Deduplicate adGroupId when grouping by advertiserId
grouped_ad_groups = id_Name_table.groupby('Advertiser ID')['Ad Group ID'].apply(lambda x: list(set(x.dropna()))).to_dict()

# Loop through each advertiserId and its corresponding adGroupIds
for AID, adGroupIdList in grouped_ad_groups.items():
    try:
        # Print the current advertiser Id
        print(f"Processing advertiser Id: {AID}")

        # Loop through each AdGroup ID
        for AGID in adGroupIdList:
            # Check if AGID is not NaN before processing
            if pd.isna(AGID):
                print(f"Skipping NaN adGroupId for advertiser Id: {AID}")
                continue

            try:
                # Print the current AdGroup Id
                print(f"Processing AdGroup Id: {AGID} for advertiser Id: {AID}")

                # Make a single API request to get all targeting types for this AdGroup
                request = service.advertisers().adGroups().bulkListAdGroupAssignedTargetingOptions(
                    advertiserId=AID,
                    adGroupIds=[AGID]  # Ensure adGroupId is passed as a list
                )
                response = request.execute()

                # Initialize a dictionary to hold the data for this advertiser and AdGroup
                new_row = {'advertiserId': AID, 'adGroupId': AGID}

                # Process each targeting type in the response
                if 'adGroupAssignedTargetingOptions' in response:
                    AdGroupListResponse = pd.json_normalize(response['adGroupAssignedTargetingOptions'])

                    if 'assignedTargetingOption.targetingType' in AdGroupListResponse.columns:
                        for targeting_type in targeting_types_list:
                            if targeting_type in AdGroupListResponse['assignedTargetingOption.targetingType'].values:
                                # Filter rows that match the targeting_type
                                filtered_response = AdGroupListResponse[AdGroupListResponse['assignedTargetingOption.targetingType'] == targeting_type]
                                # Process the targeting details for the current targeting type
                                targeting_details = process_targeting_details(filtered_response, targeting_type)
                                new_row[f'{targeting_type}'] = targeting_details
                            else:
                                new_row[f'{targeting_type}'] = None  # If the targeting type is not in the response, set to None
                    else:
                        print(f"targetingType not found in response for AdGroup Id: {AGID}")

                # Append the completed row for this advertiser and AdGroup to the DataFrame
                dataFrame_AdGroup_AssignedTargetingOptions = pd.concat([dataFrame_AdGroup_AssignedTargetingOptions, pd.DataFrame([new_row])], ignore_index=True)

            except Exception as e:
                print(f"An error occurred for AGID {AGID} in advertiser ID {AID}: {e}")

    except Exception as e:
        print(f"An error occurred for AID {AID}: {e}")

# # Output the final DataFrame
# print(dataFrame_AdGroup_AssignedTargetingOptions)

Processing advertiser Id: 1053287434
Processing advertiser Id: 1074131012
Processing AdGroup Id: 163700674482 for advertiser Id: 1074131012
Processing AdGroup Id: 162597600955 for advertiser Id: 1074131012
Processing AdGroup Id: 162697546540 for advertiser Id: 1074131012
Processing AdGroup Id: 164496366640 for advertiser Id: 1074131012
Processing advertiser Id: 1074902854
Processing advertiser Id: 1083545221
Processing advertiser Id: 1087573164
Processing advertiser Id: 1092229838
Processing advertiser Id: 1118736880
Processing advertiser Id: 1158669972
Processing advertiser Id: 1180481268
Processing advertiser Id: 1180925680
Processing advertiser Id: 1190803835
Processing advertiser Id: 1346154400
Processing AdGroup Id: 162186133162 for advertiser Id: 1346154400
Processing AdGroup Id: 166080441807 for advertiser Id: 1346154400
Processing AdGroup Id: 162186133082 for advertiser Id: 1346154400
Processing AdGroup Id: 159277899294 for advertiser Id: 1346154400
Processing AdGroup Id: 16401

### Testing

In [51]:
# ## Testing

# ## Updated with full False Values

# # Initialize the DataFrame with dynamic columns based on the specified targeting types
# dataFrame_AdGroup_AssignedTargetingOptions = pd.DataFrame({
#     'advertiserId': [],
#     'adGroupId': []
# })

# # List of targeting types to use
# targeting_types_list = [

#     'TARGETING_TYPE_AGE_RANGE',
#     'TARGETING_TYPE_APP',
#     'TARGETING_TYPE_APP_CATEGORY',
#     'TARGETING_TYPE_AUDIENCE_GROUP',
#     'TARGETING_TYPE_CATEGORY',
#     'TARGETING_TYPE_GENDER',
#     'TARGETING_TYPE_HOUSEHOLD_INCOME',
#     'TARGETING_TYPE_KEYWORD',
#     'TARGETING_TYPE_PARENTAL_STATUS',
#     'TARGETING_TYPE_SESSION_POSITION',
#     'TARGETING_TYPE_URL',
#     'TARGETING_TYPE_YOUTUBE_CHANNEL',
#     'TARGETING_TYPE_YOUTUBE_VIDEO'
# ]

# # Dynamically add columns based on the selected targeting types in targeting_types_list
# for targeting_type in targeting_types_list:
#     if targeting_type in targeting_details_mapping:
#         dataFrame_AdGroup_AssignedTargetingOptions[f'{targeting_type}'] = []

# # Function to extract and process targeting details
# def process_targeting_details(filtered_response, targeting_type):
#     targeting_fields = targeting_details_mapping[targeting_type]
#     required_columns = ['assignedTargetingOption.' + field for field in targeting_fields[:-2]]

#     # Adjust for potential missing .negative columns and other expected columns
#     for col in required_columns:
#         if '.negative' in col:  # This is the equivalent of a wildcard check
#             # Handle the .negative columns by:
#             # 1. Adding the column if it doesn't exist, with a default value of False.
#             # 2. If the column exists but has null, NaN, or empty values, replace those with False.
#             if col not in filtered_response.columns:
#                 # Ensure we're working with a copy and use .loc for assignment
#                 filtered_response = filtered_response.copy()
#                 filtered_response.loc[:, col] = False  # Add the .negative column with default False
#             else:
#                 # Replace NaN, None, and empty values with False
#                 filtered_response = filtered_response.copy()  # Work on a copy to avoid modifying views
#                 filtered_response.loc[:, col] = filtered_response[col].apply(lambda x: False if pd.isna(x) or x == '' else x)
#         else:
#             # For other missing columns, set them to None/NaN
#             if col not in filtered_response.columns:
#                 filtered_response = filtered_response.copy()
#                 filtered_response.loc[:, col] = np.nan  # Set missing columns to NaN using .loc

#     # Check if all required columns exist in the filtered response
#     if all(col in filtered_response.columns for col in required_columns):
#         # Select the relevant fields based on the mapping
#         filtered_details = filtered_response[required_columns]

#         # Sort the details according to the specified index in the mapping
#         sorting_column_index = targeting_fields[-2]
#         sorted_details = filtered_details.sort_values(by=[required_columns[sorting_column_index]])

#         # Convert the sorted DataFrame to a list of lists (each inner list corresponds to a row)
#         return sorted_details.values.tolist()
#     else:
#         missing_columns = [col for col in required_columns if col not in filtered_response.columns]
#         print(f"Missing columns for {targeting_type}: {missing_columns}")
#         return None


# # Deduplicate adGroupId when grouping by advertiserId
# grouped_ad_groups = id_Name_table.groupby('Advertiser ID')['Ad Group ID'].apply(lambda x: list(set(x.dropna()))).to_dict()

# # Loop through each advertiserId and its corresponding adGroupIds
# for AID, adGroupIdList in grouped_ad_groups.items():
#     try:
#         # Print the current advertiser Id
#         print(f"Processing advertiser Id: {AID}")

#         # Loop through each AdGroup ID
#         for AGID in adGroupIdList:
#             # Check if AGID is not NaN before processing
#             if pd.isna(AGID):
#                 print(f"Skipping NaN adGroupId for advertiser Id: {AID}")
#                 continue

#             try:
#                 # Print the current AdGroup Id
#                 print(f"Processing AdGroup Id: {AGID} for advertiser Id: {AID}")

#                 # Make a single API request to get all targeting types for this AdGroup
#                 request = service.advertisers().adGroups().bulkListAdGroupAssignedTargetingOptions(
#                     advertiserId=AID,
#                     adGroupIds=[AGID]  # Ensure adGroupId is passed as a list
#                 )
#                 response = request.execute()

#                 # Initialize a dictionary to hold the data for this advertiser and AdGroup
#                 new_row = {'advertiserId': AID, 'adGroupId': AGID}

#                 # Process each targeting type in the response
#                 if 'AdGroupAssignedTargetingOptions' in response:
#                     AdGroupListResponse = pd.json_normalize(response['AdGroupAssignedTargetingOptions'])

#                     if 'assignedTargetingOption.targetingType' in AdGroupListResponse.columns:
#                         for targeting_type in targeting_types_list:
#                             if targeting_type in AdGroupListResponse['assignedTargetingOption.targetingType'].values:
#                                 # Filter rows that match the targeting_type
#                                 filtered_response = AdGroupListResponse[AdGroupListResponse['assignedTargetingOption.targetingType'] == targeting_type]
#                                 # Process the targeting details for the current targeting type
#                                 targeting_details = process_targeting_details(filtered_response, targeting_type)
#                                 new_row[f'{targeting_type}'] = targeting_details
#                             else:
#                                 new_row[f'{targeting_type}'] = None  # If the targeting type is not in the response, set to None
#                     else:
#                         print(f"targetingType not found in response for AdGroup Id: {AGID}")

#                 # Append the completed row for this advertiser and AdGroup to the DataFrame
#                 dataFrame_AdGroup_AssignedTargetingOptions = pd.concat([dataFrame_AdGroup_AssignedTargetingOptions, pd.DataFrame([new_row])], ignore_index=True)

#             except Exception as e:
#                 print(f"An error occurred for AGID {AGID} in advertiser ID {AID}: {e}")

#     except Exception as e:
#         print(f"An error occurred for AID {AID}: {e}")

# # # Output the final DataFrame
# # print(dataFrame_AdGroup_AssignedTargetingOptions)

In [52]:
# # Deduplicate adGroupId when grouping by advertiserId
# grouped_ad_groups = id_Name_table.groupby('Advertiser ID')['Ad Group ID'].apply(lambda x: list(set(x.dropna()))).to_dict()

# # Initialize the DataFrame with dynamic columns based on the specified targeting types
# dataFrame_AdGroup_AssignedTargetingOptions = pd.DataFrame({
#     'advertiserId': [],
#     'adGroupId': []  # Include adGroupId in the DataFrame
# })

# # List of targeting types to use
# targeting_types_list = [
#     'TARGETING_TYPE_AGE_RANGE',
#     'TARGETING_TYPE_APP',
#     'TARGETING_TYPE_APP_CATEGORY',
#     'TARGETING_TYPE_AUDIENCE_GROUP',
#     'TARGETING_TYPE_CATEGORY',
#     'TARGETING_TYPE_GENDER',
#     'TARGETING_TYPE_HOUSEHOLD_INCOME',
#     'TARGETING_TYPE_KEYWORD',
#     'TARGETING_TYPE_PARENTAL_STATUS',
#     'TARGETING_TYPE_SESSION_POSITION',
#     'TARGETING_TYPE_URL',
#     'TARGETING_TYPE_YOUTUBE_CHANNEL',
#     'TARGETING_TYPE_YOUTUBE_VIDEO'
# ]

# # Dynamically add columns based on the selected targeting types in targeting_types_list
# for targeting_type in targeting_types_list:
#     if targeting_type in targeting_details_mapping:
#         dataFrame_AdGroup_AssignedTargetingOptions[f'{targeting_type}'] = []

# # Function to retry API calls in case of SSL errors
# def api_request_with_retry(request, retries=3):
#     for attempt in range(retries):
#         try:
#             response = request.execute()
#             return response
#         except SSLError as e:
#             print(f"SSL Error encountered: {e}. Retrying... ({attempt + 1}/{retries})")
#             time.sleep(2)  # Sleep for 2 seconds before retrying
#         except Exception as e:
#             print(f"Error occurred: {e}")
#             return None
#     return None

# # Loop through each advertiserId and its corresponding deduplicated adGroup IDs
# for AID, adGroupIdList in grouped_ad_groups.items():
#     try:
#         print(f"Processing advertiser Id: {AID}")

#         for AGID in adGroupIdList:
#             if pd.isna(AGID):
#                 print(f"Skipping NaN adGroupId for advertiser Id: {AID}")
#                 continue

#             try:
#                 print(f"Processing adGroup Id: {AGID} for advertiser Id: {AID}")

#                 # Make the API request with retries
#                 request = service.advertisers().adGroups().bulkListAdGroupAssignedTargetingOptions(
#                     advertiserId=AID,
#                     adGroupIds=[AGID],
#                 )
#                 response = api_request_with_retry(request)

#                 if response is None:
#                     continue

#                 new_row = {'advertiserId': AID, 'adGroupId': AGID}

#                 if 'adGroupAssignedTargetingOptions' in response:
#                     adGroupListResponse = pd.json_normalize(response['adGroupAssignedTargetingOptions'])

#                     for targeting_type in targeting_types_list:
#                         if 'adGroupAssignedTargetingOptions.targetingType' in adGroupListResponse.columns:
#                             targeting_details = adGroupListResponse[adGroupListResponse['adGroupAssignedTargetingOptions.targetingType'] == targeting_type]
#                             new_row[f'{targeting_type}'] = targeting_details.to_dict(orient='records') if not targeting_details.empty else None
#                         else:
#                             new_row[f'{targeting_type}'] = None

#                 dataFrame_AdGroup_AssignedTargetingOptions = pd.concat([dataFrame_AdGroup_AssignedTargetingOptions, pd.DataFrame([new_row])], ignore_index=True)

#             except KeyError as e:
#                 print(f"KeyError: {e} for AGID {AGID} in advertiser ID {AID}")
#             except Exception as e:
#                 print(f"An error occurred for AGID {AGID} in advertiser ID {AID}: {e}")

#     except Exception as e:
#         print(f"An error occurred for AID {AID}: {e}")

# # Output the final DataFrame
# print(dataFrame_AdGroup_AssignedTargetingOptions)

## Additional Data to Full Heirachy Details

### Impression and View Frequency Cap - Second Value

In [200]:
def frequency_calculate_seconds(row):
    time_unit = row['frequencyCap.timeUnit']
    time_count = row['frequencyCap.timeUnitCount']
    max_impressions = row['frequencyCap.maxImpressions']

    # Convert time_unit to seconds
    if time_unit == 'TIME_UNIT_MINUTES':
        seconds = time_count * 60
    elif time_unit == 'TIME_UNIT_HOURS':
        seconds = time_count * 3600
    elif time_unit == 'TIME_UNIT_DAYS':
        seconds = time_count * 86400
    elif time_unit == 'TIME_UNIT_WEEKS':
        seconds = time_count * 604800
    elif time_unit == 'TIME_UNIT_MONTHS':
        seconds = time_count * 18144000  # Assuming an average month duration
    elif time_unit == 'TIME_UNIT_LIFETIME':
        seconds = time_count * 31536000  # Assuming lifetime as 1 year (can be adjusted)
    else:
        seconds = 0

    # Calculate CamSeconds with protection against division by zero
    return (seconds / max_impressions) if max_impressions > 0 else 0

def viewFrequency_calculate_seconds(row):
    time_unit = row['youtubeAndPartnersSettings.viewFrequencyCap.timeUnit']
    time_count = row['youtubeAndPartnersSettings.viewFrequencyCap.timeUnitCount']
    max_impressions = row['youtubeAndPartnersSettings.viewFrequencyCap.maxViews']

    # Convert time_unit to seconds
    if time_unit == 'TIME_UNIT_MINUTES':
        seconds = time_count * 60
    elif time_unit == 'TIME_UNIT_HOURS':
        seconds = time_count * 3600
    elif time_unit == 'TIME_UNIT_DAYS':
        seconds = time_count * 86400
    elif time_unit == 'TIME_UNIT_WEEKS':
        seconds = time_count * 604800
    elif time_unit == 'TIME_UNIT_MONTHS':
        seconds = time_count * 18144000  # Assuming an average month duration
    elif time_unit == 'TIME_UNIT_LIFETIME':
        seconds = time_count * 31536000  # Assuming lifetime as 1 year (can be adjusted)
    else:
        seconds = 0

    # Calculate CamSeconds with protection against division by zero
    return (seconds / max_impressions) if max_impressions > 0 else 0

# Apply the function to calculate Frequency Seconds for each row in dataFrame_Campaigns_List
dataFrame_Campaigns_List['frequencyCap.seconds'] = dataFrame_Campaigns_List.apply(frequency_calculate_seconds, axis=1)

# Apply the function to calculate Frequency Seconds for each row in dataFrame_InsertionOrders_List
dataFrame_InsertionOrders_List['frequencyCap.seconds'] = dataFrame_InsertionOrders_List.apply(frequency_calculate_seconds, axis=1)

# Apply the function to calculate Frequency Seconds for each row in dataFrame_LineItems_List
dataFrame_LineItems_List['frequencyCap.seconds'] = dataFrame_LineItems_List.apply(frequency_calculate_seconds, axis=1)

# Apply the function to calculate Frequency Seconds for each row in dataFrame_LineItems_List
dataFrame_LineItems_List['youtubeAndPartnersSettings.viewFrequencyCap.maxViews.seconds'] = dataFrame_LineItems_List.apply(viewFrequency_calculate_seconds, axis=1)

## Data & Directories - Turn all list dataFrames into a CSV

In [201]:
# Define the directory to save the CSV files
data_directory = '3. Assigned Targeting Options'

# Create the directory if it doesn't exist
if not os.path.exists(data_directory):
    os.makedirs(data_directory)

# Dictionary of additional DataFrames to save
assignedTargetingOptions_dataframes_dict = {
    'dataFrame_Partners_List': dataFrame_Partners_List,
    'dataFrame_Advertisers_List': dataFrame_Advertisers_List,
    'dataFrame_Campaigns_List': dataFrame_Campaigns_List,
    'dataFrame_InsertionOrders_List': dataFrame_InsertionOrders_List,
    'dataFrame_LineItems_List': dataFrame_LineItems_List,
    'dataFrame_AdGroups_List': dataFrame_AdGroups_List,
    'id_Name_table': id_Name_table
}

# Save each DataFrame as a CSV file in the specified directory
for df_name, df in assignedTargetingOptions_dataframes_dict.items():
    file_path = os.path.join(data_directory, f'{df_name}.csv')
    df.to_csv(file_path, index=False)
    print(f"Saved {df_name} to {file_path}")

Saved dataFrame_Partners_List to 3. Assigned Targeting Options/dataFrame_Partners_List.csv
Saved dataFrame_Advertisers_List to 3. Assigned Targeting Options/dataFrame_Advertisers_List.csv
Saved dataFrame_Campaigns_List to 3. Assigned Targeting Options/dataFrame_Campaigns_List.csv
Saved dataFrame_InsertionOrders_List to 3. Assigned Targeting Options/dataFrame_InsertionOrders_List.csv
Saved dataFrame_LineItems_List to 3. Assigned Targeting Options/dataFrame_LineItems_List.csv
Saved dataFrame_AdGroups_List to 3. Assigned Targeting Options/dataFrame_AdGroups_List.csv
Saved id_Name_table to 3. Assigned Targeting Options/id_Name_table.csv


In [54]:
## See the value of one cell in a dataframe
## (Used to check some anomalies when reviewing GeoTargeting with multiple PostCode values)
## ISSUE PREVIOUSLY - Cells in CSVs have a limit, large postcode cells broke CSV formatting

# geo_region_value = dataFrame_InsertionOrder_AssignedTargetingOptions.loc[
#     dataFrame_InsertionOrder_AssignedTargetingOptions['insertionOrderId'] == '1009532422',
#     'TARGETING_TYPE_OMID'
# ].values

# print(geo_region_value)


# Combined Heirachy list and Assigned Targeting Options

## Partner

In [55]:
# Perform the merge
dataFrame_Partner = pd.merge(dataFrame_Partners_List, dataFrame_Partner_AssignedTargetingOptions, on='partnerId', how='inner')

# # Display the merged DataFrame
# print(dataFrame_Partner)

## Advertiser

In [56]:
# Perform the merge
dataFrame_Advertiser = pd.merge(dataFrame_Advertisers_List, dataFrame_Advertiser_AssignedTargetingOptions, on='advertiserId', how='inner')

# # Display the merged DataFrame
# print(dataFrame_Advertiser)

## Campaign

In [138]:
# Perform the merge with suffixes
dataFrame_Campaign = pd.merge(
    dataFrame_Campaigns_List,
    dataFrame_Campaign_AssignedTargetingOptions,
    on='campaignId',
    how='inner',
    suffixes=('', '_duplicate')  # Distinguish duplicates
)

# Drop columns with '_duplicate' suffix
dataFrame_Campaign = dataFrame_Campaign[[col for col in dataFrame_Campaign.columns if not col.endswith('_duplicate')]]

# # Display the cleaned merged DataFrame
# print(dataFrame_Campaign)

In [139]:
# # Perform the merge
# dataFrame_Campaign = pd.merge(dataFrame_Campaigns_List, dataFrame_Campaign_AssignedTargetingOptions, on='campaignId', how='inner')

# # Display the merged DataFrame
# print(dataFrame_Campaign)

## Insertion Order

In [140]:
# Perform the merge with suffixes
dataFrame_InsertionOrder = pd.merge(
    dataFrame_InsertionOrders_List,
    dataFrame_InsertionOrder_AssignedTargetingOptions,
    on='insertionOrderId',
    how='inner',
    suffixes=('', '_duplicate')  # Distinguish duplicates
)

# Drop columns with '_duplicate' suffix
dataFrame_InsertionOrder = dataFrame_InsertionOrder[[col for col in dataFrame_InsertionOrder.columns if not col.endswith('_duplicate')]]

# # Display the cleaned merged DataFrame
# print(dataFrame_InsertionOrder)

In [141]:
# # Perform the merge
# dataFrame_InsertionOrder = pd.merge(dataFrame_InsertionOrders_List, dataFrame_InsertionOrder_AssignedTargetingOptions, on='insertionOrderId', how='inner')

# # Display the merged DataFrame
# print(dataFrame_InsertionOrder)

## Line Item

In [195]:
# Perform the merge with suffixes
dataFrame_LineItem = pd.merge(
    dataFrame_LineItems_List,
    dataFrame_LineItem_AssignedTargetingOptions,
    on='lineItemId',
    how='inner',
    suffixes=('', '_duplicate')  # Distinguish duplicates
)

# Drop columns with '_duplicate' suffix
dataFrame_LineItem = dataFrame_LineItem[[col for col in dataFrame_LineItem.columns if not col.endswith('_duplicate')]]

# # Display the cleaned merged DataFrame
# print(dataFrame_LineItem)

In [143]:
# # Perform the merge
# dataFrame_LineItem = pd.merge(dataFrame_LineItem_List, dataFrame_LineItem_AssignedTargetingOptions, on='lineItemId', how='inner')

# # Display the merged DataFrame
# print(dataFrame_LineItem)

## Ad Group

In [144]:
# Perform the merge with suffixes
dataFrame_AdGroup = pd.merge(
    dataFrame_AdGroups_List,
    dataFrame_AdGroup_AssignedTargetingOptions,
    on='adGroupId',
    how='inner',
    suffixes=('', '_duplicate')  # Distinguish duplicates
)

# Drop columns with '_duplicate' suffix
dataFrame_AdGroup = dataFrame_AdGroup[[col for col in dataFrame_AdGroup.columns if not col.endswith('_duplicate')]]

# # Display the cleaned merged DataFrame
# print(dataFrame_AdGroup)

## Data & Directories - Turn all list dataFrames into a CSV

In [145]:
# Define the directory to save the CSV files
data_directory = '4. Full Heirachy Details'

# Create the directory if it doesn't exist
if not os.path.exists(data_directory):
    os.makedirs(data_directory)

# Dictionary of additional DataFrames to save
combinedSettings_dataframes_dict = {
    'dataFrame_Partner': dataFrame_Partner,
    'dataFrame_Advertiser': dataFrame_Advertiser,
    'dataFrame_Campaign': dataFrame_Campaign,
    'dataFrame_InsertionOrder': dataFrame_InsertionOrder,
    'dataFrame_LineItem': dataFrame_LineItem,
    'dataFrame_AdGroup': dataFrame_AdGroup
}

# Save each DataFrame as a CSV file in the specified directory
for df_name, df in combinedSettings_dataframes_dict.items():
    file_path = os.path.join(data_directory, f'{df_name}.csv')
    df.to_csv(file_path, index=False)
    print(f"Saved {df_name} to {file_path}")

Saved dataFrame_Partner to 4. Full Heirachy Details/dataFrame_Partner.csv
Saved dataFrame_Advertiser to 4. Full Heirachy Details/dataFrame_Advertiser.csv
Saved dataFrame_Campaign to 4. Full Heirachy Details/dataFrame_Campaign.csv
Saved dataFrame_InsertionOrder to 4. Full Heirachy Details/dataFrame_InsertionOrder.csv
Saved dataFrame_LineItem to 4. Full Heirachy Details/dataFrame_LineItem.csv
Saved dataFrame_AdGroup to 4. Full Heirachy Details/dataFrame_AdGroup.csv


**SORTED TO THIS POINT - CHECK TODO FOR NOTES ON ADAPTATIONS**

# Parameters

## ageRangeDetails - TARGETING_TYPE_AGE_RANGE

In [116]:
# # # Access the DataFrame for 'TARGETING_TYPE_AGE_RANGE'
# # targetingDF_AgeRange = targetingType_dataframes_dict.get('dataframe_TARGETING_TYPE_AGE_RANGE')

# # # Verify the contents of the DataFrame
# # print(targetingDF_AgeRange)

# parameter_TARGETING_TYPE_AGE_RANGE = [['503002', 'AGE_RANGE_25_34'], ['503001', 'AGE_RANGE_18_24']]

# parameter_TARGETING_TYPE_AGE_RANGE =  sorted(parameter_TARGETING_TYPE_AGE_RANGE, key=lambda x: int(x[0]))
# # parameter_TARGETING_TYPE_AGE_RANGE

                                                name targetingOptionId  \
0  targetingTypes/TARGETING_TYPE_AGE_RANGE/target...            503001   
1  targetingTypes/TARGETING_TYPE_AGE_RANGE/target...            503002   
2  targetingTypes/TARGETING_TYPE_AGE_RANGE/target...            503003   
3  targetingTypes/TARGETING_TYPE_AGE_RANGE/target...            503004   
4  targetingTypes/TARGETING_TYPE_AGE_RANGE/target...            503005   
5  targetingTypes/TARGETING_TYPE_AGE_RANGE/target...            503006   
6  targetingTypes/TARGETING_TYPE_AGE_RANGE/target...            503999   

              targetingType ageRangeDetails.ageRange  
0  TARGETING_TYPE_AGE_RANGE          AGE_RANGE_18_24  
1  TARGETING_TYPE_AGE_RANGE          AGE_RANGE_25_34  
2  TARGETING_TYPE_AGE_RANGE          AGE_RANGE_35_44  
3  TARGETING_TYPE_AGE_RANGE          AGE_RANGE_45_54  
4  TARGETING_TYPE_AGE_RANGE          AGE_RANGE_55_64  
5  TARGETING_TYPE_AGE_RANGE        AGE_RANGE_65_PLUS  
6  TARGETING_TYPE_AGE_

[['503001', 'AGE_RANGE_18_24'], ['503002', 'AGE_RANGE_25_34']]

In [186]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display

# Access the DataFrame for 'TARGETING_TYPE_AGE_RANGE'
df_age_range = targetingType_dataframes_dict.get('dataframe_TARGETING_TYPE_AGE_RANGE')

# Initialize the parameter variable
parameter_TARGETING_TYPE_AGE_RANGE = []

# Check if the DataFrame exists
if df_age_range is not None:
    # Display the DataFrame for reference
    print("Available Age Range Options:")
    display(df_age_range[['targetingOptionId', 'ageRangeDetails.ageRange']])

    # Radio button for selection type
    selection_type_radio = widgets.RadioButtons(
        options=['Not Required', 'Select Age'],
        description='Selection Type:',
        disabled=False
    )

    # List of age ranges with a placeholder "No Option Selected"
    full_age_ranges = df_age_range['ageRangeDetails.ageRange'].tolist()

    # Selection widget with all age ranges
    selection_widget = widgets.SelectMultiple(
        options=full_age_ranges,
        description="Select Age Ranges:",
        disabled=True  # Initially disabled until "Select Age" is chosen
    )

    # Search text box
    search_box = widgets.Text(
        placeholder='Type to search age ranges',
        description='Search:',
        disabled=True  # Initially disabled until "Select Age" is chosen
    )

    # Function to filter options based on search query
    def filter_options(change):
        search_query = change['new'].lower()
        filtered_options = [option for option in full_age_ranges if search_query in option.lower()]
        selection_widget.options = filtered_options

    # Update the selection widget options as the user types in the search box
    search_box.observe(filter_options, names='value')

    # Toggle enabling/disabling of widgets based on radio selection
    def on_radio_change(change):
        if change['new'] == 'Select Age':
            selection_widget.disabled = False
            search_box.disabled = False
        else:
            selection_widget.disabled = True
            search_box.disabled = True
            selection_widget.value = []  # Clear selection if "Not Required" is chosen
            parameter_TARGETING_TYPE_AGE_RANGE.clear()
            print("No options selected.")

    # Observe changes in the radio button
    selection_type_radio.observe(on_radio_change, names='value')

    # Display the widgets
    display(selection_type_radio, search_box, selection_widget)

    # Button to capture and process the selected options
    def on_button_click(b):
        global parameter_TARGETING_TYPE_AGE_RANGE
        selected_ranges = list(selection_widget.value)

        if selection_type_radio.value == 'Not Required':
            parameter_TARGETING_TYPE_AGE_RANGE = []
            print("No options selected.")
        elif selected_ranges:
            # Filter the DataFrame to only include selected age ranges, then sort by targetingOptionId
            selected_df = df_age_range[
                df_age_range['ageRangeDetails.ageRange'].isin(selected_ranges)
            ][['targetingOptionId', 'ageRangeDetails.ageRange']].sort_values(by='targetingOptionId')

            # Convert to list of lists
            parameter_TARGETING_TYPE_AGE_RANGE = selected_df.values.tolist()

            # Display the result
            print("Selected Targeting Options:")
            print(parameter_TARGETING_TYPE_AGE_RANGE)
        else:
            parameter_TARGETING_TYPE_AGE_RANGE = []
            print("No options selected.")

    # Button widget
    button = widgets.Button(description="Submit Selection")
    button.on_click(on_button_click)

    # Display the button
    display(button)
else:
    print("DataFrame for TARGETING_TYPE_AGE_RANGE not found.")


Available Age Range Options:


,targetingOptionId,ageRangeDetails.ageRange
0,503001,AGE_RANGE_18_24
1,503002,AGE_RANGE_25_34
2,503003,AGE_RANGE_35_44
3,503004,AGE_RANGE_45_54
4,503005,AGE_RANGE_55_64
5,503006,AGE_RANGE_65_PLUS
6,503999,AGE_RANGE_UNKNOWN


RadioButtons(description='Selection Type:', options=('Not Required', 'Select Age'), value='Not Required')

Text(value='', description='Search:', disabled=True, placeholder='Type to search age ranges')

SelectMultiple(description='Select Age Ranges:', disabled=True, options=('AGE_RANGE_18_24', 'AGE_RANGE_25_34',…

Button(description='Submit Selection', style=ButtonStyle())

Selected Targeting Options:
[['503001', 'AGE_RANGE_18_24'], ['503002', 'AGE_RANGE_25_34'], ['503003', 'AGE_RANGE_35_44'], ['503004', 'AGE_RANGE_45_54']]


## digitalContentLabelDetails - TARGETING_TYPE_DIGITAL_CONTENT_LABEL_EXCLUSION

In [ ]:
# # Client parameter(s) to check
# parameter_Advertiser_TARGETING_TYPE_DIGITAL_CONTENT_LABEL_EXCLUSION = ['CONTENT_RATING_TIER_UNRATED', 'CONTENT_RATING_TIER_TEENS', 'CONTENT_RATING_TIER_MATURE']
# parameter_Advertiser_TARGETING_TYPE_DIGITAL_CONTENT_LABEL_EXCLUSION.sort()

In [129]:
# Access the DataFrame for 'TARGETING_TYPE_DIGITAL_CONTENT_LABEL_EXCLUSION'
df_content_label_exclusion = targetingType_dataframes_dict.get('dataframe_TARGETING_TYPE_DIGITAL_CONTENT_LABEL_EXCLUSION')

# Initialize the parameter variable
parameter_TARGETING_TYPE_DIGITAL_CONTENT_LABEL_EXCLUSION = []

# Check if the DataFrame exists
if df_content_label_exclusion is not None:
    # Display the DataFrame for reference
    print("Available Digital Content Label Exclusion Options:")
    display(df_content_label_exclusion[['targetingOptionId', 'digitalContentLabelDetails.contentRatingTier']])

    # Radio button for selection type
    selection_type_radio = widgets.RadioButtons(
        options=['Not Required', 'Select Digital Content Label Exclusion'],
        description='Selection Type:',
        disabled=False
    )

    # List of content label exclusions with a placeholder "No Option Selected"
    full_content_labels = df_content_label_exclusion['digitalContentLabelDetails.contentRatingTier'].tolist()

    # Selection widget with all content label exclusions
    selection_widget = widgets.SelectMultiple(
        options=full_content_labels,
        description="Select Content Labels:",
        disabled=True  # Initially disabled until "Select Digital Content Label Exclusion" is chosen
    )

    # Search text box
    search_box = widgets.Text(
        placeholder='Type to search content labels',
        description='Search:',
        disabled=True  # Initially disabled until "Select Digital Content Label Exclusion" is chosen
    )

    # Function to filter options based on search query
    def filter_options(change):
        search_query = change['new'].lower()
        filtered_options = [option for option in full_content_labels if search_query in option.lower()]
        selection_widget.options = filtered_options

    # Update the selection widget options as the user types in the search box
    search_box.observe(filter_options, names='value')

    # Toggle enabling/disabling of widgets based on radio selection
    def on_radio_change(change):
        if change['new'] == 'Select Digital Content Label Exclusion':
            selection_widget.disabled = False
            search_box.disabled = False
        else:
            selection_widget.disabled = True
            search_box.disabled = True
            selection_widget.value = []  # Clear selection if "Not Required" is chosen
            parameter_TARGETING_TYPE_DIGITAL_CONTENT_LABEL_EXCLUSION.clear()
            print("No options selected.")

    # Observe changes in the radio button
    selection_type_radio.observe(on_radio_change, names='value')

    # Display the widgets
    display(selection_type_radio, search_box, selection_widget)

    # Button to capture and process the selected options
    def on_button_click(b):
        global parameter_TARGETING_TYPE_DIGITAL_CONTENT_LABEL_EXCLUSION
        selected_labels = list(selection_widget.value)

        # Check if "Not Required" is chosen, ignoring other selections if so
        if selection_type_radio.value == 'Not Required':
            parameter_TARGETING_TYPE_DIGITAL_CONTENT_LABEL_EXCLUSION = []
            print("No options selected.")
        elif selected_labels:
            # Filter the DataFrame to only include selected content labels, then sort by targetingOptionId
            selected_df = df_content_label_exclusion[
                df_content_label_exclusion['digitalContentLabelDetails.contentRatingTier'].isin(selected_labels)
            ][['targetingOptionId', 'digitalContentLabelDetails.contentRatingTier']].sort_values(by='targetingOptionId')

            # Convert to list of lists
            parameter_TARGETING_TYPE_DIGITAL_CONTENT_LABEL_EXCLUSION = selected_df.values.tolist()

            # Display the result
            print("Selected Targeting Options:")
            print(parameter_TARGETING_TYPE_DIGITAL_CONTENT_LABEL_EXCLUSION)
        else:
            parameter_TARGETING_TYPE_DIGITAL_CONTENT_LABEL_EXCLUSION = []
            print("No options selected.")

    # Button widget
    button = widgets.Button(description="Submit Selection")
    button.on_click(on_button_click)

    # Display the button
    display(button)
else:
    print("DataFrame for TARGETING_TYPE_DIGITAL_CONTENT_LABEL_EXCLUSION not found.")


Available Digital Content Label Exclusion Options:


,targetingOptionId,digitalContentLabelDetails.contentRatingTier
0,19875633960,CONTENT_RATING_TIER_GENERAL
1,19875634080,CONTENT_RATING_TIER_PARENTAL_GUIDANCE
2,19875634200,CONTENT_RATING_TIER_TEENS
3,19875634320,CONTENT_RATING_TIER_MATURE
4,19875634440,CONTENT_RATING_TIER_UNRATED
5,89504691659,CONTENT_RATING_TIER_FAMILIES


RadioButtons(description='Selection Type:', options=('Not Required', 'Select Digital Content Label Exclusion')…

Text(value='', description='Search:', disabled=True, placeholder='Type to search content labels')

SelectMultiple(description='Select Content Labels:', disabled=True, options=('CONTENT_RATING_TIER_GENERAL', 'C…

Button(description='Submit Selection', style=ButtonStyle())

Selected Targeting Options:
[['19875633960', 'CONTENT_RATING_TIER_GENERAL'], ['19875634080', 'CONTENT_RATING_TIER_PARENTAL_GUIDANCE'], ['19875634200', 'CONTENT_RATING_TIER_TEENS'], ['19875634320', 'CONTENT_RATING_TIER_MATURE'], ['89504691659', 'CONTENT_RATING_TIER_FAMILIES']]


## geoRegionDetails - TARGETING_TYPE_GEO_REGION


**SOMETHING TO EDIT. All Cannonical values have no space between words and comma.**

Eg. 'Wales,United Kingdom'
<br>
Downloaded values have a space,
<br>
Eg. 'Wales, United Kingdom'

In [146]:
# User defined Paramters
parameter_Advertiser_TARGETING_TYPE_GEO_REGION_Include = ['2826']
parameter_Advertiser_TARGETING_TYPE_GEO_REGION_Exclude = ['20342', '20343']
parameter_Advertiser_TARGETING_TYPE_GEO_REGION_IncludeAndExclude = parameter_Advertiser_TARGETING_TYPE_GEO_REGION_Include + parameter_Advertiser_TARGETING_TYPE_GEO_REGION_Exclude

In [147]:
# Latest CSV sheet of GeoIDs
geotargets_df = pd.read_csv("/content/geotargets-2023-08-29.csv")

# Lookup table for Target Type
target_type_lookup = {
    'Airport': 'GEO_REGION_TYPE_AIRPORT',
    'Autonomous Community': 'GEO_REGION_TYPE_AUTONOMOUS_COMMUNITY',
    'Borough': 'GEO_REGION_TYPE_BOROUGH',
    'Canton': 'GEO_REGION_TYPE_CANTON',
    'City': 'GEO_REGION_TYPE_CITY',
    'City Region': 'GEO_REGION_TYPE_CITY_REGION',
    'Congressional District': 'GEO_REGION_TYPE_CONGRESSIONAL_DISTRICT',
    'Country': 'GEO_REGION_TYPE_COUNTRY',
    'County': 'GEO_REGION_TYPE_COUNTY',
    'Department': 'GEO_REGION_TYPE_DEPARTMENT',
    'District': 'GEO_REGION_TYPE_DISTRICT',
    'Governorate': 'GEO_REGION_TYPE_GOVERNORATE',
    'Municipality': 'GEO_REGION_TYPE_MUNICIPALITY',
    'National Park': 'GEO_REGION_TYPE_NONE',
    'Neighborhood': 'GEO_REGION_TYPE_NEIGHBORHOOD',
    'Okrug': 'GEO_REGION_TYPE_OKRUG',
    'Postal Code': 'GEO_REGION_TYPE_POSTAL_CODE',
    'Prefecture': 'GEO_REGION_TYPE_PREFECTURE',
    'Province': 'GEO_REGION_TYPE_PROVINCE',
    'Region': 'GEO_REGION_TYPE_REGION',
    'State': 'GEO_REGION_TYPE_STATE',
    'Territory': 'GEO_REGION_TYPE_TERRITORY',
    'TV Region': 'GEO_REGION_TYPE_TV_REGION',
    'Union Territory': 'GEO_REGION_TYPE_UNION_TERRITORY',
    'University': 'GEO_REGION_TYPE_UNIVERSITY'
}

# Modified function to handle include and exclude with proper negatives_value
def search_geotargets(geotarget_ids, dataframe, negatives_value=False):
    try:
        # If geotarget_ids is empty, return an empty result
        if not geotarget_ids:
            return []

        # Convert geotarget_ids to string for consistent comparison and sorting
        geotarget_ids = [str(id) for id in geotarget_ids]

        # Ensure Criteria ID in the dataframe is treated as strings for consistent comparison
        dataframe['Criteria ID'] = dataframe['Criteria ID'].astype(str)

        # Filter the dataframe based on criteria ids
        filtered_df = dataframe[dataframe['Criteria ID'].isin(geotarget_ids)]

        # Check if any criteria ids were not found
        found_ids = filtered_df['Criteria ID'].tolist()
        not_found_ids = set(geotarget_ids) - set(found_ids)

        if not_found_ids:
            raise ValueError(f"The following Criteria IDs were not found: {', '.join(map(str, not_found_ids))}")

        # Extract and format the required values
        results = []
        for _, row in filtered_df.iterrows():
            criteria_id = row['Criteria ID']  # Keep it as string
            name = row['Canonical Name'].replace(',', ', ')  # Ensure there's a space after commas
            geo_type = target_type_lookup.get(row['Target Type'], 'UNKNOWN')
            negative = negatives_value

            # Create the formatted row
            formatted_row = [criteria_id, name, criteria_id, geo_type, negative]
            results.append(formatted_row)

        # Sort the results by the 'Criteria ID' (first item in each sublist) as strings but in numeric order
        results = sorted(results, key=lambda x: int(x[0]))

        return results

    except Exception as e:
        # Log the error and return an empty list
        print(f"Error occurred: {e}")
        return []

# Get Include and Exclude results separately
formattedparameter_Advertiser_TARGETING_TYPE_GEO_REGION_Include = search_geotargets(parameter_Advertiser_TARGETING_TYPE_GEO_REGION_Include, geotargets_df)
formattedparameter_Advertiser_TARGETING_TYPE_GEO_REGION_Exclude = search_geotargets(parameter_Advertiser_TARGETING_TYPE_GEO_REGION_Exclude, geotargets_df, negatives_value=True)

# Combine Include and Exclude, keeping the correct 'negatives_value'
formattedparameter_Advertiser_TARGETING_TYPE_GEO_REGION_IncludeAndExclude = (
    formattedparameter_Advertiser_TARGETING_TYPE_GEO_REGION_Include + formattedparameter_Advertiser_TARGETING_TYPE_GEO_REGION_Exclude
)

# Sort the combined list by 'Criteria ID'
formattedparameter_Advertiser_TARGETING_TYPE_GEO_REGION_IncludeAndExclude = sorted(
    formattedparameter_Advertiser_TARGETING_TYPE_GEO_REGION_IncludeAndExclude, key=lambda x: str(x[0])
)

# Output the final results
print("Include:", formattedparameter_Advertiser_TARGETING_TYPE_GEO_REGION_Include)
print("Exclude:", formattedparameter_Advertiser_TARGETING_TYPE_GEO_REGION_Exclude)
print("Combined and Sorted:", formattedparameter_Advertiser_TARGETING_TYPE_GEO_REGION_IncludeAndExclude)


Include: [['2826', 'United Kingdom', '2826', 'GEO_REGION_TYPE_COUNTRY', False]]
Exclude: [['20342', 'Scotland, United Kingdom', '20342', 'GEO_REGION_TYPE_PROVINCE', True], ['20343', 'Wales, United Kingdom', '20343', 'GEO_REGION_TYPE_PROVINCE', True]]
Combined and Sorted: [['20342', 'Scotland, United Kingdom', '20342', 'GEO_REGION_TYPE_PROVINCE', True], ['20343', 'Wales, United Kingdom', '20343', 'GEO_REGION_TYPE_PROVINCE', True], ['2826', 'United Kingdom', '2826', 'GEO_REGION_TYPE_COUNTRY', False]]


## Check Parameters

In [130]:
print(parameter_TARGETING_TYPE_AGE_RANGE)
print(parameter_TARGETING_TYPE_DIGITAL_CONTENT_LABEL_EXCLUSION)

[['503001', 'AGE_RANGE_18_24'], ['503002', 'AGE_RANGE_25_34'], ['503003', 'AGE_RANGE_35_44'], ['503004', 'AGE_RANGE_45_54'], ['503005', 'AGE_RANGE_55_64'], ['503006', 'AGE_RANGE_65_PLUS']]
[['19875633960', 'CONTENT_RATING_TIER_GENERAL'], ['19875634080', 'CONTENT_RATING_TIER_PARENTAL_GUIDANCE'], ['19875634200', 'CONTENT_RATING_TIER_TEENS'], ['19875634320', 'CONTENT_RATING_TIER_MATURE'], ['89504691659', 'CONTENT_RATING_TIER_FAMILIES']]


# Checks

In [ ]:
# dataFrame_LineItem['TARGETING_TYPE_GEO_REGION']

## Functions for checks

In [149]:
## Testing1
## ISSUES WITH THE LIST OF LISTS, WILL NEED REVIEWING

def process_row(row, checkName, column_name, QA, checkFunction, level_field):
    # Ensure we're safely extracting the value
    value = row.get(column_name, [])

    # Remove any empty sublists
    value = [v for v in value if v]

    if not isinstance(value, list):
        value = []

    result = checkFunction(value, QA)

    new_row_dict = {
        'IdList': row[level_field],
        'CheckName': checkName,
        'DownloadedValue': value,
        'CorrectValue': QA,
        'CheckResult': bool(result)
    }
    return new_row_dict

def custom_equal(a, b):
    """Compare two values, treating strings that can be cast to integers as integers."""
    try:
        return int(a) == int(b)
    except (TypeError, ValueError):
        return a == b

# Using the custom_equal function for this test
def check_nested_lists(x, y):
    if not isinstance(x, list) or not isinstance(y, list):
        return False
    if len(x) != len(y):
        return False
    for sublist_x, sublist_y in zip(x, y):
        if not all(custom_equal(xi, yi) for xi, yi in zip(sublist_x, sublist_y)):
            return False
    return True

def list_of_lists_difference(main_list, sub_list):
    """Calculate the difference between two lists of lists."""
    return [item for item in main_list if item not in sub_list]

def checkvalues_exact(df, checkName, column_name, QA, CheckResult, level_field):
    try:
        if not isinstance(df, pd.DataFrame) or df.empty:
            print("Error: Provided dataframe is not valid in checkvalues_exact.")
            return CheckResult

        rows_list = []
        for _, row in df.iterrows():
            downloaded_value = row.get(column_name, None)
            correct_value = QA if isinstance(QA, list) else []

            # Handle None values
            if downloaded_value is None:
                print(f"None value found in column {column_name} for row with {level_field}: {row[level_field]}. Marking as Incorrect.")
                value_create = correct_value
                value_delete = None
                check_result = False
            else:
                # Initialize ValueCreate and ValueDelete as empty lists
                value_create = []
                value_delete = []

                # Find values to create (elements in CorrectValue but not in DownloadedValue)
                for cv_sublist, dv_sublist in zip_longest(correct_value, downloaded_value, fillvalue=[]):
                    cv_set = set(cv_sublist)
                    dv_set = set(dv_sublist)
                    if not cv_set.issubset(dv_set):
                        value_create.append(list(cv_set - dv_set))

                # Find values to delete (elements in DownloadedValue but not in CorrectValue)
                for dv_sublist, cv_sublist in zip_longest(downloaded_value, correct_value, fillvalue=[]):
                    dv_set = set(dv_sublist)
                    cv_set = set(cv_sublist)
                    if not dv_set.issubset(cv_set):
                        value_delete.append(list(dv_set - cv_set))

                # Ensure empty lists are not added
                value_create = [vc for vc in value_create if vc]
                value_delete = [vd for vd in value_delete if vd]

                # Check result
                check_result = check_nested_lists(downloaded_value, correct_value)

            # Process each row and include ValueCreate and ValueDelete
            new_row_dict = {
                'IdList': row[level_field],
                'CheckName': checkName,
                'DownloadedValue': downloaded_value,
                'CorrectValue': correct_value,
                'CheckResult': check_result,
                'ValueCreate': value_create,
                'ValueDelete': value_delete
            }
            rows_list.append(new_row_dict)

        new_df = pd.DataFrame(rows_list)
        CheckResult = pd.concat([CheckResult, new_df], ignore_index=True)
        if 'CheckResult' in CheckResult.columns:
            CheckResult['CheckResult'] = CheckResult['CheckResult'].astype(bool)
        return CheckResult

    except Exception as e:
        print(f"An error occurred in checkvalues_exact: {e}")
        return CheckResult


def checkvalues_contained(df, checkName, column_name, QA, CheckResult, level_field):
    try:
        if not isinstance(df, pd.DataFrame) or df.empty:
            print("Error: Provided dataframe is not valid in checkvalues_contained.")
            return CheckResult

        def is_contained(x, y):
            if not x and not y:
                return True
            if not x:
                return False
            if not isinstance(x, list):
                return False
            return all(value in y for value in x)

        rows_list = []
        for _, row in df.iterrows():
            result = is_contained(row.get(column_name, []), QA)

            # Determine ValueCreate and ValueDelete based on CheckResult
            if not result:  # If CheckResult is False
                value_create = [QA[-1]] if QA else []  # Last value in QA
                value_delete = row.get(column_name, [])  # All values in column_name
            else:
                value_create = []
                value_delete = []

            # Process each row and include ValueCreate and ValueDelete
            processed_row = process_row(row, checkName, column_name, QA, is_contained, level_field)
            processed_row['ValueCreate'] = value_create
            processed_row['ValueDelete'] = value_delete
            rows_list.append(processed_row)

        new_df = pd.DataFrame(rows_list)
        CheckResult = pd.concat([CheckResult, new_df], ignore_index=True)
        if 'CheckResult' in CheckResult.columns:
            CheckResult['CheckResult'] = CheckResult['CheckResult'].astype(bool)
        return CheckResult

    except Exception as e:
        print(f"An error occurred in checkvalues_contained: {e}")
        return CheckResult

def checkvalues_atleast(df, checkName, column_name, QA, CheckResult, level_field):
    try:
        if not isinstance(df, pd.DataFrame) or df.empty:
            print("Error: Provided dataframe is not valid in checkvalues_atleast.")
            return CheckResult

        rows_list = []
        for _, row in df.iterrows():
            # Calculate ValueCreate
            value = row.get(column_name, [])
            value_create = [item for item in QA if item not in value]

            # Process each row and include ValueCreate (ValueDelete is left empty)
            processed_row = process_row(row, checkName, column_name, QA, lambda x, y: all(value in x for value in y), level_field)
            processed_row['ValueCreate'] = value_create
            processed_row['ValueDelete'] = []  # ValueDelete is empty

            rows_list.append(processed_row)

        new_df = pd.DataFrame(rows_list)
        CheckResult = pd.concat([CheckResult, new_df], ignore_index=True)
        if 'CheckResult' in CheckResult.columns:
            CheckResult['CheckResult'] = CheckResult['CheckResult'].astype(bool)
        return CheckResult

    except Exception as e:
        print(f"An error occurred in checkvalues_atleast: {e}")
        return CheckResult

## Testing (WORKING) Functions - Updated:
- checkvalues_exact
- checkvalues_contained

### Live

In [209]:
from itertools import zip_longest
import pandas as pd

def checkvalues_exact(df, checkName, column_name, QA, CheckResult, level_field):
    try:
        if not isinstance(df, pd.DataFrame) or df.empty:
            print("Error: Provided dataframe is not valid in checkvalues_exact.")
            return CheckResult

        rows_list = []
        for _, row in df.iterrows():
            downloaded_value = row.get(column_name, None)
            correct_value = QA if isinstance(QA, list) else []

            # Handle None values
            if downloaded_value is None:
                value_create = correct_value
                value_delete = None
                check_result = False
            else:
                # Identify lists to create and delete
                value_create = [cv for cv in correct_value if cv not in downloaded_value]
                value_delete = [dv for dv in downloaded_value if dv not in correct_value]

                # Remove any empty lists from ValueCreate and ValueDelete
                value_create = [vc for vc in value_create if vc]
                value_delete = [vd for vd in value_delete if vd]

                # Set to None if lists are empty after filtering
                if not value_create:
                    value_create = None
                if not value_delete:
                    value_delete = None

                # Check result
                check_result = (value_create is None and value_delete is None)

            # Process each row and include ValueCreate and ValueDelete
            new_row_dict = {
                'IdList': row[level_field],
                'CheckName': checkName,
                'DownloadedValue': downloaded_value,
                'CorrectValue': correct_value,
                'CheckResult': check_result,
                'ValueCreate': value_create,
                'ValueDelete': value_delete
            }
            rows_list.append(new_row_dict)

        new_df = pd.DataFrame(rows_list)
        CheckResult = pd.concat([CheckResult, new_df], ignore_index=True)
        if 'CheckResult' in CheckResult.columns:
            CheckResult['CheckResult'] = CheckResult['CheckResult'].astype(bool)
        return CheckResult

    except Exception as e:
        print(f"An error occurred in checkvalues_exact: {e}")
        return CheckResult

## Testing V2

def checkvalues_contained(df, checkName, column_name, QA, CheckResult, level_field):
    try:
        if not isinstance(df, pd.DataFrame) or df.empty:
            print("Error: Provided dataframe is not valid in checkvalues_contained.")
            return CheckResult

        rows_list = []
        for _, row in df.iterrows():
            downloaded_value = row.get(column_name, [])
            downloaded_value = downloaded_value if isinstance(downloaded_value, list) else []

            # Determine ValueDelete as items in DownloadedValue that are not in CorrectValue
            value_delete = [item for item in downloaded_value if item not in QA]

            # Calculate the remaining DownloadedValue after removing ValueDelete
            remaining_downloaded = [item for item in downloaded_value if item not in value_delete]

            # Determine if remaining DownloadedValue is contained within CorrectValue
            is_fully_contained = all(item in QA for item in remaining_downloaded)

            # Determine ValueCreate based on the given conditions
            if not downloaded_value:  # If DownloadedValue is empty
                value_create = QA
            elif is_fully_contained:  # If remaining values are fully contained within CorrectValue
                value_create = None
            else:  # If some values are missing
                value_create = [item for item in QA if item not in downloaded_value]

            # Remove empty lists if they exist
            value_create = value_create if value_create else None
            value_delete = value_delete if value_delete else None

            # Check result
            check_result = (value_create is None and value_delete is None)

            # Process each row and include ValueCreate and ValueDelete
            processed_row = {
                'IdList': row[level_field],
                'CheckName': checkName,
                'DownloadedValue': downloaded_value,
                'CorrectValue': QA,
                'CheckResult': check_result,
                'ValueCreate': value_create,
                'ValueDelete': value_delete
            }
            rows_list.append(processed_row)

        new_df = pd.DataFrame(rows_list)
        CheckResult = pd.concat([CheckResult, new_df], ignore_index=True)
        if 'CheckResult' in CheckResult.columns:
            CheckResult['CheckResult'] = CheckResult['CheckResult'].astype(bool)
        return CheckResult

    except Exception as e:
        print(f"An error occurred in checkvalues_contained: {e}")
        return CheckResult

### Testing

In [ ]:
## Testing V1
# def checkvalues_contained(df, checkName, column_name, QA, CheckResult, level_field):
#     try:
#         if not isinstance(df, pd.DataFrame) or df.empty:
#             print("Error: Provided dataframe is not valid in checkvalues_contained.")
#             return CheckResult

#         def is_contained(x, y):
#             # Ensure x and y are lists; otherwise, treat them as empty lists
#             x = x if isinstance(x, list) else []
#             y = y if isinstance(y, list) else []
#             return all(value in y for value in x)

#         rows_list = []
#         for _, row in df.iterrows():
#             downloaded_value = row.get(column_name, [])
#             result = is_contained(downloaded_value, QA)

#             # Determine ValueCreate and ValueDelete based on CheckResult
#             if not result:  # If CheckResult is False
#                 value_create = [QA[-1]] if QA else []  # Last value in QA if QA is not empty
#                 value_delete = downloaded_value if downloaded_value else []  # All values in column_name if not empty
#             else:
#                 value_create = []
#                 value_delete = []

#             # Remove empty lists if they exist
#             value_create = value_create if value_create else None
#             value_delete = value_delete if value_delete else None

#             # Process each row and include ValueCreate and ValueDelete
#             processed_row = {
#                 'IdList': row[level_field],
#                 'CheckName': checkName,
#                 'DownloadedValue': downloaded_value,
#                 'CorrectValue': QA,
#                 'CheckResult': result,
#                 'ValueCreate': value_create,
#                 'ValueDelete': value_delete
#             }
#             rows_list.append(processed_row)

#         new_df = pd.DataFrame(rows_list)
#         CheckResult = pd.concat([CheckResult, new_df], ignore_index=True)
#         if 'CheckResult' in CheckResult.columns:
#             CheckResult['CheckResult'] = CheckResult['CheckResult'].astype(bool)
#         return CheckResult

#     except Exception as e:
#         print(f"An error occurred in checkvalues_contained: {e}")
#         return CheckResult

## Line Item

In [214]:
## Running the checks against the insertionOrder settings
lineItemCheckResult = pd.DataFrame(columns=['IdList',
                                            'CheckName',
                                            'CorrectValue',
                                            'DownloadedValue',
                                            'CheckResult'])
lineItemCheckResult['CheckResult'] = lineItemCheckResult['CheckResult'].astype(bool)

### Exact Settings
# Geo
lineItemCheckResult = checkvalues_exact(dataFrame_LineItem, 'Localisation - Is Geo Targeting set at Line Item level', 'TARGETING_TYPE_GEO_REGION', formattedparameter_Advertiser_TARGETING_TYPE_GEO_REGION_IncludeAndExclude, lineItemCheckResult, 'lineItemId')
### Contained Settings
# Age
lineItemCheckResult = checkvalues_contained(dataFrame_LineItem, 'Demographic - Is age targeting set', 'TARGETING_TYPE_AGE_RANGE', parameter_TARGETING_TYPE_AGE_RANGE, lineItemCheckResult, 'lineItemId')

In [215]:
lineItemCheckResult

,IdList,CheckName,CorrectValue,DownloadedValue,CheckResult,ValueCreate,ValueDelete
0,21712780928,Localisation - Is Geo Targeting set at Line It...,"[[20342, Scotland, United Kingdom, 20342, GEO_...","[[2036, Australia, 2036, GEO_REGION_TYPE_COUNT...",False,"[[20342, Scotland, United Kingdom, 20342, GEO_...","[[2036, Australia, 2036, GEO_REGION_TYPE_COUNT..."
1,21314256910,Localisation - Is Geo Targeting set at Line It...,"[[20342, Scotland, United Kingdom, 20342, GEO_...","[[2036, Australia, 2036, GEO_REGION_TYPE_COUNT...",False,"[[20342, Scotland, United Kingdom, 20342, GEO_...","[[2036, Australia, 2036, GEO_REGION_TYPE_COUNT..."
2,44928966,Localisation - Is Geo Targeting set at Line It...,"[[20342, Scotland, United Kingdom, 20342, GEO_...","[[2036, Australia, 2036, GEO_REGION_TYPE_COUNT...",False,"[[20342, Scotland, United Kingdom, 20342, GEO_...","[[2036, Australia, 2036, GEO_REGION_TYPE_COUNT..."
3,44928970,Localisation - Is Geo Targeting set at Line It...,"[[20342, Scotland, United Kingdom, 20342, GEO_...","[[2036, Australia, 2036, GEO_REGION_TYPE_COUNT...",False,"[[20342, Scotland, United Kingdom, 20342, GEO_...","[[2036, Australia, 2036, GEO_REGION_TYPE_COUNT..."
4,44928968,Localisation - Is Geo Targeting set at Line It...,"[[20342, Scotland, United Kingdom, 20342, GEO_...","[[2036, Australia, 2036, GEO_REGION_TYPE_COUNT...",False,"[[20342, Scotland, United Kingdom, 20342, GEO_...","[[2036, Australia, 2036, GEO_REGION_TYPE_COUNT..."
...,...,...,...,...,...,...,...
131,21286655205,Demographic - Is age targeting set,"[[503001, AGE_RANGE_18_24], [503002, AGE_RANGE...",[],False,"[[503001, AGE_RANGE_18_24], [503002, AGE_RANGE...",None
132,21297076960,Demographic - Is age targeting set,"[[503001, AGE_RANGE_18_24], [503002, AGE_RANGE...",[],False,"[[503001, AGE_RANGE_18_24], [503002, AGE_RANGE...",None
133,21293142155,Demographic - Is age targeting set,"[[503001, AGE_RANGE_18_24], [503002, AGE_RANGE...",[],False,"[[503001, AGE_RANGE_18_24], [503002, AGE_RANGE...",None
134,21178406878,Demographic - Is age targeting set,"[[503001, AGE_RANGE_18_24], [503002, AGE_RANGE...",[],False,"[[503001, AGE_RANGE_18_24], [503002, AGE_RANGE...",None


## Data & Directories - Turn all list dataFrames into a CSV

In [213]:
# Define the directory to save the CSV files
data_directory = '5. Check Results'

# Create the directory if it doesn't exist
if not os.path.exists(data_directory):
    os.makedirs(data_directory)

# Dictionary of DataFrames to save
dataframes_dict = {
    'lineItemCheckResult': lineItemCheckResult
}

# Save each DataFrame as a CSV file in the specified directory
for df_name, df in dataframes_dict.items():
    file_path = os.path.join(data_directory, f'{df_name}.csv')
    df.to_csv(file_path, index=False)
    print(f"Saved {df_name} to {file_path}")

Saved lineItemCheckResult to 5. Check Results/lineItemCheckResult.csv


# **Look to change the CheckResult to include the targeting type that needs to be changed. ALso how the lists of create and delete are processed. **

# Adaptation

In [ ]:
# def build_create_requests_from_mapping(row, targeting_type, targeting_details_mapping):

#     # Get the relevant fields from the mapping
#     relevant_fields = targeting_details_mapping.get(targeting_type, ())

#     # Exclude 'assignedTargetingOptionId', the last two values, and any value containing 'displayName'
#     fields_to_include = [field for field in relevant_fields[:-2]
#                          if 'assignedTargetingOptionId' not in field
#                          and 'displayName' not in field]

#     if not fields_to_include:
#         return []

#     assigned_targeting_options = []
#     for value_set in row:
#         details = {}
#         for field_name, value in zip(fields_to_include, value_set):
#             field_key = '.'.join(field_name.split('.')[:-1])  # Get the parent key (e.g., 'appDetails')
#             sub_key = field_name.split('.')[-1]  # Get the specific subfield (e.g., 'appId')

#             if field_key not in details:
#                 details[field_key] = {}

#             details[field_key][sub_key] = value

#         if details:
#             assigned_targeting_options.append(details)

#     if assigned_targeting_options:
#         return {
#             'targetingType': targeting_type,
#             'assignedTargetingOptions': assigned_targeting_options
#         }
#     return None

# # Example usage in the bulk edit function
# def bulk_edit_targeting(service, advertiser_id, line_item_id, targeting_df, targeting_details_mapping):
#     """
#     Bulk edits the targeting options for a line item based on the provided DataFrame.

#     Args:
#         service: The googleapiclient.discovery.Resource instance used to interact
#             with the Display & Video 360 API.
#         advertiser_id: The ID of the parent advertiser of the line item to which the
#             listed targeting options are assigned.
#         line_item_id: The ID of the line item to which the listed targeting options
#             are assigned.
#         targeting_df: DataFrame containing the targeting types, create, and delete columns.
#         targeting_details_mapping: A dictionary mapping each targeting type to its relevant fields.

#     Returns:
#         None
#     """
#     create_requests = []
#     delete_requests = []

#     for _, row in targeting_df.iterrows():
#         targeting_type = row['Targeting Type']
#         create_values = row['Create']
#         delete_values = row['Delete']

#         # Build the create requests
#         create_request = build_create_requests_from_mapping(create_values, targeting_type, targeting_details_mapping)
#         if create_request:
#             create_requests.append(create_request)

#         # Build delete request if delete values are present
#         if pd.notna(delete_values) and delete_values:
#             delete_request = {
#                 "targetingType": targeting_type,
#                 "assignedTargetingOptionIds": delete_values,
#             }
#             delete_requests.append(delete_request)

#     # Build the bulk edit request body
#     bulk_edit_line_item_request = {
#         "lineItemIds": [line_item_id],
#         "deleteRequests": delete_requests,
#         "createRequests": create_requests,
#     }

#     # Edit the line item targeting
#     response = (
#         service.advertisers()
#         .lineItems()
#         .bulkEditAssignedTargetingOptions(
#             advertiserId=advertiser_id,
#             body=bulk_edit_line_item_request,
#         )
#         .execute()
#     )

#     # If response is not empty, display the updated line items
#     if response:
#         print_bulk_edit_targeting_response(response)
#     else:
#         print("Error: No response was returned.")

# # Example usage:
# # Assume you have a DataFrame `targeting_df` with columns "Targeting Type", "Create", "Delete"
# # And you have an authenticated `service` instance.

# advertiser_id = "YOUR_ADVERTISER_ID"
# line_item_id = "YOUR_LINE_ITEM_ID"

# bulk_edit_targeting(service, advertiser_id, line_item_id, lineItemCheckResult, targeting_details_mapping)


# Testing - Client assignedTargetingOptions

## Functions for assignedTargetingOptions

In [ ]:
## Testing the addition of one larger function to run all levels bar Line Item and Ad Group

# # Function to initialize DataFrame with dynamic columns based on targeting types
# def initialize_targeting_dataframe(entity_id_name, targeting_types_list, details_mapping):
#     data_frame = pd.DataFrame({entity_id_name: []})
#     for targeting_type in targeting_types_list:
#         if targeting_type in details_mapping:
#             data_frame[f'{targeting_type}'] = []
#     return data_frame

# # Function to process targeting details using predefined mapping
# def process_targeting_details(response_df, targeting_type, details_mapping):
#     if targeting_type not in details_mapping:
#         return []

#     targeting_fields = details_mapping[targeting_type]
#     sorting_column_index = targeting_fields[-2] if isinstance(targeting_fields[-2], int) else 0

#     targeting_details = get_targeting_details(
#         response_df, targeting_type, *targeting_fields[:-2]
#     )

#     # Flatten, sort, and format targeting details
#     flattened_details = list(zip(*targeting_details))
#     sorted_details = sort_by_column(flattened_details, sorting_column_index)
#     formatted_details = [[entry[i] for i in range(len(entry))] for entry in sorted_details]

#     return formatted_details

# def collect_targeting_data(ids, service, entity_type, targeting_types_list, details_mapping):
#     data = []
#     for entity_id in ids:
#         for targeting_type in targeting_types_list:
#             try:
#                 # Different API calls based on entity type
#                 if entity_type == 'partner':
#                     request = service.partners().targetingTypes().assignedTargetingOptions().list(
#                         partnerId=entity_id,
#                         targetingType=targeting_type
#                     )
#                 elif entity_type == 'advertiser':
#                     request = service.advertisers().listAssignedTargetingOptions(
#                         advertiserId=entity_id,
#                         targetingType=targeting_type
#                     )
#                 elif entity_type == 'campaign':
#                     request = service.advertisers().campaigns().listAssignedTargetingOptions(
#                         advertiserId=entity_id['advertiserId'],
#                         campaignId=entity_id['campaignId'],
#                         targetingType=targeting_type
#                     )
#                 elif entity_type == 'insertionOrder':
#                     request = service.advertisers().insertionOrders().listAssignedTargetingOptions(
#                         advertiserId=entity_id['advertiserId'],
#                         insertionOrderId=entity_id['insertionOrderId'],
#                         targetingType=targeting_type
#                     )
#                 else:
#                     raise ValueError(f"Unknown entity type: {entity_type}")

#                 response = request.execute()
#                 # Process response data (this part needs to be customized based on your data structure)
#                 data.append(response)  # or use appropriate logic

#             except Exception as e:
#                 print(f"An error occurred for {entity_type} ID {entity_id}: {e}")

#     return pd.DataFrame(data)


## Collect Data

In [ ]:
## Testing the addition of one larger function to run all levels bar Line Item and Ad Group

# # Collect Partner Targeting Data
# dataFrame_Partner_AssignedTargetingOptions = collect_targeting_data(
#     partnerIdList,
#     service,
#     'partner',
#     targeting_types_list=['TARGETING_TYPE_CHANNEL'],
#     details_mapping=targeting_details_mapping
# )


# # Collect Advertiser Targeting Data
# dataFrame_Advertiser_AssignedTargetingOptions = collect_targeting_data(
#     advertiserIdList,
#     service,
#     'advertiser',
#     targeting_types_list=[
#         'TARGETING_TYPE_CHANNEL',
#         'TARGETING_TYPE_DIGITAL_CONTENT_LABEL_EXCLUSION',
#         'TARGETING_TYPE_OMID',
#         'TARGETING_TYPE_SENSITIVE_CATEGORY_EXCLUSION',
#         'TARGETING_TYPE_YOUTUBE_VIDEO',
#         'TARGETING_TYPE_YOUTUBE_CHANNEL',
#         'TARGETING_TYPE_KEYWORD'
#     ],
#     details_mapping=targeting_details_mapping
# )


# # Collect Campaign Targeting Data
# dataFrame_Campaign_AssignedTargetingOptions = collect_targeting_data(
#     [{'advertiserId': adv_id, 'campaignId': camp_id} for adv_id, camp_id in grouped_campaigns.items()],
#     service,
#     'campaign',
#     targeting_types_list=[
#         'TARGETING_TYPE_AGE_RANGE',
#         'TARGETING_TYPE_AUTHORIZED_SELLER_STATUS',
#         'TARGETING_TYPE_CONTENT_INSTREAM_POSITION',
#         'TARGETING_TYPE_CONTENT_OUTSTREAM_POSITION',
#         'TARGETING_TYPE_DIGITAL_CONTENT_LABEL_EXCLUSION',
#         'TARGETING_TYPE_ENVIRONMENT',
#         'TARGETING_TYPE_EXCHANGE',
#         'TARGETING_TYPE_GENDER',
#         'TARGETING_TYPE_GEO_REGION',
#         'TARGETING_TYPE_HOUSEHOLD_INCOME',
#         'TARGETING_TYPE_INVENTORY_SOURCE',
#         'TARGETING_TYPE_INVENTORY_SOURCE_GROUP',
#         'TARGETING_TYPE_LANGUAGE',
#         'TARGETING_TYPE_ON_SCREEN_POSITION',
#         'TARGETING_TYPE_PARENTAL_STATUS',
#         'TARGETING_TYPE_SENSITIVE_CATEGORY_EXCLUSION',
#         'TARGETING_TYPE_SUB_EXCHANGE',
#         'TARGETING_TYPE_THIRD_PARTY_VERIFIER',
#         'TARGETING_TYPE_VIEWABILITY'
#     ],
#     details_mapping=targeting_details_mapping
# )


# # Collect Insertion Order Targeting Data
# dataFrame_InsertionOrder_AssignedTargetingOptions = collect_targeting_data(
#     [{'advertiserId': adv_id, 'insertionOrderId': io_id} for adv_id, io_id in grouped_insertion_orders.items()],
#     service,
#     'insertionOrder',
#     targeting_types_list=[
#         'TARGETING_TYPE_AGE_RANGE',
#         'TARGETING_TYPE_APP',
#         'TARGETING_TYPE_APP_CATEGORY',
#         'TARGETING_TYPE_AUDIENCE_GROUP',
#         'TARGETING_TYPE_AUDIO_CONTENT_TYPE',
#         'TARGETING_TYPE_AUTHORIZED_SELLER_STATUS',
#         'TARGETING_TYPE_BROWSER',
#         'TARGETING_TYPE_BUSINESS_CHAIN',
#         'TARGETING_TYPE_CARRIER_AND_ISP',
#         'TARGETING_TYPE_CATEGORY',
#         'TARGETING_TYPE_CHANNEL',
#         'TARGETING_TYPE_CONTENT_DURATION',
#         'TARGETING_TYPE_CONTENT_GENRE',
#         'TARGETING_TYPE_CONTENT_INSTREAM_POSITION',
#         'TARGETING_TYPE_CONTENT_OUTSTREAM_POSITION',
#         'TARGETING_TYPE_CONTENT_STREAM_TYPE',
#         'TARGETING_TYPE_DAY_AND_TIME',
#         'TARGETING_TYPE_DEVICE_MAKE_MODEL',
#         'TARGETING_TYPE_DEVICE_TYPE',
#         'TARGETING_TYPE_DIGITAL_CONTENT_LABEL_EXCLUSION',
#         'TARGETING_TYPE_ENVIRONMENT',
#         'TARGETING_TYPE_EXCHANGE',
#         'TARGETING_TYPE_GENDER',
#         'TARGETING_TYPE_GEO_REGION',
#         'TARGETING_TYPE_HOUSEHOLD_INCOME',
#         'TARGETING_TYPE_INVENTORY_SOURCE',
#         'TARGETING_TYPE_INVENTORY_SOURCE_GROUP',
#         'TARGETING_TYPE_KEYWORD',
#         'TARGETING_TYPE_LANGUAGE',
#         'TARGETING_TYPE_NATIVE_CONTENT_POSITION',
#         'TARGETING_TYPE_NEGATIVE_KEYWORD_LIST',
#         'TARGETING_TYPE_OMID',
#         'TARGETING_TYPE_ON_SCREEN_POSITION',
#         'TARGETING_TYPE_OPERATING_SYSTEM',
#         'TARGETING_TYPE_PARENTAL_STATUS',
#         'TARGETING_TYPE_POI',
#         'TARGETING_TYPE_PROXIMITY_LOCATION_LIST',
#         'TARGETING_TYPE_REGIONAL_LOCATION_LIST',
#         'TARGETING_TYPE_SENSITIVE_CATEGORY_EXCLUSION',
#         'TARGETING_TYPE_SUB_EXCHANGE',
#         'TARGETING_TYPE_THIRD_PARTY_VERIFIER',
#         'TARGETING_TYPE_URL',
#         'TARGETING_TYPE_USER_REWARDED_CONTENT',
#         'TARGETING_TYPE_VIDEO_PLAYER_SIZE',
#         'TARGETING_TYPE_VIEWABILITY'
#     ],
#     details_mapping=targeting_details_mapping
# )


An error occurred for advertiser ID 2690451: Got an unexpected keyword argument targetingType
An error occurred for advertiser ID 2690451: Got an unexpected keyword argument targetingType
An error occurred for advertiser ID 2690451: Got an unexpected keyword argument targetingType
An error occurred for advertiser ID 2690451: Got an unexpected keyword argument targetingType
An error occurred for advertiser ID 2690451: Got an unexpected keyword argument targetingType
An error occurred for advertiser ID 2690451: Got an unexpected keyword argument targetingType
An error occurred for advertiser ID 2690451: Got an unexpected keyword argument targetingType
An error occurred for advertiser ID 2888635: Got an unexpected keyword argument targetingType
An error occurred for advertiser ID 2888635: Got an unexpected keyword argument targetingType
An error occurred for advertiser ID 2888635: Got an unexpected keyword argument targetingType
An error occurred for advertiser ID 2888635: Got an unexpect

KeyboardInterrupt: 

# Old Code for reference

In [ ]:
# ## WORKING

# # Initialize the DataFrame with the desired columns
# dataFrame_Partner_AssignedTargetingOptions = pd.DataFrame({
#     'partnerId': [],
#     'channelDetails': []
# })

# # List of targeting types to iterate over
# targeting_types_list = [
#     'TARGETING_TYPE_CHANNEL',
# ]

# # Loop through each partner ID and targeting type
# for PID in partnerIdList:
#     for targeting_type in targeting_types_list:
#         try:
#             # Print the current partner ID
#             print(f"Processing partner ID: {PID}")

#             # Make the API request for each combination of partner and targeting type
#             request = service.partners().targetingTypes().assignedTargetingOptions().list(
#                 partnerId=PID,
#                 targetingType=targeting_type
#             )
#             response = request.execute()

#             channelDetails = None  # Default to None in case there's no response

#             if 'assignedTargetingOptions' in response:
#                 partnerListResponse = pd.json_normalize(response['assignedTargetingOptions'])

#                 # Extract and process the channelDetails
#                 channelDetails = get_targeting_details(
#                     partnerListResponse,
#                     'TARGETING_TYPE_CHANNEL',
#                     'channelDetails.channelId',
#                     'channelDetails.negative'
#                 )
#                 channelDetails = sort_by_column(channelDetails, 1)  # Sorting based on 'channelDetails.channelId'

#             # Append to the DataFrame
#             new_row = {
#                 'partnerId': PID,
#                 'channelDetails': channelDetails
#             }

#             dataFrame_Partner_AssignedTargetingOptions = pd.concat([dataFrame_Partner_AssignedTargetingOptions, pd.DataFrame([new_row])], ignore_index=True)

#         except Exception as e:
#             print(f"An error occurred for PID {PID}: {e}")

# # Output the final DataFrame
# print(dataFrame_Partner_AssignedTargetingOptions)


In [ ]:
# # List of targeting types you want to check
# targeting_types_list = [
#     'TARGETING_TYPE_CHANNEL'
# ]

# # Initialize the dictionary to store results
# # Initialize the dictionary to store results
# advertiser_assignedTargetingOptions_dict = {}

# for AID in advertiserIdList:
#     advertiser_assignedTargetingOptions_dict[AID] = {}
#     for targeting_type in targeting_types_list:
#         try:
#             request = service.advertisers().targetingTypes().assignedTargetingOptions().list(
#                 advertiserId=AID,
#                 targetingType=targeting_type
#             )
#             response = request.execute()

#             if 'assignedTargetingOptions' in response:
#                 advertiserListResponse = pd.json_normalize(response['assignedTargetingOptions'])

#                 # Get the parameters and sorting index for the current targeting type
#                 params_with_index = targeting_details_mapping.get(targeting_type)

#                 if params_with_index:
#                     # Unpack the sorting index from the parameters
#                     *params, sort_index = params_with_index

#                     # Extract details using get_targeting_details
#                     targeting_details = get_targeting_details(
#                         advertiserListResponse,
#                         targeting_type,
#                         *params  # Unpack the parameters
#                     )

#                     # Sort the targeting details if needed
#                     if isinstance(targeting_details, list) and len(targeting_details) > 1:
#                         # Use the specified sort index from the mapping
#                         targeting_details = sort_by_column(targeting_details, sort_index)

#                     if isinstance(targeting_details, list):
#                         targeting_details.sort()  # Sort the list if it's not already sorted
#                     else:
#                         print(f"Expected list but got {type(targeting_details)} for {targeting_type} details.")
#                 else:
#                     print(f"No parameters defined for targeting type {targeting_type}.")
#                     targeting_details = None
#             else:
#                 # Handle the case where 'assignedTargetingOptions' is not in the response
#                 targeting_details = None

#             # Store the result in the dictionary under the specific targeting type
#             advertiser_assignedTargetingOptions_dict[AID][targeting_type] = targeting_details

#         except Exception as e:
#             print(f"An error occurred for AID {AID}, targeting type {targeting_type}: {e}")

# # Add each targeting type's details as new columns in the existing DataFrame
# for targeting_type in targeting_types_list:
#     # Create a simplified column name by removing the 'TARGETING_TYPE_' prefix and converting to lowercase
#     col_name = f'targetingType_{targeting_type.replace("TARGETING_TYPE_", "").lower()}'

#     dataFrame_Advertisers_list[col_name] = dataFrame_Advertisers_list['advertiserId'].map(
#         lambda x: advertiser_assignedTargetingOptions_dict[x].get(targeting_type, None)
#     )

In [ ]:
# import pandas as pd

# # Initialize the DataFrame with dynamic columns based on the specified targeting types
# dataFrame_Advertiser_AssignedTargetingOptions = pd.DataFrame({
#     'advertiserId': []
# })

# # List of targeting types to iterate over
# targeting_types_list = [
#     'TARGETING_TYPE_CHANNEL',
#     'TARGETING_TYPE_DIGITAL_CONTENT_LABEL_EXCLUSION',
#     'TARGETING_TYPE_OMID',
#     'TARGETING_TYPE_SENSITIVE_CATEGORY_EXCLUSION',
#     'TARGETING_TYPE_YOUTUBE_VIDEO',
#     'TARGETING_TYPE_YOUTUBE_CHANNEL',
#     'TARGETING_TYPE_KEYWORD'
# ]

# # Dynamically add columns based on the selected targeting types in targeting_types_list
# for targeting_type in targeting_types_list:
#     if targeting_type in targeting_details_mapping:
#         dataFrame_Advertiser_AssignedTargetingOptions[f'{targeting_type}'] = None

# # Loop through each advertiser ID
# for AID in advertiserIdList:
#     new_row = {'advertiserId': AID}

#     for targeting_type in targeting_types_list:
#         try:
#             # Print the current advertiser ID and targeting type
#             print(f"Processing advertiser ID: {AID} for targeting type: {targeting_type}")

#             # Make the API request for each combination of advertiser and targeting type
#             request = service.advertisers().targetingTypes().assignedTargetingOptions().list(
#                 advertiserId=AID,
#                 targetingType=targeting_type
#             )
#             response = request.execute()

#             targeting_details = None  # Default to None in case there's no response

#             if 'assignedTargetingOptions' in response:
#                 advertiserListResponse = pd.json_normalize(response['assignedTargetingOptions'])

#                 # Extract and process the targeting details using the mapping
#                 targeting_fields = targeting_details_mapping[targeting_type]

#                 # Use the get_targeting_details function to extract details
#                 targeting_details = get_targeting_details(
#                     advertiserListResponse,
#                     targeting_type,
#                     *targeting_fields[:-2]  # Unpack the fields to be passed to the function
#                 )

#                 # Use the sort_by_column function to sort the details
#                 sorting_column_index = targeting_fields[-2]
#                 targeting_details = sort_by_column(targeting_details, sorting_column_index)

#             # Store the targeting details in the new_row dictionary
#             new_row[f'{targeting_type}'] = targeting_details

#         except Exception as e:
#             print(f"An error occurred for advertiser ID {AID}: {e}")

#     # Append the new_row to the DataFrame after processing all targeting types for the advertiser
#     dataFrame_Advertiser_AssignedTargetingOptions = pd.concat([dataFrame_Advertiser_AssignedTargetingOptions, pd.DataFrame([new_row])], ignore_index=True)

# # Output the final DataFrame
# print(dataFrame_Advertiser_AssignedTargetingOptions)


In [ ]:
# ## Old code

# # Initialize the DataFrame with dynamic columns based on the specified targeting types
# dataFrame_InsertionOrder_AssignedTargetingOptions = pd.DataFrame({
#     'advertiserId': [],
#     'insertionOrderId': []  # Include insertionOrderId in the DataFrame
# })

# # List of targeting types to use
# targeting_types_list = [
#     'TARGETING_TYPE_AGE_RANGE',
#     'TARGETING_TYPE_APP',
#     'TARGETING_TYPE_APP_CATEGORY',
#     'TARGETING_TYPE_AUDIENCE_GROUP',
#     'TARGETING_TYPE_AUDIO_CONTENT_TYPE',
#     'TARGETING_TYPE_AUTHORIZED_SELLER_STATUS',
#     'TARGETING_TYPE_BROWSER',
#     'TARGETING_TYPE_BUSINESS_CHAIN',
#     'TARGETING_TYPE_CARRIER_AND_ISP',
#     'TARGETING_TYPE_CATEGORY',
#     'TARGETING_TYPE_CHANNEL',
#     'TARGETING_TYPE_CONTENT_DURATION',
#     'TARGETING_TYPE_CONTENT_GENRE',
#     'TARGETING_TYPE_CONTENT_INSTREAM_POSITION',
#     'TARGETING_TYPE_CONTENT_OUTSTREAM_POSITION',
#     'TARGETING_TYPE_CONTENT_STREAM_TYPE',
#     'TARGETING_TYPE_DAY_AND_TIME',
#     'TARGETING_TYPE_DEVICE_MAKE_MODEL',
#     'TARGETING_TYPE_DEVICE_TYPE',
#     'TARGETING_TYPE_DIGITAL_CONTENT_LABEL_EXCLUSION',
#     'TARGETING_TYPE_ENVIRONMENT',
#     'TARGETING_TYPE_EXCHANGE',
#     'TARGETING_TYPE_GENDER',
#     'TARGETING_TYPE_GEO_REGION',
#     'TARGETING_TYPE_HOUSEHOLD_INCOME',
#     'TARGETING_TYPE_INVENTORY_SOURCE',
#     'TARGETING_TYPE_INVENTORY_SOURCE_GROUP',
#     'TARGETING_TYPE_KEYWORD',
#     'TARGETING_TYPE_LANGUAGE',
#     'TARGETING_TYPE_NATIVE_CONTENT_POSITION',
#     'TARGETING_TYPE_NEGATIVE_KEYWORD_LIST',
#     'TARGETING_TYPE_OMID',
#     'TARGETING_TYPE_ON_SCREEN_POSITION',
#     'TARGETING_TYPE_OPERATING_SYSTEM',
#     'TARGETING_TYPE_PARENTAL_STATUS',
#     'TARGETING_TYPE_POI',
#     'TARGETING_TYPE_PROXIMITY_LOCATION_LIST',
#     'TARGETING_TYPE_REGIONAL_LOCATION_LIST',
#     'TARGETING_TYPE_SENSITIVE_CATEGORY_EXCLUSION',
#     'TARGETING_TYPE_SUB_EXCHANGE',
#     'TARGETING_TYPE_THIRD_PARTY_VERIFIER',
#     'TARGETING_TYPE_URL',
#     'TARGETING_TYPE_USER_REWARDED_CONTENT',
#     'TARGETING_TYPE_VIDEO_PLAYER_SIZE',
#     'TARGETING_TYPE_VIEWABILITY'
# ]

# # Dynamically add columns based on the selected targeting types in targeting_types_list
# for targeting_type in targeting_types_list:
#     if targeting_type in targeting_details_mapping:
#         dataFrame_InsertionOrder_AssignedTargetingOptions[f'{targeting_type}'] = []

# # Function to extract and process targeting details
# def process_targeting_details(insertionOrderListResponse, targeting_type):
#     targeting_fields = targeting_details_mapping[targeting_type]
#     targeting_details = get_targeting_details(
#         insertionOrderListResponse,
#         targeting_type,
#         *targeting_fields[:-2]  # Unpack the fields to be passed to the function
#     )
#     sorting_column_index = targeting_fields[-2]
#     targeting_details = sort_by_column(targeting_details, sorting_column_index)
#     return targeting_details

# # Loop through each advertiser Id
# for AID in advertiserIdList:
#     try:
#         # Print the current advertiser Id
#         print(f"Processing advertiser Id: {AID}")

#         # Retrieve the list of insertionOrder IDs for the given advertiser
#         insertionOrders_response = service.advertisers().insertionOrders().list(
#             advertiserId=AID
#         ).execute()

#         insertionOrderIdList = [insertionOrder['insertionOrderId'] for insertionOrder in insertionOrders_response.get('insertionOrders', [])]

#         # Loop through each insertionOrder ID
#         for IOID in insertionOrderIdList:
#             try:
#                 # Print the current insertionOrder Id
#                 print(f"Processing insertionOrder Id: {IOID} for advertiser Id: {AID}")

#                 # Make a single API request to get all targeting types for this insertionOrder
#                 request = service.advertisers().insertionOrders().listAssignedTargetingOptions(
#                     advertiserId=AID,
#                     insertionOrderId=IOID
#                 )
#                 response = request.execute()

#                 # Initialize a dictionary to hold the data for this advertiser and insertionOrder
#                 new_row = {'advertiserId': AID, 'insertionOrderId': IOID}

#                 # Process each targeting type in the response
#                 if 'assignedTargetingOptions' in response:
#                     insertionOrderListResponse = pd.json_normalize(response['assignedTargetingOptions'])

#                     for targeting_type in targeting_types_list:
#                         if targeting_type in insertionOrderListResponse['targetingType'].values:
#                             # Process the targeting details for the current targeting type
#                             targeting_details = process_targeting_details(insertionOrderListResponse, targeting_type)
#                             new_row[f'{targeting_type}'] = targeting_details
#                         else:
#                             new_row[f'{targeting_type}'] = None  # If the targeting type is not in the response, set to None

#                 # Append the completed row for this advertiser and insertionOrder to the DataFrame
#                 dataFrame_InsertionOrder_AssignedTargetingOptions = pd.concat([dataFrame_InsertionOrder_AssignedTargetingOptions, pd.DataFrame([new_row])], ignore_index=True)

#             except Exception as e:
#                 print(f"An error occurred for IOID {IOID} in advertiser ID {AID}: {e}")

#     except Exception as e:
#         print(f"An error occurred for AID {AID}: {e}")

# # Output the final DataFrame
# print(dataFrame_InsertionOrder_AssignedTargetingOptions)


In [ ]:
# # Initialize the DataFrame with dynamic columns based on the specified targeting types
# dataFrame_Campaign_AssignedTargetingOptions = pd.DataFrame({
#     'advertiserId': [],
#     'campaignId': []  # Include campaignId in the DataFrame
# })

# # List of targeting types to use
# targeting_types_list = [
#     'TARGETING_TYPE_AGE_RANGE',
#     'TARGETING_TYPE_AUTHORIZED_SELLER_STATUS',
#     'TARGETING_TYPE_CONTENT_INSTREAM_POSITION',
#     'TARGETING_TYPE_CONTENT_OUTSTREAM_POSITION',
#     'TARGETING_TYPE_DIGITAL_CONTENT_LABEL_EXCLUSION',
#     'TARGETING_TYPE_ENVIRONMENT',
#     'TARGETING_TYPE_EXCHANGE',
#     'TARGETING_TYPE_GENDER',
#     'TARGETING_TYPE_GEO_REGION',
#     'TARGETING_TYPE_HOUSEHOLD_INCOME',
#     'TARGETING_TYPE_INVENTORY_SOURCE',
#     'TARGETING_TYPE_INVENTORY_SOURCE_GROUP',
#     'TARGETING_TYPE_LANGUAGE',
#     'TARGETING_TYPE_ON_SCREEN_POSITION',
#     'TARGETING_TYPE_PARENTAL_STATUS',
#     'TARGETING_TYPE_SENSITIVE_CATEGORY_EXCLUSION',
#     'TARGETING_TYPE_SUB_EXCHANGE',
#     'TARGETING_TYPE_THIRD_PARTY_VERIFIER',
#     'TARGETING_TYPE_VIEWABILITY'
# ]

# # Dynamically add columns based on the selected targeting types in targeting_types_list
# for targeting_type in targeting_types_list:
#     if targeting_type in targeting_details_mapping:
#         dataFrame_Campaign_AssignedTargetingOptions[f'{targeting_type}'] = []

# # Function to extract and process targeting details
# def process_targeting_details(campaignListResponse, targeting_type):
#     targeting_fields = targeting_details_mapping[targeting_type]
#     targeting_details = get_targeting_details(
#         campaignListResponse,
#         targeting_type,
#         *targeting_fields[:-2]  # Unpack the fields to be passed to the function
#     )
#     sorting_column_index = targeting_fields[-2]
#     targeting_details = sort_by_column(targeting_details, sorting_column_index)
#     return targeting_details

# # Loop through each advertiser Id
# for AID in advertiserIdList:
#     try:
#         # Print the current advertiser Id
#         print(f"Processing advertiser Id: {AID}")

#         # Retrieve the list of campaign IDs for the given advertiser
#         campaigns_response = service.advertisers().campaigns().list(
#             advertiserId=AID
#         ).execute()

#         campaignIdList = [campaign['campaignId'] for campaign in campaigns_response.get('campaigns', [])]

#         # Loop through each campaign ID
#         for CID in campaignIdList:
#             try:
#                 # Print the current campaign Id
#                 print(f"Processing campaign Id: {CID} for advertiser Id: {AID}")

#                 # Make a single API request to get all targeting types for this campaign
#                 request = service.advertisers().campaigns().listAssignedTargetingOptions(
#                     advertiserId=AID,
#                     campaignId=CID
#                 )
#                 response = request.execute()

#                 # Initialize a dictionary to hold the data for this advertiser and campaign
#                 new_row = {'advertiserId': AID, 'campaignId': CID}

#                 # Process each targeting type in the response
#                 if 'assignedTargetingOptions' in response:
#                     campaignListResponse = pd.json_normalize(response['assignedTargetingOptions'])

#                     for targeting_type in targeting_types_list:
#                         if targeting_type in campaignListResponse['targetingType'].values:
#                             # Process the targeting details for the current targeting type
#                             targeting_details = process_targeting_details(campaignListResponse, targeting_type)
#                             new_row[f'{targeting_type}'] = targeting_details
#                         else:
#                             new_row[f'{targeting_type}'] = None  # If the targeting type is not in the response, set to None

#                 # Append the completed row for this advertiser and campaign to the DataFrame
#                 dataFrame_Campaign_AssignedTargetingOptions = pd.concat([dataFrame_Campaign_AssignedTargetingOptions, pd.DataFrame([new_row])], ignore_index=True)

#             except Exception as e:
#                 print(f"An error occurred for CID {CID} in advertiser ID {AID}: {e}")

#     except Exception as e:
#         print(f"An error occurred for AID {AID}: {e}")

# # Output the final DataFrame
# print(dataFrame_Campaign_AssignedTargetingOptions)

In [ ]:
# # Initialize the DataFrame with dynamic columns based on the specified targeting types
# dataFrame_LineItem_AssignedTargetingOptions = pd.DataFrame({
#     'advertiserId': [],
#     'lineItemId': []  # Include lineItemId in the DataFrame
# })

# # List of targeting types to use
# targeting_types_list = [
#   'TARGETING_TYPE_AGE_RANGE',
#   'TARGETING_TYPE_APP',
#   'TARGETING_TYPE_APP_CATEGORY',
#   'TARGETING_TYPE_AUDIENCE_GROUP',
#   'TARGETING_TYPE_AUDIO_CONTENT_TYPE',
#   'TARGETING_TYPE_AUTHORIZED_SELLER_STATUS',
#   'TARGETING_TYPE_BROWSER',
#   'TARGETING_TYPE_BUSINESS_CHAIN',
#   'TARGETING_TYPE_CARRIER_AND_ISP',
#   'TARGETING_TYPE_CATEGORY',
#   'TARGETING_TYPE_CHANNEL',
#   'TARGETING_TYPE_CONTENT_DURATION',
#   'TARGETING_TYPE_CONTENT_GENRE',
#   'TARGETING_TYPE_CONTENT_INSTREAM_POSITION',
#   'TARGETING_TYPE_CONTENT_OUTSTREAM_POSITION',
#   'TARGETING_TYPE_CONTENT_STREAM_TYPE',
#   'TARGETING_TYPE_DAY_AND_TIME',
#   'TARGETING_TYPE_DEVICE_MAKE_MODEL',
#   'TARGETING_TYPE_DEVICE_TYPE',
#   'TARGETING_TYPE_DIGITAL_CONTENT_LABEL_EXCLUSION',
#   'TARGETING_TYPE_ENVIRONMENT',
#   'TARGETING_TYPE_EXCHANGE',
#   'TARGETING_TYPE_GENDER',
#   'TARGETING_TYPE_GEO_REGION',
#   'TARGETING_TYPE_HOUSEHOLD_INCOME',
#   'TARGETING_TYPE_INVENTORY_SOURCE',
#   'TARGETING_TYPE_INVENTORY_SOURCE_GROUP',
#   'TARGETING_TYPE_KEYWORD',
#   'TARGETING_TYPE_LANGUAGE',
#   'TARGETING_TYPE_NATIVE_CONTENT_POSITION',
#   'TARGETING_TYPE_NEGATIVE_KEYWORD_LIST',
#   'TARGETING_TYPE_OMID',
#   'TARGETING_TYPE_ON_SCREEN_POSITION',
#   'TARGETING_TYPE_OPERATING_SYSTEM',
#   'TARGETING_TYPE_PARENTAL_STATUS',
#   'TARGETING_TYPE_POI',
#   'TARGETING_TYPE_PROXIMITY_LOCATION_LIST',
#   'TARGETING_TYPE_REGIONAL_LOCATION_LIST',
#   'TARGETING_TYPE_SENSITIVE_CATEGORY_EXCLUSION',
#   'TARGETING_TYPE_SUB_EXCHANGE',
#   'TARGETING_TYPE_THIRD_PARTY_VERIFIER',
#   'TARGETING_TYPE_URL',
#   'TARGETING_TYPE_USER_REWARDED_CONTENT',
#   'TARGETING_TYPE_VIDEO_PLAYER_SIZE',
#   'TARGETING_TYPE_VIEWABILITY',
#   'TARGETING_TYPE_YOUTUBE_CHANNEL',
#   'TARGETING_TYPE_YOUTUBE_VIDEO'
# ]

# # Dynamically add columns based on the selected targeting types in targeting_types_list
# for targeting_type in targeting_types_list:
#     if targeting_type in targeting_details_mapping:
#         dataFrame_LineItem_AssignedTargetingOptions[f'{targeting_type}'] = []

# # Function to extract and process targeting details
# def process_targeting_details(lineItemListResponse, targeting_type):
#     targeting_fields = targeting_details_mapping[targeting_type]
#     targeting_details = get_targeting_details(
#         lineItemListResponse,
#         targeting_type,
#         *targeting_fields[:-2]  # Unpack the fields to be passed to the function
#     )
#     sorting_column_index = targeting_fields[-2]
#     targeting_details = sort_by_column(targeting_details, sorting_column_index)
#     return targeting_details

# # Loop through each advertiser Id
# for AID in advertiserIdList:
#     try:
#         # Print the current advertiser Id
#         print(f"Processing advertiser Id: {AID}")

#         # Retrieve the list of lineItem IDs for the given advertiser
#         lineItems_response = service.advertisers().lineItems().list(
#             advertiserId=AID
#         ).execute()

#         lineItemIdList = [lineItem['lineItemId'] for lineItem in lineItems_response.get('lineItems', [])]

#         # Loop through each lineItem ID
#         for LIID in lineItemIdList:
#             try:
#                 # Print the current lineItem Id
#                 print(f"Processing lineItem Id: {LIID} for advertiser Id: {AID}")

#                 # Make a single API request to get all targeting types for this lineItem
#                 request = service.advertisers().lineItems().bulkListAssignedTargetingOptions(
#                     advertiserId=AID,
#                     lineItemIds=[LIID],  # Pass lineItemId as a list
#                 )
#                 response = request.execute()

#                 # Initialize a dictionary to hold the data for this advertiser and lineItem
#                 new_row = {'advertiserId': AID, 'lineItemId': LIID}

#                 # Process each targeting type in the response
#                 if 'assignedTargetingOptions' in response:
#                     lineItemListResponse = pd.json_normalize(response['assignedTargetingOptions'])

#                     for targeting_type in targeting_types_list:
#                         if targeting_type in lineItemListResponse['targetingType'].values:
#                             # Process the targeting details for the current targeting type
#                             targeting_details = process_targeting_details(lineItemListResponse, targeting_type)
#                             new_row[f'{targeting_type}'] = targeting_details
#                         else:
#                             new_row[f'{targeting_type}'] = None  # If the targeting type is not in the response, set to None

#                 # Append the completed row for this advertiser and lineItem to the DataFrame
#                 dataFrame_LineItem_AssignedTargetingOptions = pd.concat([dataFrame_LineItem_AssignedTargetingOptions, pd.DataFrame([new_row])], ignore_index=True)

#             except Exception as e:
#                 print(f"An error occurred for LIID {LIID} in advertiser ID {AID}: {e}")

#     except Exception as e:
#         print(f"An error occurred for AID {AID}: {e}")

# # Output the final DataFrame
# print(dataFrame_LineItem_AssignedTargetingOptions)


In [ ]:
# # Initialize the DataFrame with dynamic columns based on the specified targeting types
# dataFrame_LineItem_AssignedTargetingOptions = pd.DataFrame({
#     'advertiserId': [],
#     'lineItemId': []
# })

# # List of targeting types to use
# targeting_types_list = [
#   'TARGETING_TYPE_AGE_RANGE',
#   'TARGETING_TYPE_APP',
#   'TARGETING_TYPE_APP_CATEGORY',
#   'TARGETING_TYPE_AUDIENCE_GROUP',
#   'TARGETING_TYPE_AUDIO_CONTENT_TYPE',
#   'TARGETING_TYPE_AUTHORIZED_SELLER_STATUS',
#   'TARGETING_TYPE_BROWSER',
#   'TARGETING_TYPE_BUSINESS_CHAIN',
#   'TARGETING_TYPE_CARRIER_AND_ISP',
#   'TARGETING_TYPE_CATEGORY',
#   'TARGETING_TYPE_CHANNEL',
#   'TARGETING_TYPE_CONTENT_DURATION',
#   'TARGETING_TYPE_CONTENT_GENRE',
#   'TARGETING_TYPE_CONTENT_INSTREAM_POSITION',
#   'TARGETING_TYPE_CONTENT_OUTSTREAM_POSITION',
#   'TARGETING_TYPE_CONTENT_STREAM_TYPE',
#   'TARGETING_TYPE_DAY_AND_TIME',
#   'TARGETING_TYPE_DEVICE_MAKE_MODEL',
#   'TARGETING_TYPE_DEVICE_TYPE',
#   'TARGETING_TYPE_DIGITAL_CONTENT_LABEL_EXCLUSION',
#   'TARGETING_TYPE_ENVIRONMENT',
#   'TARGETING_TYPE_EXCHANGE',
#   'TARGETING_TYPE_GENDER',
#   'TARGETING_TYPE_GEO_REGION',
#   'TARGETING_TYPE_HOUSEHOLD_INCOME',
#   'TARGETING_TYPE_INVENTORY_SOURCE',
#   'TARGETING_TYPE_INVENTORY_SOURCE_GROUP',
#   'TARGETING_TYPE_KEYWORD',
#   'TARGETING_TYPE_LANGUAGE',
#   'TARGETING_TYPE_NATIVE_CONTENT_POSITION',
#   'TARGETING_TYPE_NEGATIVE_KEYWORD_LIST',
#   'TARGETING_TYPE_OMID',
#   'TARGETING_TYPE_ON_SCREEN_POSITION',
#   'TARGETING_TYPE_OPERATING_SYSTEM',
#   'TARGETING_TYPE_PARENTAL_STATUS',
#   'TARGETING_TYPE_POI',
#   'TARGETING_TYPE_PROXIMITY_LOCATION_LIST',
#   'TARGETING_TYPE_REGIONAL_LOCATION_LIST',
#   'TARGETING_TYPE_SENSITIVE_CATEGORY_EXCLUSION',
#   'TARGETING_TYPE_SUB_EXCHANGE',
#   'TARGETING_TYPE_THIRD_PARTY_VERIFIER',
#   'TARGETING_TYPE_URL',
#   'TARGETING_TYPE_USER_REWARDED_CONTENT',
#   'TARGETING_TYPE_VIDEO_PLAYER_SIZE',
#   'TARGETING_TYPE_VIEWABILITY',
#   'TARGETING_TYPE_YOUTUBE_CHANNEL',
#   'TARGETING_TYPE_YOUTUBE_VIDEO'
# ]

# # Dynamically add columns based on the selected targeting types in targeting_types_list
# for targeting_type in targeting_types_list:
#     if targeting_type in targeting_details_mapping:
#         dataFrame_LineItem_AssignedTargetingOptions[f'{targeting_type}'] = []

# # Function to extract and process targeting details
# def process_targeting_details(lineItemListResponse, targeting_type):
#     targeting_fields = targeting_details_mapping[targeting_type]
#     targeting_details = get_targeting_details(
#         lineItemListResponse,
#         targeting_type,
#         *targeting_fields[:-2]  # Unpack the fields to be passed to the function
#     )
#     sorting_column_index = targeting_fields[-2]
#     targeting_details = sort_by_column(targeting_details, sorting_column_index)
#     return targeting_details

# # Deduplicate lineItemId when grouping by advertiserId
# grouped_line_items = id_Name_table.groupby('Advertiser ID')['Line Item ID'].apply(lambda x: list(set(x.dropna()))).to_dict()

# # Loop through each advertiserId and its corresponding lineItemIds
# for AID, lineItemIdList in grouped_line_items.items():
#     try:
#         # Print the current advertiser Id
#         print(f"Processing advertiser Id: {AID}")

#         # Loop through each lineItem ID
#         for LIID in lineItemIdList:
#             # Check if LIID is not NaN before processing
#             if pd.isna(LIID):
#                 print(f"Skipping NaN lineItemId for advertiser Id: {AID}")
#                 continue

#             try:
#                 # Print the current lineItem Id
#                 print(f"Processing lineItem Id: {LIID} for advertiser Id: {AID}")

#                 # Make a single API request to get all targeting types for this lineItem
#                 request = service.advertisers().lineItems().bulkListAssignedTargetingOptions(
#                     advertiserId=AID,
#                     lineItemIds=LIID
#                 )
#                 response = request.execute()

#                 # Initialize a dictionary to hold the data for this advertiser and lineItem
#                 new_row = {'advertiserId': AID, 'lineItemId': LIID}

#                 # Process each targeting type in the response
#                 if 'lineItemAssignedTargetingOptions' in response:
#                     lineItemListResponse = pd.json_normalize(response['lineItemAssignedTargetingOptions'])

#                     for targeting_type in targeting_types_list:
#                         if targeting_type in lineItemListResponse['targetingType'].values:
#                             # Process the targeting details for the current targeting type
#                             targeting_details = process_targeting_details(lineItemListResponse, targeting_type)
#                             new_row[f'{targeting_type}'] = targeting_details
#                         else:
#                             new_row[f'{targeting_type}'] = None  # If the targeting type is not in the response, set to None

#                 # Append the completed row for this advertiser and lineItem to the DataFrame
#                 dataFrame_LineItems_AssignedTargetingOptions = pd.concat([dataFrame_LineItems_AssignedTargetingOptions, pd.DataFrame([new_row])], ignore_index=True)

#             except Exception as e:
#                 print(f"An error occurred for LIID {LIID} in advertiser ID {AID}: {e}")

#     except Exception as e:
#         print(f"An error occurred for AID {AID}: {e}")

# # Output the final DataFrame
# print(dataFrame_LineItems_AssignedTargetingOptions)

In [ ]:
# # Deduplicate lineItemId when grouping by advertiserId
# grouped_line_items = id_Name_table.groupby('Advertiser ID')['Line Item ID'].apply(lambda x: list(set(x.dropna()))).to_dict()

# # Initialize the DataFrame with dynamic columns based on the specified targeting types
# dataFrame_LineItem_AssignedTargetingOptions = pd.DataFrame({
#     'advertiserId': [],
#     'lineItemId': []  # Include lineItemId in the DataFrame
# })

# # List of targeting types to use
# targeting_types_list = [
#   'TARGETING_TYPE_AGE_RANGE',
#   'TARGETING_TYPE_APP',
#   'TARGETING_TYPE_APP_CATEGORY',
#   'TARGETING_TYPE_AUDIENCE_GROUP',
#   'TARGETING_TYPE_AUDIO_CONTENT_TYPE',
#   'TARGETING_TYPE_AUTHORIZED_SELLER_STATUS',
#   'TARGETING_TYPE_BROWSER',
#   'TARGETING_TYPE_BUSINESS_CHAIN',
#   'TARGETING_TYPE_CARRIER_AND_ISP',
#   'TARGETING_TYPE_CATEGORY',
#   'TARGETING_TYPE_CHANNEL',
#   'TARGETING_TYPE_CONTENT_DURATION',
#   'TARGETING_TYPE_CONTENT_GENRE',
#   'TARGETING_TYPE_CONTENT_INSTREAM_POSITION',
#   'TARGETING_TYPE_CONTENT_OUTSTREAM_POSITION',
#   'TARGETING_TYPE_CONTENT_STREAM_TYPE',
#   'TARGETING_TYPE_DAY_AND_TIME',
#   'TARGETING_TYPE_DEVICE_MAKE_MODEL',
#   'TARGETING_TYPE_DEVICE_TYPE',
#   'TARGETING_TYPE_DIGITAL_CONTENT_LABEL_EXCLUSION',
#   'TARGETING_TYPE_ENVIRONMENT',
#   'TARGETING_TYPE_EXCHANGE',
#   'TARGETING_TYPE_GENDER',
#   'TARGETING_TYPE_GEO_REGION',
#   'TARGETING_TYPE_HOUSEHOLD_INCOME',
#   'TARGETING_TYPE_INVENTORY_SOURCE',
#   'TARGETING_TYPE_INVENTORY_SOURCE_GROUP',
#   'TARGETING_TYPE_KEYWORD',
#   'TARGETING_TYPE_LANGUAGE',
#   'TARGETING_TYPE_NATIVE_CONTENT_POSITION',
#   'TARGETING_TYPE_NEGATIVE_KEYWORD_LIST',
#   'TARGETING_TYPE_OMID',
#   'TARGETING_TYPE_ON_SCREEN_POSITION',
#   'TARGETING_TYPE_OPERATING_SYSTEM',
#   'TARGETING_TYPE_PARENTAL_STATUS',
#   'TARGETING_TYPE_POI',
#   'TARGETING_TYPE_PROXIMITY_LOCATION_LIST',
#   'TARGETING_TYPE_REGIONAL_LOCATION_LIST',
#   'TARGETING_TYPE_SENSITIVE_CATEGORY_EXCLUSION',
#   'TARGETING_TYPE_SUB_EXCHANGE',
#   'TARGETING_TYPE_THIRD_PARTY_VERIFIER',
#   'TARGETING_TYPE_URL',
#   'TARGETING_TYPE_USER_REWARDED_CONTENT',
#   'TARGETING_TYPE_VIDEO_PLAYER_SIZE',
#   'TARGETING_TYPE_VIEWABILITY',
#   'TARGETING_TYPE_YOUTUBE_CHANNEL',
#   'TARGETING_TYPE_YOUTUBE_VIDEO'
# ]

# # Dynamically add columns based on the selected targeting types in targeting_types_list
# for targeting_type in targeting_types_list:
#     if targeting_type in targeting_details_mapping:
#         dataFrame_LineItem_AssignedTargetingOptions[f'{targeting_type}'] = []

# # Function to extract and process targeting details
# def process_targeting_details(lineItemListResponse, targeting_type):
#     targeting_fields = targeting_details_mapping[targeting_type]
#     targeting_details = get_targeting_details(
#         lineItemListResponse,
#         targeting_type,
#         *targeting_fields[:-2]  # Unpack the fields to be passed to the function
#     )
#     sorting_column_index = targeting_fields[-2]
#     targeting_details = sort_by_column(targeting_details, sorting_column_index)
#     return targeting_details

# # Loop through each advertiserId and its corresponding deduplicated lineItem IDs
# for AID, lineItemIdList in grouped_line_items.items():
#     try:
#         # Print the current advertiser Id
#         print(f"Processing advertiser Id: {AID}")

#         # Loop through each lineItem ID
#         for LIID in lineItemIdList:
#             # Check if LIID is not NaN before processing
#             if pd.isna(LIID):
#                 print(f"Skipping NaN lineItemId for advertiser Id: {AID}")
#                 continue

#             try:
#                 # Print the current lineItem Id
#                 print(f"Processing lineItem Id: {LIID} for advertiser Id: {AID}")

#                 # Make a single API request to get all targeting types for this lineItem
#                 request = service.advertisers().lineItems().bulkListAssignedTargetingOptions(
#                     advertiserId=AID,
#                     lineItemIds=[LIID],  # Pass lineItemId as a list
#                 )
#                 response = request.execute()

#                 # Initialize a dictionary to hold the data for this advertiser and lineItem
#                 new_row = {'advertiserId': AID, 'lineItemId': LIID}

#                 # Process each targeting type in the response
#                 if 'lineItemAssignedTargetingOptions' in response:
#                     lineItemListResponse = pd.json_normalize(response['lineItemAssignedTargetingOptions'])

#                     for targeting_type in targeting_types_list:
#                         if targeting_type in lineItemListResponse['targetingType'].values:
#                             # Process the targeting details for the current targeting type
#                             targeting_details = process_targeting_details(lineItemListResponse, targeting_type)
#                             new_row[f'{targeting_type}'] = targeting_details
#                         else:
#                             new_row[f'{targeting_type}'] = None  # If the targeting type is not in the response, set to None

#                 # Append the completed row for this advertiser and lineItem to the DataFrame
#                 dataFrame_LineItem_AssignedTargetingOptions = pd.concat([dataFrame_LineItem_AssignedTargetingOptions, pd.DataFrame([new_row])], ignore_index=True)

#             except Exception as e:
#                 print(f"An error occurred for LIID {LIID} in advertiser ID {AID}: {e}")

#     except Exception as e:
#         print(f"An error occurred for AID {AID}: {e}")

# # Output the final DataFrame
# print(dataFrame_LineItem_AssignedTargetingOptions)

In [ ]:
# # Define the directory to save the CSV files
# data_directory = 'Assigned Targeting Options'

# # Create the directory if it doesn't exist
# if not os.path.exists(data_directory):
#     os.makedirs(data_directory)

# # Dictionary of DataFrames to save
# dataframes_dict = {
#     'dataFrame_Partner_AssignedTargetingOptions': dataFrame_Partner_AssignedTargetingOptions,
#     'dataFrame_Advertiser_AssignedTargetingOptions': dataFrame_Advertiser_AssignedTargetingOptions
# }

# # Save each DataFrame as a CSV file in the specified directory
# for df_name, df in dataframes_dict.items():
#     file_path = os.path.join(data_directory, f'{df_name}.csv')
#     df.to_csv(file_path, index=False)
#     print(f"Saved {df_name} to {file_path}")



In [ ]:
# ### Have since added the assignedTargetingOption

# # Initialize the DataFrame with dynamic columns based on the specified targeting types
# dataFrame_InsertionOrder_AssignedTargetingOptions = pd.DataFrame({
#     'advertiserId': [],
#     'insertionOrderId': []  # Include insertionOrderId in the DataFrame
# })

# # List of targeting types to iterate over (add more targeting types as needed)
# targeting_types_list = [
#         'TARGETING_TYPE_AGE_RANGE',
#         'TARGETING_TYPE_APP',
#         'TARGETING_TYPE_APP_CATEGORY',
#         'TARGETING_TYPE_AUDIENCE_GROUP',
#         'TARGETING_TYPE_AUDIO_CONTENT_TYPE',
#         'TARGETING_TYPE_AUTHORIZED_SELLER_STATUS',
#         'TARGETING_TYPE_BROWSER',
#         'TARGETING_TYPE_BUSINESS_CHAIN',
#         'TARGETING_TYPE_CARRIER_AND_ISP',
#         'TARGETING_TYPE_CATEGORY',
#         'TARGETING_TYPE_CHANNEL',
#         'TARGETING_TYPE_CONTENT_DURATION',
#         'TARGETING_TYPE_CONTENT_GENRE',
#         'TARGETING_TYPE_CONTENT_INSTREAM_POSITION',
#         'TARGETING_TYPE_CONTENT_OUTSTREAM_POSITION',
#         'TARGETING_TYPE_CONTENT_STREAM_TYPE',
#         'TARGETING_TYPE_DAY_AND_TIME',
#         'TARGETING_TYPE_DEVICE_MAKE_MODEL',
#         'TARGETING_TYPE_DEVICE_TYPE',
#         'TARGETING_TYPE_DIGITAL_CONTENT_LABEL_EXCLUSION',
#         'TARGETING_TYPE_ENVIRONMENT',
#         'TARGETING_TYPE_EXCHANGE',
#         'TARGETING_TYPE_GENDER',
#         'TARGETING_TYPE_GEO_REGION',
#         'TARGETING_TYPE_HOUSEHOLD_INCOME',
#         'TARGETING_TYPE_INVENTORY_SOURCE',
#         'TARGETING_TYPE_INVENTORY_SOURCE_GROUP',
#         'TARGETING_TYPE_KEYWORD',
#         'TARGETING_TYPE_LANGUAGE',
#         'TARGETING_TYPE_NATIVE_CONTENT_POSITION',
#         'TARGETING_TYPE_NEGATIVE_KEYWORD_LIST',
#         'TARGETING_TYPE_OMID',
#         'TARGETING_TYPE_ON_SCREEN_POSITION',
#         'TARGETING_TYPE_OPERATING_SYSTEM',
#         'TARGETING_TYPE_PARENTAL_STATUS',
#         'TARGETING_TYPE_POI',
#         'TARGETING_TYPE_PROXIMITY_LOCATION_LIST',
#         'TARGETING_TYPE_REGIONAL_LOCATION_LIST',
#         'TARGETING_TYPE_SENSITIVE_CATEGORY_EXCLUSION',
#         'TARGETING_TYPE_SUB_EXCHANGE',
#         'TARGETING_TYPE_THIRD_PARTY_VERIFIER',
#         'TARGETING_TYPE_URL',
#         'TARGETING_TYPE_USER_REWARDED_CONTENT',
#         'TARGETING_TYPE_VIDEO_PLAYER_SIZE',
#         'TARGETING_TYPE_VIEWABILITY'
# ]

# # Dynamically add columns based on the selected targeting types in targeting_types_list
# for targeting_type in targeting_types_list:
#     if targeting_type in targeting_details_mapping:
#         dataFrame_InsertionOrder_AssignedTargetingOptions[f'{targeting_type}'] = []

# # Function to extract and process targeting details
# def process_targeting_details(insertionOrderListResponse, targeting_type):
#     targeting_fields = targeting_details_mapping[targeting_type]
#     targeting_details = get_targeting_details(
#         insertionOrderListResponse,
#         targeting_type,
#         *targeting_fields[:-2]  # Unpack the fields to be passed to the function
#     )
#     sorting_column_index = targeting_fields[-2]
#     targeting_details = sort_by_column(targeting_details, sorting_column_index)
#     return targeting_details

# # Loop through each advertiser Id
# for AID in advertiserIdList:
#     try:
#         # Print the current advertiser Id
#         print(f"Processing advertiser Id: {AID}")

#         # Retrieve the list of insertionOrder IDs for the given advertiser
#         insertionOrders_response = service.advertisers().insertionOrders().list(
#             advertiserId=AID
#         ).execute()

#         insertionOrderIdList = [insertionOrder['insertionOrderId'] for insertionOrder in insertionOrders_response.get('insertionOrders', [])]

#         # Loop through each insertionOrder ID
#         for IOID in insertionOrderIdList:
#             try:
#                 # Print the current insertionOrder Id
#                 print(f"Processing insertionOrder Id: {IOID} for advertiser Id: {AID}")

#                 # Make a single API request to get all targeting types for this insertionOrder
#                 request = service.advertisers().insertionOrders().listAssignedTargetingOptions(
#                     advertiserId=AID,
#                     insertionOrderId=IOID
#                 )
#                 response = request.execute()

#                 # Initialize a dictionary to hold the data for this advertiser and insertionOrder
#                 new_row = {'advertiserId': AID, 'insertionOrderId': IOID}

#                 # Process each targeting type in the response
#                 if 'assignedTargetingOptions' in response:
#                     insertionOrderListResponse = pd.json_normalize(response['assignedTargetingOptions'])

#                     for targeting_type in targeting_types_list:
#                         if targeting_type in insertionOrderListResponse['targetingType'].values:
#                             # Process the targeting details for the current targeting type
#                             targeting_details = process_targeting_details(insertionOrderListResponse, targeting_type)
#                             new_row[f'{targeting_type}'] = targeting_details
#                         else:
#                             new_row[f'{targeting_type}'] = None  # If the targeting type is not in the response, set to None

#                 # Append the completed row for this advertiser and insertionOrder to the DataFrame
#                 dataFrame_InsertionOrder_AssignedTargetingOptions = pd.concat([dataFrame_InsertionOrder_AssignedTargetingOptions, pd.DataFrame([new_row])], ignore_index=True)

#             except Exception as e:
#                 print(f"An error occurred for IOID {IOID} in advertiser ID {AID}: {e}")

#     except Exception as e:
#         print(f"An error occurred for AID {AID}: {e}")

# # Output the final DataFrame
# print(dataFrame_InsertionOrder_AssignedTargetingOptions)


In [ ]:
# ### OLDER CODE, WITHOUT AssignedTargetingOption

# # Initialize the DataFrame with dynamic columns based on the specified targeting types
# dataFrame_Advertiser_AssignedTargetingOptions = pd.DataFrame({
#     'advertiserId': [],
# })

# # List of targeting types to iterate over (add more targeting types as needed)
# targeting_types_list = [
#     'TARGETING_TYPE_CHANNEL',
#     'TARGETING_TYPE_DIGITAL_CONTENT_LABEL_EXCLUSION',
#     'TARGETING_TYPE_OMID',
#     'TARGETING_TYPE_SENSITIVE_CATEGORY_EXCLUSION',
#     'TARGETING_TYPE_YOUTUBE_VIDEO',
#     'TARGETING_TYPE_YOUTUBE_CHANNEL',
#     'TARGETING_TYPE_KEYWORD'
# ]

# # Dynamically add columns based on the selected targeting types in targeting_types_list
# for targeting_type in targeting_types_list:
#     if targeting_type in targeting_details_mapping:
#         dataFrame_Advertiser_AssignedTargetingOptions[f'{targeting_type}'] = []

# # Function to extract and process targeting details
# def process_targeting_details(advertiserListResponse, targeting_type):
#     targeting_fields = targeting_details_mapping[targeting_type]
#     targeting_details = get_targeting_details(
#         advertiserListResponse,
#         targeting_type,
#         *targeting_fields[:-2]  # Unpack the fields to be passed to the function
#     )
#     sorting_column_index = targeting_fields[-2]
#     targeting_details = sort_by_column(targeting_details, sorting_column_index)
#     return targeting_details

# # Loop through each advertiser Id
# for AID in advertiserIdList:
#     try:
#         # Print the current advertiser Id
#         print(f"Processing advertiser Id: {AID}")

#         # Make a single API request to get all targeting types
#         request = service.advertisers().listAssignedTargetingOptions(
#             advertiserId=AID
#         )
#         response = request.execute()

#         # Initialize a dictionary to hold the data for this advertiser
#         new_row = {'advertiserId': AID}

#         # Process each targeting type in the response
#         if 'assignedTargetingOptions' in response:
#             advertiserListResponse = pd.json_normalize(response['assignedTargetingOptions'])

#             for targeting_type in targeting_types_list:
#                 if targeting_type in advertiserListResponse['targetingType'].values:
#                     # Process the targeting details for the current targeting type
#                     targeting_details = process_targeting_details(advertiserListResponse, targeting_type)
#                     new_row[f'{targeting_type}'] = targeting_details
#                 else:
#                     new_row[f'{targeting_type}'] = None  # If the targeting type is not in the response, set to None

#         # Append the completed row for this advertiser to the DataFrame
#         dataFrame_Advertiser_AssignedTargetingOptions = pd.concat([dataFrame_Advertiser_AssignedTargetingOptions, pd.DataFrame([new_row])], ignore_index=True)

#     except Exception as e:
#         print(f"An error occurred for AID {AID}: {e}")

# # Output the final DataFrame
# print(dataFrame_Advertiser_AssignedTargetingOptions)


# Working code but for later deployment

## Using TARGETING_TYPE as a dataframe for selection of parameters

In [108]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display

# Access the DataFrame for 'TARGETING_TYPE_AGE_RANGE'
df_age_range = targetingType_dataframes_dict.get('dataframe_TARGETING_TYPE_AGE_RANGE')

# Initialize the parameter variable
parameter_TARGETING_TYPE_AGE_RANGE = None

# Check if the DataFrame exists
if df_age_range is not None:
    # Display the DataFrame for reference
    print("Available Age Range Options:")
    display(df_age_range[['targetingOptionId', 'ageRangeDetails.ageRange']])

    # Add "No Option Selected" to the list of age ranges
    age_ranges = ["No Option Selected"] + df_age_range['ageRangeDetails.ageRange'].tolist()
    selection_widget = widgets.SelectMultiple(
        options=age_ranges,
        description="Select Age Ranges:",
        disabled=False
    )

    # Display the selection widget
    display(selection_widget)

    # Button to capture and process the selected options
    def on_button_click(b):
        global parameter_TARGETING_TYPE_AGE_RANGE
        selected_ranges = list(selection_widget.value)

        # Check if "No Option Selected" is chosen, ignoring other selections if so
        if "No Option Selected" in selected_ranges:
            parameter_TARGETING_TYPE_AGE_RANGE = "No options selected"
            print("No options selected.")
        elif selected_ranges:
            # Filter the DataFrame to only include selected age ranges, then sort by targetingOptionId
            selected_df = df_age_range[
                df_age_range['ageRangeDetails.ageRange'].isin(selected_ranges)
            ].sort_values(by='targetingOptionId')

            # Set and display the result
            parameter_TARGETING_TYPE_AGE_RANGE = "TARGETING_TYPE_AGE_RANGE: " + ', '.join(map(str, selected_df['targetingOptionId'].tolist()))
            print("Selected Targeting Options:")
            print(parameter_TARGETING_TYPE_AGE_RANGE)
        else:
            parameter_TARGETING_TYPE_AGE_RANGE = "No options selected"
            print("No options selected.")

    # Button widget
    button = widgets.Button(description="Submit Selection")
    button.on_click(on_button_click)

    # Display the button
    display(button)
else:
    print("DataFrame for TARGETING_TYPE_AGE_RANGE not found.")


Available Age Range Options:


,targetingOptionId,ageRangeDetails.ageRange
0,503001,AGE_RANGE_18_24
1,503002,AGE_RANGE_25_34
2,503003,AGE_RANGE_35_44
3,503004,AGE_RANGE_45_54
4,503005,AGE_RANGE_55_64
5,503006,AGE_RANGE_65_PLUS
6,503999,AGE_RANGE_UNKNOWN


SelectMultiple(description='Select Age Ranges:', options=('No Option Selected', 'AGE_RANGE_18_24', 'AGE_RANGE_…

Button(description='Submit Selection', style=ButtonStyle())

No options selected.
